# Settings



In [1]:
'''To display all output results of a Jupyter cell.'''
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
'''To ensure that the output results of extensive output results are not truncated.'''
#pd.options.display.max_rows = 4000

'To ensure that the output results of extensive output results are not truncated.'

# Import of packages

In [3]:
'''To install the required packages that are not included in Colaboratory.'''
#Installation of packages that are not included in Colaboratory
!pip install kneed
!pip install osmnx

'To install the required packages that are not included in Colaboratory.'

  Using cached matplotlib-3.3.4-cp38-cp38-macosx_10_9_x86_64.whl (8.5 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3


In [4]:
'''To install a specific version of the matplotlib package.'''
!pip install matplotlib==3.1.3

'To install a specific version of the matplotlib package.'

  Using cached matplotlib-3.1.3-cp38-cp38-macosx_10_9_x86_64.whl (13.2 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.3.4
    Uninstalling matplotlib-3.3.4:
      Successfully uninstalled matplotlib-3.3.4
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

osmnx 1.0.1 requires matplotlib>=3.3, but you'll have matplotlib 3.1.3 which is incompatible.


In [5]:
'''To import the required packages.'''
import pandas as pd
import numpy as np
import networkx as nx
import collections
import matplotlib.pyplot as plt
import math

from kneed import KneeLocator
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA

import osmnx as ox

'To import the required packages.'

# **Belgian railway system**

# Import of the Belgian railway datasets

In [6]:
'''To register the GitHub link with the Belgian data as a variable.'''
datalink = "https://raw.githubusercontent.com/polkuleuven/thesis/main/"

'To register the GitHub link with the Belgian data as a variable.'

In [7]:
'''To import the agency dataset that contains limited information about Belgian NMBS/SNCB railway agency.'''
agency = pd.read_csv(datalink + "agency_Belgium.csv", sep=",")
#agency

'To import the agency dataset that contains limited information about Belgian NMBS/SNCB railway agency.'

In [8]:
'''To import the stops dataset that contains information about the ids, the names and the geographical coordinates of the Belgian railway stations.'''
stops = pd.read_csv(datalink + "stops_Belgium.csv", encoding='iso-8859-1', sep=",")
stops

'To import the stops dataset that contains information about the ids, the names and the geographical coordinates of the Belgian railway stations.'

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,platform_code
0,8015345,NaN,Aachen Hbf (DE),NaN,50.77083,6.105277,NaN,NaN,0,NaN,NaN
1,8200100,NaN,Luxembourg (LU),NaN,49.60000,6.133333,NaN,NaN,0,NaN,NaN
2,8200101,NaN,Dommeldange (LU),NaN,49.63390,6.136765,NaN,NaN,0,NaN,NaN
3,8200102,NaN,Pfaffenthal-Kirchberg (LU),NaN,49.61913,6.132853,NaN,NaN,0,NaN,NaN
4,8200110,NaN,Mersch (LU),NaN,49.74889,6.106111,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2636,8896909,NaN,Izegem,NaN,50.92115,3.212089,NaN,NaN,0,S8896909,NaN
2637,S8896925,NaN,Ingelmunster,NaN,50.91433,3.255416,NaN,NaN,1,NaN,NaN
2638,8896925_1,NaN,Ingelmunster,NaN,50.91433,3.255416,NaN,NaN,0,S8896925,1
2639,8896925_2,NaN,Ingelmunster,NaN,50.91433,3.255416,NaN,NaN,0,S8896925,2


In [9]:
'''To import the translations dataset that provides the French-, Dutch-, German- and English-language translations of the Belgian railway stations.'''
translations = pd.read_csv(datalink + "translations_Belgium.csv", encoding='iso-8859-1', sep=",")
#translations

'To import the translations dataset that provides the French-, Dutch-, German- and English-language translations of the Belgian railway stations.'

In [10]:
'''To import the transfers dataset that gives the minimum transfer time to switch routes at each Belgian railway station.'''
transfers = pd.read_csv(datalink + "transfers_Belgium.csv", sep=",")
transfers

'To import the transfers dataset that gives the minimum transfer time to switch routes at each Belgian railway station.'

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_trip_id,to_trip_id
0,8015345,8015345,2,300,NaN,NaN
1,8200100,8200100,2,300,NaN,NaN
2,8200101,8200101,2,300,NaN,NaN
3,8200102,8200102,2,300,NaN,NaN
4,8200110,8200110,2,300,NaN,NaN
...,...,...,...,...,...,...
603,8896503,8896503,2,300,NaN,NaN
604,8896735,8896735,2,300,NaN,NaN
605,8896800,8896800,2,300,NaN,NaN
606,8896909,8896909,2,300,NaN,NaN


In [11]:
'''To import the routes dataset that provides the id, the name and the type of vehicle used for all Belgian railway routes.'''
routes = pd.read_csv(datalink + "routes_Belgium.csv", encoding='iso-8859-1', sep=",")
routes

'To import the routes dataset that provides the id, the name and the type of vehicle used for all Belgian railway routes.'

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,1,NMBS/SNCB,BUS,Lichtervelde -- La Panne,NaN,700,NaN,NaN,NaN
1,10,NMBS/SNCB,BUS,Charleroi-Sud -- Tamines,NaN,700,NaN,NaN,NaN
2,100,NMBS/SNCB,BUS,Gand-Saint-Pierre -- Renaix,NaN,700,NaN,NaN,NaN
3,101,NMBS/SNCB,BUS,Courtrai -- Roulers,NaN,700,NaN,NaN,NaN
4,102,NMBS/SNCB,BUS,Mons -- La LouviÃ¨re-Sud,NaN,700,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
734,95,NMBS/SNCB,BUS,Malines -- Saint-Nicolas,NaN,700,NaN,NaN,NaN
735,96,NMBS/SNCB,BUS,Bruxelles-Midi -- Nivelles,NaN,700,NaN,NaN,NaN
736,97,NMBS/SNCB,BUS,Audenarde -- Renaix,NaN,700,NaN,NaN,NaN
737,98,NMBS/SNCB,BUS,Mons -- Jurbise,NaN,700,NaN,NaN,NaN


In [12]:
'''To import the trips dataset that gives for all routes an overview of the trips and the headsigns of these trips belonging to the Belgian railway route.
The service_id is an indication of all the dates this trip is valid (consultable in the calendar_dates dataset).'''
trips = pd.read_csv(datalink + "trips_Belgium.csv", encoding='iso-8859-1', sep=",")
trips

'To import the trips dataset that gives for all routes an overview of the trips and the headsigns of these trips belonging to the Belgian railway route.\nThe service_id is an indication of all the dates this trip is valid (consultable in the calendar_dates dataset).'

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,trip_type
0,1,1,88____:049::8892338:8892205:6:1925:20210314,Lichtervelde,11000,NaN,1,NaN,1
1,2,2,88____:049::8883006:8881000:8:2446:20210319,Mons,11000,NaN,2,NaN,1
2,3,3,82____:049::8200100:8869054:2:614:20210321,Arlon,11002,NaN,3,NaN,1
3,3,3,88____:049::8869054:8866001:2:624:20210321,Arlon,11002,NaN,3,NaN,1
4,3,3,82____:049::8200100:8869054:2:714:20210321,Arlon,11003,NaN,4,NaN,1
...,...,...,...,...,...,...,...,...,...
30100,736,498,88____:046::8865003:8864345:6:1730:20211211,Marloie,987,NaN,24668,NaN,1
30101,736,218,88____:046::8865003:8864345:6:1830:20211210,Marloie,988,NaN,24669,NaN,1
30102,736,498,88____:046::8865003:8864345:6:1930:20211211,Marloie,989,NaN,24670,NaN,1
30103,736,218,88____:046::8865003:8864345:6:2030:20211210,Marloie,990,NaN,24671,NaN,1


In [13]:
'''To import the stop_times dataset that gives for all trips an overview of the ids of the stations served and the sequence in which these stations are served. 
In addition, for all the trips the arrival and departure times at the stations served are given.'''
stop_times = pd.read_csv(datalink + "stop_times_Belgium.csv", sep=",")
stop_times

'To import the stop_times dataset that gives for all trips an overview of the ids of the stations served and the sequence in which these stations are served. \nIn addition, for all the trips the arrival and departure times at the stations served are given.'

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,88____:049::8892338:8892205:6:1925:20210314,17:35:00,17:35:00,8892338,1,NaN,0,1,NaN
1,88____:049::8892338:8892205:6:1925:20210314,17:50:00,17:50:00,8892320,2,NaN,0,0,NaN
2,88____:049::8892338:8892205:6:1925:20210314,18:05:00,18:05:00,8892304,3,NaN,0,0,NaN
3,88____:049::8892338:8892205:6:1925:20210314,18:35:00,18:35:00,8892452,4,NaN,0,0,NaN
4,88____:049::8892338:8892205:6:1925:20210314,19:05:00,19:05:00,8892403,5,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...
440853,88____:046::8865003:8864345:6:2130:20211211,21:06:00,21:06:00,8865227,2,NaN,0,0,NaN
440854,88____:046::8865003:8864345:6:2130:20211211,21:16:00,21:16:00,8864337,3,NaN,0,0,NaN
440855,88____:046::8865003:8864345:6:2130:20211211,21:21:00,21:21:00,8864311,4,NaN,0,0,NaN
440856,88____:046::8865003:8864345:6:2130:20211211,21:24:00,21:25:00,8864006,5,NaN,0,0,NaN


In [14]:
'''To import the calendar dataset that gives the first and last date of all data observations.'''
calendar = pd.read_csv(datalink + "calendar_Belgium.csv", sep=",")
#calendar

'To import the calendar dataset that gives the first and last date of all data observations.'

In [15]:
'''To import the calendar_dates dataset that gives for each service_id all the exact dates when that service_id is valid.'''
calendar_dates = pd.read_csv(datalink + "calendar_dates_Belgium.csv", sep=",")
#calendar_dates

'To import the calendar_dates dataset that gives for each service_id all the exact dates when that service_id is valid.'

In [16]:
''' ??? '''
stop_time_overrides = pd.read_csv(datalink + "stop_time_overrides_Belgium.csv", sep=",")
stop_time_overrides

' ??? '

,trip_id,stop_sequence,service_id,stop_id
0,88____:007::8885704:8885001:4:523:20210418,2,14,8885753_1
1,88____:007::8885704:8885001:4:523:20210418,4,14,8885001_5
2,88____:007::8885704:8885001:4:623:20210418,2,14,8885753_1
3,88____:007::8885704:8885001:4:623:20210418,4,14,8885001_5
4,88____:007::8885704:8885001:4:723:20210418,2,14,8885753_1
...,...,...,...,...
214673,88____:046::8865003:8864345:6:2130:20211211,3,837,8864337_2
214674,88____:046::8865003:8864345:6:2130:20211211,3,611,8864337_1
214675,88____:046::8865003:8864345:6:2130:20211211,4,498,8864311_2
214676,88____:046::8865003:8864345:6:2130:20211211,5,498,8864006_6


# Cleaning of the Belgian railway data

In [17]:
'''To eliminate the stop_ids in the stops dataset that contain an underscore or that start with a character 'S'. '''
stops_cleaned = stops[(~stops['stop_id'].str.contains('_')) & (~stops['stop_id'].str.contains('S'))]

'''To modify the object datatype of the stop_id column to the numpy int64 datatype'''
stops_cleaned.loc[:,'stop_id'] = stops_cleaned.loc[:,'stop_id'].astype(np.int64)
stops_cleaned

"To eliminate the stop_ids in the stops dataset that contain an underscore or that start with a character 'S'. "

'To modify the object datatype of the stop_id column to the numpy int64 datatype'

/Users/pol/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,platform_code
0,8015345,NaN,Aachen Hbf (DE),NaN,50.77083,6.105277,NaN,NaN,0,NaN,NaN
1,8200100,NaN,Luxembourg (LU),NaN,49.60000,6.133333,NaN,NaN,0,NaN,NaN
2,8200101,NaN,Dommeldange (LU),NaN,49.63390,6.136765,NaN,NaN,0,NaN,NaN
3,8200102,NaN,Pfaffenthal-Kirchberg (LU),NaN,49.61913,6.132853,NaN,NaN,0,NaN,NaN
4,8200110,NaN,Mersch (LU),NaN,49.74889,6.106111,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2622,8896503,NaN,Ypres,NaN,50.84740,2.876590,NaN,NaN,0,S8896503,NaN
2627,8896735,NaN,Poperinge,NaN,50.85445,2.736345,NaN,NaN,0,S8896735,NaN
2632,8896800,NaN,Roulers,NaN,50.94903,3.130415,NaN,NaN,0,S8896800,NaN
2636,8896909,NaN,Izegem,NaN,50.92115,3.212089,NaN,NaN,0,S8896909,NaN


In [18]:
'''To merge a selection of the trips dataset and a selection of the routes dataset on route_id'''
trip_route_short_name = pd.merge(trips[['route_id','service_id','trip_id', 'trip_headsign']], routes[['route_id', 'route_short_name', 'route_long_name']], on='route_id')

'''To select the trips that belong to the routes that have a route_short_name that begins with an 'S' or is equal to 'IC', 'L' or 'P.'''
allowed_route_type = {'IC', 'L', 'P'}
filtered_trips = trip_route_short_name[(trip_route_short_name['route_short_name'].isin(allowed_route_type)) | (trip_route_short_name['route_short_name'].str.startswith('S'))]
filtered_trips = filtered_trips.drop(columns=['route_short_name'])
filtered_trips

'To merge a selection of the trips dataset and a selection of the routes dataset on route_id'

"To select the trips that belong to the routes that have a route_short_name that begins with an 'S' or is equal to 'IC', 'L' or 'P."

,route_id,service_id,trip_id,trip_headsign,route_long_name
4726,117,14,88____:007::8885704:8885001:4:523:20210418,Tournai,Tournai -- Mouscron
4727,117,14,88____:007::8885704:8885001:4:623:20210418,Tournai,Tournai -- Mouscron
4728,117,14,88____:007::8885704:8885001:4:723:20210418,Tournai,Tournai -- Mouscron
4729,117,14,88____:007::8885704:8885001:4:823:20210418,Tournai,Tournai -- Mouscron
4730,117,14,88____:007::8885704:8885001:4:923:20210418,Tournai,Tournai -- Mouscron
...,...,...,...,...,...
30100,739,21,88____:007::8821105:8812005:22:1323:20210418,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord
30101,739,21,88____:007::8812005:8400131:23:1618:20210418,Den Haag HS (NL),Den Haag HS (NL) -- Bruxelles-Nord
30102,739,21,84____:007::8400131:8400280:3:1720:20210418,Den Haag HS (NL),Den Haag HS (NL) -- Bruxelles-Nord
30103,739,21,84____:007::8400280:8821105:4:1600:20210418,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord


# Exploratory data analysis with the Belgian railway data

In [19]:
'''To calculate the number of unique route_ids before removing the routes with a route_short_name that does not begin with an S and is not 'IC', 'L', or 'P'.'''
initial_set_routes = {r for r in routes['route_id']}
len(initial_set_routes)

"To calculate the number of unique route_ids before removing the routes with a route_short_name that does not begin with an S and is not 'IC', 'L', or 'P'."

739

In [20]:
'''To calculate the number of unique route_ids after removing the routes with a route_short_name that does not begin with an S and is not 'IC', 'L', or 'P'.'''
set_routes = {r for r in filtered_trips['route_id']}
len(set_routes)

"To calculate the number of unique route_ids after removing the routes with a route_short_name that does not begin with an S and is not 'IC', 'L', or 'P'."

594

In [21]:
'''To calculate the number of stations in the stops_cleaned dataset'''
set_stations = {s for s in stops_cleaned['stop_id']}
len(set_stations)

'To calculate the number of stations in the stops_cleaned dataset'

608

# **French railway system**

# French TGV railway system

## Import of the French TGV railway datasets

In [22]:
'''To import the agency dataset that contains limited information about the French TGV railway agency.'''
agency_TGV = pd.read_csv(datalink + "agency_TGV.csv", sep=",")
#agency_TGV

'To import the agency dataset that contains limited information about the French TGV railway agency.'

In [23]:
'''To import the feed_info dataset that contains limited information about the French TGV railway feed.'''
feed_info_TGV = pd.read_csv(datalink + "feed_info_TGV.csv", sep=",")
#feed_info_TGV

'To import the feed_info dataset that contains limited information about the French TGV railway feed.'

In [24]:
'''To import the stops dataset that contains information about the ids, the names and the geographical coordinates of the French TGV railway stations.'''
stops_TGV = pd.read_csv(datalink + "stops_TGV.csv", sep=",")
stops_TGV

'To import the stops dataset that contains information about the ids, the names and the geographical coordinates of the French TGV railway stations.'

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,StopArea:OCE80143099,Offenburg,NaN,48.476500,7.946800,NaN,NaN,1,NaN
1,StopPoint:OCETGV INOUI-80143099,Offenburg,NaN,48.476500,7.946800,NaN,NaN,0,StopArea:OCE80143099
2,StopArea:OCE80143198,Lahr (Schwarzw),NaN,48.340600,7.835300,NaN,NaN,1,NaN
3,StopPoint:OCETGV INOUI-80143198,Lahr (Schwarzw),NaN,48.340600,7.835300,NaN,NaN,0,StopArea:OCE80143198
4,StopArea:OCE80143313,Emmendingen,NaN,48.119300,7.847900,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...
506,StopArea:OCE87784009,Perpignan,NaN,42.696096,2.879355,NaN,NaN,1,NaN
507,StopPoint:OCETGV INOUI-87784009,Perpignan,NaN,42.696096,2.879355,NaN,NaN,0,StopArea:OCE87784009
508,StopPoint:OCETGV-87784009,Perpignan,NaN,42.696096,2.879355,NaN,NaN,0,StopArea:OCE87784009
509,StopArea:OCE88140010,Bruxelles Midi,NaN,50.836000,4.336000,NaN,NaN,1,NaN


In [25]:
'''To import the transfers dataset that gives the minimum transfer time to switch routes at each French TGV railway station.'''
transfers_TGV = pd.read_csv(datalink + "transfers_TGV.csv", sep=",")
transfers_TGV

'To import the transfers dataset that gives the minimum transfer time to switch routes at each French TGV railway station.'

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_route_id,to_route_id
0,StopPoint:OCETGV INOUI-80143099,StopPoint:OCETGV INOUI-80143099,2,300,NaN,NaN
1,StopPoint:OCETGV INOUI-80143198,StopPoint:OCETGV INOUI-80143198,2,300,NaN,NaN
2,StopPoint:OCETGV INOUI-80143313,StopPoint:OCETGV INOUI-80143313,2,300,NaN,NaN
3,StopPoint:OCETGV INOUI-80143503,StopPoint:OCETGV INOUI-80143503,2,300,NaN,NaN
4,StopPoint:OCETGV INOUI-82001000,StopPoint:OCETGV INOUI-82001000,2,300,NaN,NaN
...,...,...,...,...,...,...
1888,StopPoint:OCEOUIGO-87271494,StopPoint:OCETGV-87271494,2,300,OCESN-1109949,OCESN-1626110
1889,StopPoint:OCEOUIGO-87313882,StopPoint:OCETGV-87313882,2,300,OCESN-1109949,OCESN-1626110
1890,StopPoint:OCEOUIGO-87271494,StopPoint:OCETGV-87271494,2,300,OCESN-1109949,OCESN-1626111
1891,StopPoint:OCEOUIGO-87313882,StopPoint:OCETGV-87313882,2,300,OCESN-1109949,OCESN-1626111


In [26]:
'''To import the routes dataset that provides the id, the name and the type of vehicle used for the French TGV railway routes.'''
routes_TGV = pd.read_csv(datalink + "routes_TGV.csv", sep=",")
routes_TGV

'To import the routes dataset that provides the id, the name and the type of vehicle used for the French TGV railway routes.'

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,OCESN-1041887,OCESN,633,SUD-EST,NaN,2,NaN,NaN,NaN
1,OCESN-1105915,OCESN,333,EST,NaN,2,NaN,NaN,NaN
2,OCESN-1105916,OCESN,433A,ATL-NORD,NaN,2,NaN,NaN,NaN
3,OCESN-1109949,OCESN,533-00,NORD,NaN,2,NaN,NaN,NaN
4,OCESN-1209971,OCESN,803A,Paris - Metz/Luxembourg Allemagne,NaN,2,NaN,NaN,NaN
5,OCESN-1209972,OCESN,807A,Paris - Strasbourg/Alsace Allemagne,NaN,2,NaN,NaN,NaN
6,OCESN-127,OCESN,1,IS NORD-SUD/EST,NaN,2,NaN,NaN,NaN
7,OCESN-131,OCESN,41,Sud de Lyon,NaN,2,NaN,NaN,NaN
8,OCESN-133,OCESN,61,Est - Atlantique,NaN,2,NaN,NaN,NaN
9,OCESN-135,OCESN,251,PARIS - LILLE,NaN,2,NaN,NaN,NaN


In [27]:
'''To import the trips dataset that gives for all routes an overview of the trips and the headsigns of these trips belonging to the French TGV railway route.
The service_id is an indication of all the dates this trip is valid (consultable in the calendar_dates dataset).'''
trips_TGV = pd.read_csv(datalink + "trips_TGV.csv", sep=",")
trips_TGV

'To import the trips dataset that gives for all routes an overview of the trips and the headsigns of these trips belonging to the French TGV railway route.\nThe service_id is an indication of all the dates this trip is valid (consultable in the calendar_dates dataset).'

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,OCESN-1209972,1,OCESN2052F198219:2021-02-19T18:51:30Z,2052,NaN,1,NaN
1,OCESN-1209972,2,OCESN2052F224808:2021-02-20T18:50:55Z,2052,NaN,2,NaN
2,OCESN-1209972,3,OCESN2060F160675:2021-02-19T18:51:30Z,2060,0.0,3,NaN
3,OCESN-1209972,4,OCESN2060F192057:2021-02-21T18:50:52Z,2060,0.0,4,NaN
4,OCESN-1209972,5,OCESN2060F212734:2021-02-21T18:50:52Z,2060,0.0,5,NaN
...,...,...,...,...,...,...,...
3035,OCESN-1461992,1076,OCESN9898F202613:2021-02-21T18:50:52Z,9898,0.0,3036,NaN
3036,OCESN-1461992,117,OCESN9898F223732:2021-02-21T18:50:52Z,9898,0.0,3037,NaN
3037,OCESN-1610112,298,OCESN990F96807:2021-02-21T18:50:52Z,990,0.0,3038,NaN
3038,OCESN-1610112,298,OCESN996F96801:2021-02-21T18:50:52Z,996,0.0,3039,NaN


In [28]:
'''To import the stop_times dataset that gives for all trips an overview of the ids of the stations served and the sequence in which these stations are served. 
In addition, for all the trips the arrival and departure times at the stations served are given.'''
stop_times_TGV = pd.read_csv(datalink + "stop_times_TGV.csv", sep=",")
stop_times_TGV

'To import the stop_times dataset that gives for all trips an overview of the ids of the stations served and the sequence in which these stations are served. \nIn addition, for all the trips the arrival and departure times at the stations served are given.'

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,OCESN2052F198219:2021-02-19T18:51:30Z,17:12:00,17:12:00,StopPoint:OCETGV INOUI-87212027,0,NaN,0,1,NaN
1,OCESN2052F198219:2021-02-19T18:51:30Z,19:05:00,19:05:00,StopPoint:OCETGV INOUI-87113001,1,NaN,1,0,NaN
2,OCESN2052F224808:2021-02-20T18:50:55Z,17:08:00,17:08:00,StopPoint:OCETGV INOUI-87212027,0,NaN,0,1,NaN
3,OCESN2052F224808:2021-02-20T18:50:55Z,19:05:00,19:05:00,StopPoint:OCETGV INOUI-87113001,1,NaN,1,0,NaN
4,OCESN2060F160675:2021-02-19T18:51:30Z,18:55:00,18:55:00,StopPoint:OCETGV INOUI-87212027,0,NaN,0,1,NaN
...,...,...,...,...,...,...,...,...,...
16756,OCESN998F96797:2021-02-20T18:50:55Z,15:04:00,15:09:00,StopPoint:OCETGV-87393702,6,NaN,0,0,NaN
16757,OCESN998F96797:2021-02-20T18:50:55Z,16:02:00,16:06:00,StopPoint:OCETGV-87396002,7,NaN,0,0,NaN
16758,OCESN998F96797:2021-02-20T18:50:55Z,16:42:00,16:46:00,StopPoint:OCETGV-87484006,8,NaN,0,0,NaN
16759,OCESN998F96797:2021-02-20T18:50:55Z,17:24:00,17:28:00,StopPoint:OCETGV-87481002,9,NaN,0,0,NaN


In [29]:
'''To import the calendar dataset that gives the first and last date of all French TGV railway data observations.'''
calendar_TGV = pd.read_csv(datalink + "calendar_TGV.csv", sep=",")
#calendar_TGV

'To import the calendar dataset that gives the first and last date of all French TGV railway data observations.'

In [30]:
'''To import the calendar_dates dataset that gives for each service_id of the French TGV railway system all the exact 
dates when that service_id is valid.'''
calendar_dates_TGV = pd.read_csv(datalink + "calendar_dates_TGV.csv", sep=",")
#calendar_dates_TGV

'To import the calendar_dates dataset that gives for each service_id of the French TGV railway system all the exact \ndates when that service_id is valid.'

## Cleaning of the French TGV railway datasets

In [31]:
''' To clean the stops_TGV df.'''
# To keep only the stop_ids that begin with 'StopPoint:OCETGV-' or 'StopPoint:OCEOUIGO-' or 'StopPoint:OCETGV INOUI'.
stops_filtered_TGV = stops_TGV[stops_TGV['stop_id'].str.startswith(('StopPoint:OCETGV-', 'StopPoint:OCEOUIGO-','StopPoint:OCETGV INOUI'))]

# To make the stop_ids numerical such that a station is recognised in the same way if it appears in several French railway systems
stops_cleaned_TGV_column = stops_filtered_TGV['stop_id'].str.split('-').str[1]
stops_cleaned_TGV = stops_filtered_TGV.copy()
stops_cleaned_TGV['stop_id'] = stops_cleaned_TGV_column
stops_cleaned_TGV.loc[:,'stop_id'] = stops_cleaned_TGV.loc[:,'stop_id'].astype(np.int64)
stops_cleaned_TGV = stops_cleaned_TGV.drop_duplicates()
stops_cleaned_TGV

' To clean the stops_TGV df.'

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
1,80143099,Offenburg,NaN,48.476500,7.946800,NaN,NaN,0,StopArea:OCE80143099
3,80143198,Lahr (Schwarzw),NaN,48.340600,7.835300,NaN,NaN,0,StopArea:OCE80143198
5,80143313,Emmendingen,NaN,48.119300,7.847900,NaN,NaN,0,StopArea:OCE80143313
7,80143503,Freiburg (Breisgau) Hbf,NaN,47.997900,7.841600,NaN,NaN,0,StopArea:OCE80143503
9,82001000,Luxembourg,NaN,49.599722,6.134722,NaN,NaN,0,StopArea:OCE82001000
...,...,...,...,...,...,...,...,...,...
498,87781005,Béziers,NaN,43.336295,3.218895,NaN,NaN,0,StopArea:OCE87781005
501,87781104,Narbonne,NaN,43.190618,3.005667,NaN,NaN,0,StopArea:OCE87781104
504,87781278,Agde,NaN,43.317280,3.466203,NaN,NaN,0,StopArea:OCE87781278
507,87784009,Perpignan,NaN,42.696096,2.879355,NaN,NaN,0,StopArea:OCE87784009


In [32]:
''' To clean the stop_times_TGV df.'''
# To make the stop_ids numerical such that a station is recognised in the same way if it appears in several French railway systems
stop_times_cleaned_TGV_column = stop_times_TGV['stop_id'].str.split('-').str[1]
stop_times_cleaned_TGV = stop_times_TGV.copy()
stop_times_cleaned_TGV['stop_id'] = stop_times_cleaned_TGV_column
stop_times_cleaned_TGV.loc[:,'stop_id'] = stop_times_cleaned_TGV.loc[:,'stop_id'].astype(np.int64)
stop_times_cleaned_TGV

' To clean the stop_times_TGV df.'

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,OCESN2052F198219:2021-02-19T18:51:30Z,17:12:00,17:12:00,87212027,0,NaN,0,1,NaN
1,OCESN2052F198219:2021-02-19T18:51:30Z,19:05:00,19:05:00,87113001,1,NaN,1,0,NaN
2,OCESN2052F224808:2021-02-20T18:50:55Z,17:08:00,17:08:00,87212027,0,NaN,0,1,NaN
3,OCESN2052F224808:2021-02-20T18:50:55Z,19:05:00,19:05:00,87113001,1,NaN,1,0,NaN
4,OCESN2060F160675:2021-02-19T18:51:30Z,18:55:00,18:55:00,87212027,0,NaN,0,1,NaN
...,...,...,...,...,...,...,...,...,...
16756,OCESN998F96797:2021-02-20T18:50:55Z,15:04:00,15:09:00,87393702,6,NaN,0,0,NaN
16757,OCESN998F96797:2021-02-20T18:50:55Z,16:02:00,16:06:00,87396002,7,NaN,0,0,NaN
16758,OCESN998F96797:2021-02-20T18:50:55Z,16:42:00,16:46:00,87484006,8,NaN,0,0,NaN
16759,OCESN998F96797:2021-02-20T18:50:55Z,17:24:00,17:28:00,87481002,9,NaN,0,0,NaN


In [33]:
''' To clean the transfers_TGV df.'''
# To make the stop_ids numerical such that a station is recognised in the same way if it appears in several French railway systems
transfers_cleaned_TGV_column1 = transfers_TGV['from_stop_id'].str.split('-').str[1]
transfers_cleaned_TGV_column2 = transfers_TGV['to_stop_id'].str.split('-').str[1]
transfers_cleaned_TGV = transfers_TGV.copy()
transfers_cleaned_TGV['from_stop_id'] = transfers_cleaned_TGV_column1
transfers_cleaned_TGV['to_stop_id'] = transfers_cleaned_TGV_column2
transfers_cleaned_TGV.loc[:,'from_stop_id'] = transfers_cleaned_TGV.loc[:,'from_stop_id'].astype(np.int64)
transfers_cleaned_TGV.loc[:,'to_stop_id'] = transfers_cleaned_TGV.loc[:,'to_stop_id'].astype(np.int64)
transfers_cleaned_TGV

' To clean the transfers_TGV df.'

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_route_id,to_route_id
0,80143099,80143099,2,300,NaN,NaN
1,80143198,80143198,2,300,NaN,NaN
2,80143313,80143313,2,300,NaN,NaN
3,80143503,80143503,2,300,NaN,NaN
4,82001000,82001000,2,300,NaN,NaN
...,...,...,...,...,...,...
1888,87271494,87271494,2,300,OCESN-1109949,OCESN-1626110
1889,87313882,87313882,2,300,OCESN-1109949,OCESN-1626110
1890,87271494,87271494,2,300,OCESN-1109949,OCESN-1626111
1891,87313882,87313882,2,300,OCESN-1109949,OCESN-1626111


In [34]:
''' To clean the trips_TGV df.'''
# To add starting characters 'TGV-' to the service_id column to avoid confusion when the same service_id numbers are used 
# for different dates in datasets of different railway systems
trips_cleaned_TGV_column = 'TGV-' + trips_TGV['service_id'].astype(str)
trips_cleaned_TGV = trips_TGV.copy()
trips_cleaned_TGV['service_id'] = trips_cleaned_TGV_column
trips_cleaned_TGV

' To clean the trips_TGV df.'

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,OCESN-1209972,TGV-1,OCESN2052F198219:2021-02-19T18:51:30Z,2052,NaN,1,NaN
1,OCESN-1209972,TGV-2,OCESN2052F224808:2021-02-20T18:50:55Z,2052,NaN,2,NaN
2,OCESN-1209972,TGV-3,OCESN2060F160675:2021-02-19T18:51:30Z,2060,0.0,3,NaN
3,OCESN-1209972,TGV-4,OCESN2060F192057:2021-02-21T18:50:52Z,2060,0.0,4,NaN
4,OCESN-1209972,TGV-5,OCESN2060F212734:2021-02-21T18:50:52Z,2060,0.0,5,NaN
...,...,...,...,...,...,...,...
3035,OCESN-1461992,TGV-1076,OCESN9898F202613:2021-02-21T18:50:52Z,9898,0.0,3036,NaN
3036,OCESN-1461992,TGV-117,OCESN9898F223732:2021-02-21T18:50:52Z,9898,0.0,3037,NaN
3037,OCESN-1610112,TGV-298,OCESN990F96807:2021-02-21T18:50:52Z,990,0.0,3038,NaN
3038,OCESN-1610112,TGV-298,OCESN996F96801:2021-02-21T18:50:52Z,996,0.0,3039,NaN


In [35]:
''' To clean the calendar_TGV df.'''
# To add starting characters 'TGV-' to the service_id column to avoid confusion when the same service_id numbers are used 
# for different dates in datasets of different railway systems
calendar_cleaned_TGV_column = 'TGV-' + calendar_TGV['service_id'].astype(str)
calendar_cleaned_TGV = calendar_TGV.copy()
calendar_cleaned_TGV['service_id'] = calendar_cleaned_TGV_column
calendar_cleaned_TGV

' To clean the calendar_TGV df.'

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,TGV-1,0,0,0,0,0,0,0,20210221,20210522
1,TGV-2,0,0,0,0,0,0,0,20210221,20210522
2,TGV-3,0,0,0,0,0,0,0,20210221,20210522
3,TGV-4,0,0,0,0,0,0,0,20210221,20210522
4,TGV-5,0,0,0,0,0,0,0,20210221,20210522
...,...,...,...,...,...,...,...,...,...,...
1221,TGV-1222,0,0,0,0,0,0,0,20210221,20210522
1222,TGV-1223,0,0,0,0,0,0,0,20210221,20210522
1223,TGV-1224,0,0,0,0,0,0,0,20210221,20210522
1224,TGV-1225,0,0,0,0,0,0,0,20210221,20210522


In [36]:
''' To clean the calendar_dates_TGV df.'''
# To add starting characters 'TGV-' to the service_id column to avoid confusion when the same service_id numbers are used 
# for different dates in datasets of different railway systems
calendar_dates_cleaned_TGV_column = 'TGV-' + calendar_dates_TGV['service_id'].astype(str)
calendar_dates_cleaned_TGV = calendar_dates_TGV.copy()
calendar_dates_cleaned_TGV['service_id'] = calendar_dates_cleaned_TGV_column
calendar_dates_cleaned_TGV

' To clean the calendar_dates_TGV df.'

,service_id,date,exception_type
0,TGV-1,20210405,1
1,TGV-1,20210411,1
2,TGV-1,20210418,1
3,TGV-1,20210425,1
4,TGV-1,20210502,1
...,...,...,...
23179,TGV-0,20210518,1
23180,TGV-0,20210519,1
23181,TGV-0,20210520,1
23182,TGV-0,20210521,1


# TEST

In [37]:
testTGV1 = pd.merge(trips_cleaned_TGV[['route_id', 'trip_id']], stop_times_cleaned_TGV[['trip_id', 'stop_id', 'stop_sequence']], on=['trip_id'])
testTGV2 = pd.merge(testTGV1, stops_cleaned_TGV[['stop_id', 'stop_name']], on=['stop_id'])
testTGV3 = testTGV2.groupby(['route_id','trip_id', 'stop_sequence'])[['stop_name']].last()

In [38]:
testTGV3

stop_name
route_id      trip_id                               stop_sequence                                
OCESN-1041887 OCESN7802F213086:2021-02-19T13:09:32Z 0                               Lyon Perrache
                                                    1                              Lyon Part Dieu
                                                    2               Paris Gare de Lyon Hall 1 - 2
              OCESN7803F206420:2021-02-19T13:09:32Z 0               Paris Gare de Lyon Hall 1 - 2
                                                    1                              Lyon Part Dieu
...                                                                                           ...
OCESN-645702  OCESN9249F225395:2021-02-21T18:50:52Z 4                                      Modane
                                                    5                                BARDONECCHIA
                                                    6              OULX CESANA CLAVIERE SESTRIERE
                                                    7                            Turin Porta Susa
                                                    8                      MILANO PORTA GARIBALDI

[16761 rows x 1 columns]

## Exploratory data analysis with the French TGV railway datasets

In [39]:
'''To calculate the number of unique route_ids in the French TGV railway system '''
set_routes_TGV = {r for r in routes_TGV['route_id']}
print("There are",len(set_routes_TGV), "unique routes in the TGV railway dataset.")

'To calculate the number of unique route_ids in the French TGV railway system '

There are 31 unique routes in the TGV railway dataset.


In [40]:
'''To calculate the number of stations in the stops_cleaned_TGV dataset'''
set_stations_TGV = {s for s in stops_cleaned_TGV['stop_id']}
print("There are",len(set_stations_TGV), "unique stations in the TGV railway dataset.")

'To calculate the number of stations in the stops_cleaned_TGV dataset'

There are 188 unique stations in the TGV railway dataset.


# French Intercités railway system

## Import of the French Intercités railway datasets

In [41]:
'''To import the agency dataset that contains limited information about the French Intercités railway agency.'''
agency_Intercites = pd.read_csv(datalink + "agency_Intercites.csv", sep=",")
#agency_Intercites

'To import the agency dataset that contains limited information about the French Intercités railway agency.'

In [42]:
'''To import the feed_info dataset that contains limited information about the French Intercités railway feed.'''
feed_info_Intercites = pd.read_csv(datalink + "feed_info_Intercites.csv", sep=",")
#feed_info_Intercites

'To import the feed_info dataset that contains limited information about the French Intercités railway feed.'

In [43]:
'''To import the stops dataset that contains information about the ids, the names and the geographical coordinates of the French Intercités railway stations.'''
stops_Intercites = pd.read_csv(datalink + "stops_Intercites.csv", sep=",")
stops_Intercites

'To import the stops dataset that contains information about the ids, the names and the geographical coordinates of the French Intercités railway stations.'

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,StopArea:OCE87481002,Nantes,NaN,47.217491,-1.542077,NaN,NaN,1,NaN
1,StopPoint:OCEINTERCITES-87481002,Nantes,NaN,47.217491,-1.542077,NaN,NaN,0,StopArea:OCE87481002
2,StopArea:OCE87484006,Angers Saint-Laud,NaN,47.464647,-0.556820,NaN,NaN,1,NaN
3,StopPoint:OCEINTERCITES-87484006,Angers Saint-Laud,NaN,47.464647,-0.556820,NaN,NaN,0,StopArea:OCE87484006
4,StopArea:OCE87485003,La Rochelle,NaN,46.152705,-1.145853,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...
270,StopPoint:OCEINTERCITES de nuit-87784264,Port-Vendres,NaN,42.513608,3.102576,NaN,NaN,0,StopArea:OCE87784264
271,StopArea:OCE87784298,Banyuls-sur-Mer,NaN,42.482958,3.125050,NaN,NaN,1,NaN
272,StopPoint:OCEINTERCITES de nuit-87784298,Banyuls-sur-Mer,NaN,42.482958,3.125050,NaN,NaN,0,StopArea:OCE87784298
273,StopArea:OCE87785006,Cerbère,NaN,42.441773,3.163403,NaN,NaN,1,NaN


In [44]:
'''To import the transfers dataset that gives the minimum transfer time to switch routes at each French Intercités railway station.'''
transfers_Intercites = pd.read_csv(datalink + "transfers_Intercites.csv", sep=",")
transfers_Intercites

'To import the transfers dataset that gives the minimum transfer time to switch routes at each French Intercités railway station.'

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_route_id,to_route_id
0,StopPoint:OCEINTERCITES-87481002,StopPoint:OCEINTERCITES-87481002,2,300,NaN,NaN
1,StopPoint:OCEINTERCITES-87484006,StopPoint:OCEINTERCITES-87484006,2,300,NaN,NaN
2,StopPoint:OCEINTERCITES-87485003,StopPoint:OCEINTERCITES-87485003,2,300,NaN,NaN
3,StopPoint:OCEINTERCITES-87485144,StopPoint:OCEINTERCITES-87485144,2,300,NaN,NaN
4,StopPoint:OCEINTERCITES-87486019,StopPoint:OCEINTERCITES-87486019,2,300,NaN,NaN
...,...,...,...,...,...,...
153,StopPoint:OCEINTERCITES-87611004,StopPoint:OCEINTERCITES de nuit-87611004,2,300,OCESN-16628,OCESN-1289990
154,StopPoint:OCEINTERCITES-87611244,StopPoint:OCEINTERCITES de nuit-87611244,2,300,OCESN-16628,OCESN-1289990
155,StopPoint:OCEINTERCITES-87613000,StopPoint:OCEINTERCITES de nuit-87613000,2,300,OCESN-16628,OCESN-1289990
156,StopPoint:OCEINTERCITES-87613141,StopPoint:OCEINTERCITES de nuit-87613141,2,300,OCESN-16628,OCESN-1289990


In [45]:
'''To import the routes dataset that provides the id, the name and the type of vehicle used for the French Intercités railway routes.'''
routes_Intercites = pd.read_csv(datalink + "routes_Intercites.csv", sep=",")
routes_Intercites

'To import the routes dataset that provides the id, the name and the type of vehicle used for the French Intercités railway routes.'

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,OCESN-1289990,OCESN,555,Paris-Toulouse-Cerbère/Latour de Carol/Rodez/Albi,NaN,2,NaN,NaN,NaN
1,OCESN-16628,OCESN,560,Paris-Limoges-Toulouse,NaN,2,NaN,NaN,NaN
2,OCESN-645663,OCESN,722,Paris-Clermont Fd,NaN,2,NaN,NaN,NaN
3,OCESN-645666,OCESN,723,Clermont Fd-Béziers,NaN,2,NaN,NaN,NaN
4,OCESN-87481002-87485003,OCEdefault,591,Nantes - La Rochelle,NaN,2,NaN,NaN,NaN
5,OCESN-87481002-87722025,OCEdefault,121,Nantes - Lyon Perrache,NaN,2,NaN,NaN,NaN
6,OCESN-87547000-87742007,OCEdefault,770,Paris Austerlitz - Modane,NaN,2,NaN,NaN,NaN
7,OCESN-87611004-87673004,OCEdefault,552,Toulouse Matabiau - Bayonne,NaN,2,NaN,NaN,NaN
8,OCESN-87751008-87611004,OCEdefault,180,Marseille Saint-Charles - Toulouse Matabiau,NaN,2,NaN,NaN,NaN


In [46]:
'''To import the trips dataset that gives for all routes an overview of the trips and the headsigns of these trips belonging to the French Intercités railway route.
The service_id is an indication of all the dates this trip is valid (consultable in the calendar_dates dataset).'''
trips_Intercites = pd.read_csv(datalink + "trips_Intercites.csv", sep=",")
trips_Intercites

'To import the trips dataset that gives for all routes an overview of the trips and the headsigns of these trips belonging to the French Intercités railway route.\nThe service_id is an indication of all the dates this trip is valid (consultable in the calendar_dates dataset).'

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,OCESN-87611004-87673004,1,OCESN14140F201397:2021-03-08T18:51:30Z,14140,0.0,1,NaN
1,OCESN-87611004-87673004,2,OCESN14140F236412:2021-03-09T18:51:45Z,14140,0.0,2,NaN
2,OCESN-87611004-87673004,3,OCESN14140F237288:2021-03-09T18:51:45Z,14140,0.0,3,NaN
3,OCESN-87611004-87673004,4,OCESN14140F254304:2021-03-04T18:51:09Z,14140,0.0,4,NaN
4,OCESN-87611004-87673004,5,OCESN14141F108219:2021-03-08T18:51:30Z,14141,1.0,5,NaN
...,...,...,...,...,...,...,...
444,OCESN-87481002-87485003,133,OCESN62415F139728:2021-02-25T08:49:59Z,62415,NaN,445,NaN
445,OCESN-87481002-87485003,298,OCESN62416F141095:2021-02-25T18:50:56Z,62416,NaN,446,NaN
446,OCESN-87481002-87485003,133,OCESN62417F128148:2021-02-25T08:49:59Z,62417,NaN,447,NaN
447,OCESN-87481002-87485003,296,OCESN62418F231420:2021-03-09T18:51:45Z,62418,NaN,448,NaN


In [47]:
'''To import the stop_times dataset that gives for all trips an overview of the ids of the stations served and the sequence in which these stations are served. 
In addition, for all the trips the arrival and departure times at the stations served are given.'''
stop_times_Intercites = pd.read_csv(datalink + "stop_times_TER.csv", sep=",")
stop_times_Intercites

'To import the stop_times dataset that gives for all trips an overview of the ids of the stations served and the sequence in which these stations are served. \nIn addition, for all the trips the arrival and departure times at the stations served are given.'

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,OCESN13101F274333:2021-03-09T18:51:45Z,05:46:00,05:46:00,StopPoint:OCETrain TER-87384008,0,NaN,0,1,NaN
1,OCESN13101F274333:2021-03-09T18:51:45Z,06:34:00,06:36:00,StopPoint:OCETrain TER-87415604,1,NaN,0,0,NaN
2,OCESN13101F274333:2021-03-09T18:51:45Z,06:43:00,06:44:00,StopPoint:OCETrain TER-87415620,2,NaN,0,0,NaN
3,OCESN13101F274333:2021-03-09T18:51:45Z,06:58:00,06:59:00,StopPoint:OCETrain TER-87415877,3,NaN,0,0,NaN
4,OCESN13101F274333:2021-03-09T18:51:45Z,07:07:00,07:08:00,StopPoint:OCETrain TER-87411207,4,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...
177818,OCESN96789F261181:2021-03-09T18:51:45Z,06:13:00,06:14:00,StopPoint:OCETrain TER-85010033,3,NaN,0,0,NaN
177819,OCESN96789F261181:2021-03-09T18:51:45Z,06:15:00,06:16:00,StopPoint:OCETrain TER-85010009,4,NaN,0,0,NaN
177820,OCESN96789F261181:2021-03-09T18:51:45Z,06:17:00,06:18:00,StopPoint:OCETrain TER-85010066,5,NaN,0,0,NaN
177821,OCESN96789F261181:2021-03-09T18:51:45Z,06:21:00,06:22:00,StopPoint:OCETrain TER-85010074,6,NaN,0,0,NaN


In [48]:
'''To import the calendar dataset that gives the first and last date of all French Intercités railway data observations.'''
calendar_Intercites = pd.read_csv(datalink + "calendar_Intercites.csv", sep=",")
#calendar_Intercites

'To import the calendar dataset that gives the first and last date of all French Intercités railway data observations.'

In [49]:
'''To import the calendar_dates dataset that gives for each service_id of the French Intercités railway system all the exact 
dates when that service_id is valid.'''
calendar_dates_Intercites = pd.read_csv(datalink + "calendar_dates_Intercites.csv", sep=",")
#calendar_dates_Intercites

'To import the calendar_dates dataset that gives for each service_id of the French Intercités railway system all the exact \ndates when that service_id is valid.'

## Cleaning of the French Intercités railway datasets

In [50]:
''' To clean the stops_Intercites df.'''
# To keep only the stop_ids that begin with 'StopPoint:OCEINTERCITES de nuit-' or 'StopPoint:OCEINTERCITES-'.
stops_filtered_Intercites = stops_Intercites[stops_Intercites['stop_id'].str.startswith(('StopPoint:OCEINTERCITES de nuit-', 'StopPoint:OCEINTERCITES-'))]

# To make the stop_ids numerical such that a station is recognised in the same way if it appears in several French railway systems
stops_cleaned_Intercites_column = stops_filtered_Intercites['stop_id'].str.split('-').str[1]
stops_cleaned_Intercites = stops_filtered_Intercites.copy()
stops_cleaned_Intercites['stop_id'] = stops_cleaned_Intercites_column
stops_cleaned_Intercites.loc[:,'stop_id'] = stops_cleaned_Intercites.loc[:,'stop_id'].astype(np.int64)
stops_cleaned_Intercites = stops_cleaned_Intercites.drop_duplicates()
stops_cleaned_Intercites

' To clean the stops_Intercites df.'

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
1,87481002,Nantes,NaN,47.217491,-1.542077,NaN,NaN,0,StopArea:OCE87481002
3,87484006,Angers Saint-Laud,NaN,47.464647,-0.556820,NaN,NaN,0,StopArea:OCE87484006
5,87485003,La Rochelle,NaN,46.152705,-1.145853,NaN,NaN,0,StopArea:OCE87485003
7,87485144,Rochefort,NaN,45.947189,-0.963556,NaN,NaN,0,StopArea:OCE87485144
9,87486019,La Roche-sur-Yon,NaN,46.672075,-1.435660,NaN,NaN,0,StopArea:OCE87486019
...,...,...,...,...,...,...,...,...,...
266,87784231,Argelès-sur-Mer,NaN,42.543908,3.021919,NaN,NaN,0,StopArea:OCE87784231
268,87784256,Collioure,NaN,42.526928,3.078202,NaN,NaN,0,StopArea:OCE87784256
270,87784264,Port-Vendres,NaN,42.513608,3.102576,NaN,NaN,0,StopArea:OCE87784264
272,87784298,Banyuls-sur-Mer,NaN,42.482958,3.125050,NaN,NaN,0,StopArea:OCE87784298


In [51]:
''' To clean the stop_times_Intercites df.'''
# To make the stop_ids numerical such that a station is recognised in the same way if it appears in several French railway systems
stop_times_cleaned_Intercites_column = stop_times_Intercites['stop_id'].str.split('-').str[1]
stop_times_cleaned_Intercites = stop_times_Intercites.copy()
stop_times_cleaned_Intercites['stop_id'] = stop_times_cleaned_Intercites_column
stop_times_cleaned_Intercites.loc[:,'stop_id'] = stop_times_cleaned_Intercites.loc[:,'stop_id'].astype(np.int64)
stop_times_cleaned_Intercites

' To clean the stop_times_Intercites df.'

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,OCESN13101F274333:2021-03-09T18:51:45Z,05:46:00,05:46:00,87384008,0,NaN,0,1,NaN
1,OCESN13101F274333:2021-03-09T18:51:45Z,06:34:00,06:36:00,87415604,1,NaN,0,0,NaN
2,OCESN13101F274333:2021-03-09T18:51:45Z,06:43:00,06:44:00,87415620,2,NaN,0,0,NaN
3,OCESN13101F274333:2021-03-09T18:51:45Z,06:58:00,06:59:00,87415877,3,NaN,0,0,NaN
4,OCESN13101F274333:2021-03-09T18:51:45Z,07:07:00,07:08:00,87411207,4,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...
177818,OCESN96789F261181:2021-03-09T18:51:45Z,06:13:00,06:14:00,85010033,3,NaN,0,0,NaN
177819,OCESN96789F261181:2021-03-09T18:51:45Z,06:15:00,06:16:00,85010009,4,NaN,0,0,NaN
177820,OCESN96789F261181:2021-03-09T18:51:45Z,06:17:00,06:18:00,85010066,5,NaN,0,0,NaN
177821,OCESN96789F261181:2021-03-09T18:51:45Z,06:21:00,06:22:00,85010074,6,NaN,0,0,NaN


In [52]:
''' To clean the transfers_Intercites df.'''
# To make the stop_ids numerical such that a station is recognised in the same way if it appears in several French railway systems
transfers_cleaned_Intercites_column1 = transfers_Intercites['from_stop_id'].str.split('-').str[1]
transfers_cleaned_Intercites_column2 = transfers_Intercites['to_stop_id'].str.split('-').str[1]
transfers_cleaned_Intercites = transfers_Intercites.copy()
transfers_cleaned_Intercites['from_stop_id'] = transfers_cleaned_Intercites_column1
transfers_cleaned_Intercites['to_stop_id'] = transfers_cleaned_Intercites_column2
transfers_cleaned_Intercites.loc[:,'from_stop_id'] = transfers_cleaned_Intercites.loc[:,'from_stop_id'].astype(np.int64)
transfers_cleaned_Intercites.loc[:,'to_stop_id'] = transfers_cleaned_Intercites.loc[:,'to_stop_id'].astype(np.int64)
transfers_cleaned_Intercites

' To clean the transfers_Intercites df.'

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_route_id,to_route_id
0,87481002,87481002,2,300,NaN,NaN
1,87484006,87484006,2,300,NaN,NaN
2,87485003,87485003,2,300,NaN,NaN
3,87485144,87485144,2,300,NaN,NaN
4,87486019,87486019,2,300,NaN,NaN
...,...,...,...,...,...,...
153,87611004,87611004,2,300,OCESN-16628,OCESN-1289990
154,87611244,87611244,2,300,OCESN-16628,OCESN-1289990
155,87613000,87613000,2,300,OCESN-16628,OCESN-1289990
156,87613141,87613141,2,300,OCESN-16628,OCESN-1289990


In [53]:
''' To clean the trips_Intercites df.'''
# To add starting characters 'Intercites-' to the service_id column to avoid confusion when the same service_id numbers are used 
# for different dates in datasets of different railway systems
trips_cleaned_Intercites_column = 'Intercites-' + trips_Intercites['service_id'].astype(str)
trips_cleaned_Intercites = trips_Intercites.copy()
trips_cleaned_Intercites['service_id'] = trips_cleaned_Intercites_column
trips_cleaned_Intercites

' To clean the trips_Intercites df.'

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,OCESN-87611004-87673004,Intercites-1,OCESN14140F201397:2021-03-08T18:51:30Z,14140,0.0,1,NaN
1,OCESN-87611004-87673004,Intercites-2,OCESN14140F236412:2021-03-09T18:51:45Z,14140,0.0,2,NaN
2,OCESN-87611004-87673004,Intercites-3,OCESN14140F237288:2021-03-09T18:51:45Z,14140,0.0,3,NaN
3,OCESN-87611004-87673004,Intercites-4,OCESN14140F254304:2021-03-04T18:51:09Z,14140,0.0,4,NaN
4,OCESN-87611004-87673004,Intercites-5,OCESN14141F108219:2021-03-08T18:51:30Z,14141,1.0,5,NaN
...,...,...,...,...,...,...,...
444,OCESN-87481002-87485003,Intercites-133,OCESN62415F139728:2021-02-25T08:49:59Z,62415,NaN,445,NaN
445,OCESN-87481002-87485003,Intercites-298,OCESN62416F141095:2021-02-25T18:50:56Z,62416,NaN,446,NaN
446,OCESN-87481002-87485003,Intercites-133,OCESN62417F128148:2021-02-25T08:49:59Z,62417,NaN,447,NaN
447,OCESN-87481002-87485003,Intercites-296,OCESN62418F231420:2021-03-09T18:51:45Z,62418,NaN,448,NaN


In [54]:
''' To clean the calendar_Intercites df.'''
# To add starting characters 'Intercites-' to the service_id column to avoid confusion when the same service_id numbers are used 
# for different dates in datasets of different railway systems
calendar_cleaned_Intercites_column = 'Intercites-' + calendar_Intercites['service_id'].astype(str)
calendar_cleaned_Intercites = calendar_Intercites.copy()
calendar_cleaned_Intercites['service_id'] = calendar_cleaned_Intercites_column
calendar_cleaned_Intercites

' To clean the calendar_Intercites df.'

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,Intercites-1,0,0,0,0,0,0,0,20210309,20210608
1,Intercites-2,0,0,0,0,0,0,0,20210309,20210608
2,Intercites-3,0,0,0,0,0,0,0,20210309,20210608
3,Intercites-4,0,0,0,0,0,0,0,20210309,20210608
4,Intercites-5,0,0,0,0,0,0,0,20210309,20210608
...,...,...,...,...,...,...,...,...,...,...
294,Intercites-295,0,0,0,0,0,0,0,20210309,20210608
295,Intercites-296,0,0,0,0,0,0,0,20210309,20210608
296,Intercites-297,0,0,0,0,0,0,0,20210309,20210608
297,Intercites-298,0,0,0,0,0,0,0,20210309,20210608


In [55]:
''' To clean the calendar_dates_Intercites df.'''
# To add starting characters 'Intercites-' to the service_id column to avoid confusion when the same service_id numbers are used 
# for different dates in datasets of different railway systems
calendar_dates_cleaned_Intercites_column = 'Intercites-' + calendar_dates_Intercites['service_id'].astype(str)
calendar_dates_cleaned_Intercites = calendar_dates_Intercites.copy()
calendar_dates_cleaned_Intercites['service_id'] = calendar_dates_cleaned_Intercites_column
calendar_dates_cleaned_Intercites

' To clean the calendar_dates_Intercites df.'

,service_id,date,exception_type
0,Intercites-1,20210523,1
1,Intercites-2,20210312,1
2,Intercites-2,20210315,1
3,Intercites-2,20210319,1
4,Intercites-2,20210329,1
...,...,...,...
5254,Intercites-0,20210604,1
5255,Intercites-0,20210605,1
5256,Intercites-0,20210606,1
5257,Intercites-0,20210607,1


# TEST

In [56]:
test3 = pd.merge(stop_times_cleaned_Intercites[['trip_id', 'stop_id', 'stop_sequence']], stops_cleaned_Intercites[['stop_id', 'stop_name']], on=['stop_id'])
test4 = pd.merge(trips_cleaned_Intercites[['route_id', 'trip_id']], stop_times_cleaned_Intercites[['trip_id', 'stop_id', 'stop_sequence']], on=['trip_id'])
test4

,route_id,trip_id,stop_id,stop_sequence


## Exploratory data analysis with the French Intercités railway datasets

In [57]:
'''To calculate the number of unique route_ids in the French Intercités railway system '''
set_routes_Intercites = {r for r in routes_Intercites['route_id']}
print("There are",len(set_routes_Intercites), "unique routes in the Intercités railway dataset.")

'To calculate the number of unique route_ids in the French Intercités railway system '

There are 9 unique routes in the Intercités railway dataset.


In [58]:
'''To calculate the number of stations in the stops_cleaned_Intercites dataset'''
set_stations_Intercites = {s for s in stops_cleaned_Intercites['stop_id']}
print("There are",len(set_stations_Intercites), "unique stations in the Intercites railway dataset.")

'To calculate the number of stations in the stops_cleaned_Intercites dataset'

There are 133 unique stations in the Intercites railway dataset.


# French TER railway system

## Import of the French TER railway datasets

In [59]:
'''To import the agency dataset that contains limited information about the French TER railway agency.'''
agency_TER = pd.read_csv(datalink + "agency_TER.csv", sep=",")
#agency_TER

'To import the agency dataset that contains limited information about the French TER railway agency.'

In [60]:
'''To import the feed_info dataset that contains limited information about the French TER railway feed.'''
feed_info_TER = pd.read_csv(datalink + "feed_info_TER.csv", sep=",")
#feed_info_TER

'To import the feed_info dataset that contains limited information about the French TER railway feed.'

In [61]:
'''To import the stops dataset that contains information about the ids, the names and the geographical coordinates of the French TER railway stations.'''
stops_TER = pd.read_csv(datalink + "stops_TER.csv", sep=",")
stops_TER

'To import the stops dataset that contains information about the ids, the names and the geographical coordinates of the French TER railway stations.'

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,StopArea:OCE80142893,Appenweier,NaN,48.5419,7.9735,NaN,NaN,1,NaN
1,StopPoint:OCETrain TER-80142893,Appenweier,NaN,48.5419,7.9735,NaN,NaN,0,StopArea:OCE80142893
2,StopArea:OCE80142901,Legelshurst,NaN,48.5585,7.9138,NaN,NaN,1,NaN
3,StopPoint:OCETrain TER-80142901,Legelshurst,NaN,48.5585,7.9138,NaN,NaN,0,StopArea:OCE80142901
4,StopArea:OCE80142919,Kork,NaN,48.5702,7.8744,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...
8641,StopArea:OCE88857040,Mouscron,NaN,50.7380,3.2250,NaN,NaN,1,NaN
8642,StopPoint:OCECar TER-88857040,Mouscron,NaN,50.7380,3.2250,NaN,NaN,0,StopArea:OCE88857040
8643,StopPoint:OCETrain TER-88857040,Mouscron,NaN,50.7380,3.2250,NaN,NaN,0,StopArea:OCE88857040
8644,StopArea:OCE88960080,Courtrai,NaN,50.8240,3.2600,NaN,NaN,1,NaN


In [62]:
'''To import the transfers dataset that gives the minimum transfer time to switch routes at each French TER railway station.'''
transfers_TER = pd.read_csv(datalink + "transfers_TER.csv", sep=",")
transfers_TER

'To import the transfers dataset that gives the minimum transfer time to switch routes at each French TER railway station.'

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_route_id,to_route_id
0,StopPoint:OCETrain TER-80142893,StopPoint:OCETrain TER-80142893,2,300,NaN,NaN
1,StopPoint:OCETrain TER-80142901,StopPoint:OCETrain TER-80142901,2,300,NaN,NaN
2,StopPoint:OCETrain TER-80142919,StopPoint:OCETrain TER-80142919,2,300,NaN,NaN
3,StopPoint:OCETrain TER-80142927,StopPoint:OCETrain TER-80142927,2,300,NaN,NaN
4,StopPoint:OCECar TER-80142927,StopPoint:OCECar TER-80142927,2,300,NaN,NaN
...,...,...,...,...,...,...
19836,StopPoint:OCENavette-87175042,StopPoint:OCECar TER-87175042,2,180,OCESN-617068,OCESN-355
19837,StopPoint:OCENavette-87175257,StopPoint:OCECar TER-87175257,2,180,OCESN-617068,OCESN-355
19838,StopPoint:OCENavette-87175042,StopPoint:OCECar TER-87175042,2,180,OCESN-617068,OCESN-360
19839,StopPoint:OCENavette-87175778,StopPoint:OCECar TER-87175778,2,180,OCESN-617068,OCESN-362


In [63]:
'''To import the routes dataset that provides the id, the name and the type of vehicle used for the French TER railway routes.'''
routes_TER = pd.read_csv(datalink + "routes_TER.csv", sep=",")
routes_TER

'To import the routes dataset that provides the id, the name and the type of vehicle used for the French TER railway routes.'

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,OCESN-1,OCESN,JJ01,Rennes - Brest,NaN,2,NaN,NaN,NaN
1,OCESN-10,OCESN,JJ24,Dol - Saint Brieuc,NaN,2,NaN,NaN,NaN
2,OCESN-1021888,OCESN,H09,PARIS-DIJON - MULHOUSE - TER GRAND EST,NaN,2,NaN,NaN,NaN
3,OCESN-1065887,OCESN,110,Navettes BUS TGV HPI,NaN,3,NaN,NaN,NaN
4,OCESN-1081890,OCESN,CTX03,Cars travaux LYP - GCO,NaN,3,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
540,OCESN-FR:Line::3DCDF960-BA93-4443-899F-283A2A1...,OCESN,LEX04,ANNEMASSE - COPPET,NaN,2,NaN,NaN,NaN
541,OCESN-FR:Line::608A9B0B-6DF0-475B-82BA-E9792D7...,OCESN,LEX02,ANNECY - COPPET,NaN,3,NaN,NaN,NaN
542,OCESN-FR:Line::7EF9EA42-DEB3-4717-ABDF-2E1B8BC...,OCESN,LEX01,EVIAN - COPPET,NaN,3,NaN,NaN,NaN
543,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,OCESN,LEX06,BELLEGARDE - GENEVE,NaN,2,NaN,NaN,NaN


In [64]:
'''To import the trips dataset that gives for all routes an overview of the trips and the headsigns of these trips belonging to the French TER railway route.
The service_id is an indication of all the dates this trip is valid (consultable in the calendar_dates dataset).'''
trips_TER = pd.read_csv(datalink + "trips_TER.csv", sep=",")
trips_TER

'To import the trips dataset that gives for all routes an overview of the trips and the headsigns of these trips belonging to the French TER railway route.\nThe service_id is an indication of all the dates this trip is valid (consultable in the calendar_dates dataset).'

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,OCESN-1998130,1,OCESN13101F274333:2021-03-09T18:51:45Z,13101,1.0,1,NaN
1,OCESN-1998130,1,OCESN13102F275321:2021-03-09T18:51:45Z,13102,0.0,2,NaN
2,OCESN-1998130,2,OCESN13103F270470:2021-03-07T18:51:13Z,13103,1.0,3,NaN
3,OCESN-1998130,1,OCESN13103F275317:2021-03-09T18:51:45Z,13103,1.0,4,NaN
4,OCESN-1998130,3,OCESN13103F275331:2021-03-09T18:51:45Z,13103,1.0,5,NaN
...,...,...,...,...,...,...,...
20670,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,2785,OCESN96782F261466:2021-03-09T18:51:45Z,96782,NaN,20671,NaN
20671,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,2785,OCESN96783F261183:2021-03-01T18:51:13Z,96783,NaN,20672,NaN
20672,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,2785,OCESN96784F261182:2021-03-08T18:51:30Z,96784,NaN,20673,NaN
20673,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,2796,OCESN96785F256122:2021-03-07T18:51:13Z,96785,NaN,20674,NaN


In [65]:
'''To import the stop_times dataset that gives for all trips an overview of the ids of the stations served and the sequence in which these stations are served. 
In addition, for all the trips the arrival and departure times at the stations served are given.'''
stop_times_TER = pd.read_csv(datalink + "stop_times_TER.csv", sep=",")
stop_times_TER

'To import the stop_times dataset that gives for all trips an overview of the ids of the stations served and the sequence in which these stations are served. \nIn addition, for all the trips the arrival and departure times at the stations served are given.'

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,OCESN13101F274333:2021-03-09T18:51:45Z,05:46:00,05:46:00,StopPoint:OCETrain TER-87384008,0,NaN,0,1,NaN
1,OCESN13101F274333:2021-03-09T18:51:45Z,06:34:00,06:36:00,StopPoint:OCETrain TER-87415604,1,NaN,0,0,NaN
2,OCESN13101F274333:2021-03-09T18:51:45Z,06:43:00,06:44:00,StopPoint:OCETrain TER-87415620,2,NaN,0,0,NaN
3,OCESN13101F274333:2021-03-09T18:51:45Z,06:58:00,06:59:00,StopPoint:OCETrain TER-87415877,3,NaN,0,0,NaN
4,OCESN13101F274333:2021-03-09T18:51:45Z,07:07:00,07:08:00,StopPoint:OCETrain TER-87411207,4,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...
177818,OCESN96789F261181:2021-03-09T18:51:45Z,06:13:00,06:14:00,StopPoint:OCETrain TER-85010033,3,NaN,0,0,NaN
177819,OCESN96789F261181:2021-03-09T18:51:45Z,06:15:00,06:16:00,StopPoint:OCETrain TER-85010009,4,NaN,0,0,NaN
177820,OCESN96789F261181:2021-03-09T18:51:45Z,06:17:00,06:18:00,StopPoint:OCETrain TER-85010066,5,NaN,0,0,NaN
177821,OCESN96789F261181:2021-03-09T18:51:45Z,06:21:00,06:22:00,StopPoint:OCETrain TER-85010074,6,NaN,0,0,NaN


In [66]:
'''To import the calendar dataset that gives the first and last date of all French TER railway data observations.'''
calendar_TER = pd.read_csv(datalink + "calendar_TER.csv", sep=",")
#calendar_TER

'To import the calendar dataset that gives the first and last date of all French TER railway data observations.'

In [67]:
'''To import the calendar_dates dataset that gives for each service_id of the French TER railway system all the exact 
dates when that service_id is valid.'''
calendar_dates_TER = pd.read_csv(datalink + "calendar_dates_TER.csv", sep=",")
#calendar_dates_TER

'To import the calendar_dates dataset that gives for each service_id of the French TER railway system all the exact \ndates when that service_id is valid.'

## Cleaning of the French TER railway datasets

In [68]:
''' To clean the stops_TER df.'''
# To keep only the stop_ids that begin with 'StopPoint:OCETrain TER-'
stops_filtered_TER = stops_TER[stops_TER['stop_id'].str.startswith('StopPoint:OCETrain TER-')]

# To make the stop_ids numerical such that a station is recognised in the same way if it appears in several French railway systems
stops_cleaned_TER_column = stops_filtered_TER['stop_id'].str.split('-').str[1]
stops_cleaned_TER = stops_filtered_TER.copy()
stops_cleaned_TER['stop_id'] = stops_cleaned_TER_column
stops_cleaned_TER.loc[:,'stop_id'] = stops_cleaned_TER.loc[:,'stop_id'].astype(np.int64)
stops_cleaned_TER 

' To clean the stops_TER df.'

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
1,80142893,Appenweier,NaN,48.5419,7.9735,NaN,NaN,0,StopArea:OCE80142893
3,80142901,Legelshurst,NaN,48.5585,7.9138,NaN,NaN,0,StopArea:OCE80142901
5,80142919,Kork,NaN,48.5702,7.8744,NaN,NaN,0,StopArea:OCE80142919
8,80142927,Kehl,NaN,48.5767,7.8063,NaN,NaN,0,StopArea:OCE80142927
10,80143099,Offenburg,NaN,48.4765,7.9468,NaN,NaN,0,StopArea:OCE80143099
...,...,...,...,...,...,...,...,...,...
8636,88810000,Mons,NaN,50.4540,3.9420,NaN,NaN,0,StopArea:OCE88810000
8638,88850010,Tournai,NaN,50.6130,3.3960,NaN,NaN,0,StopArea:OCE88850010
8640,88850685,FROYENNES,NaN,50.6290,3.3550,NaN,NaN,0,StopArea:OCE88850685
8643,88857040,Mouscron,NaN,50.7380,3.2250,NaN,NaN,0,StopArea:OCE88857040


In [69]:
''' To clean the stop_times_TER df.'''
# To keep only the stop_ids that begin with 'StopPoint:OCETrain TER-'
stop_times_filtered_TER = stop_times_TER[stop_times_TER['stop_id'].str.startswith('StopPoint:OCETrain TER-')]

# To make the stop_ids numerical such that a station is recognised in the same way if it appears in several French railway systems
stop_times_cleaned_TER_column = stop_times_filtered_TER['stop_id'].str.split('-').str[1]
stop_times_cleaned_TER = stop_times_filtered_TER.copy()
stop_times_cleaned_TER['stop_id'] = stop_times_cleaned_TER_column
stop_times_cleaned_TER.loc[:,'stop_id'] = stop_times_cleaned_TER.loc[:,'stop_id'].astype(np.int64)
stop_times_cleaned_TER 

' To clean the stop_times_TER df.'

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,OCESN13101F274333:2021-03-09T18:51:45Z,05:46:00,05:46:00,87384008,0,NaN,0,1,NaN
1,OCESN13101F274333:2021-03-09T18:51:45Z,06:34:00,06:36:00,87415604,1,NaN,0,0,NaN
2,OCESN13101F274333:2021-03-09T18:51:45Z,06:43:00,06:44:00,87415620,2,NaN,0,0,NaN
3,OCESN13101F274333:2021-03-09T18:51:45Z,06:58:00,06:59:00,87415877,3,NaN,0,0,NaN
4,OCESN13101F274333:2021-03-09T18:51:45Z,07:07:00,07:08:00,87411207,4,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...
177818,OCESN96789F261181:2021-03-09T18:51:45Z,06:13:00,06:14:00,85010033,3,NaN,0,0,NaN
177819,OCESN96789F261181:2021-03-09T18:51:45Z,06:15:00,06:16:00,85010009,4,NaN,0,0,NaN
177820,OCESN96789F261181:2021-03-09T18:51:45Z,06:17:00,06:18:00,85010066,5,NaN,0,0,NaN
177821,OCESN96789F261181:2021-03-09T18:51:45Z,06:21:00,06:22:00,85010074,6,NaN,0,0,NaN


In [70]:
''' To clean the transfers_TER df.'''
# To keep only the stop_ids that begin with 'StopPoint:OCETrain TER-'
transfers_filtered_TER = transfers_TER[transfers_TER['from_stop_id'].str.startswith('StopPoint:OCETrain TER-') & transfers_TER['to_stop_id'].str.startswith('StopPoint:OCETrain TER-')]

# To make the stop_ids numerical such that a station is recognised in the same way if it appears in several French railway systems
transfers_cleaned_TER_column1 = transfers_filtered_TER['from_stop_id'].str.split('-').str[1]
transfers_cleaned_TER_column2 = transfers_filtered_TER['to_stop_id'].str.split('-').str[1]
transfers_cleaned_TER = transfers_filtered_TER.copy()
transfers_cleaned_TER['from_stop_id'] = transfers_cleaned_TER_column1
transfers_cleaned_TER['to_stop_id'] = transfers_cleaned_TER_column2
transfers_cleaned_TER.loc[:,'from_stop_id'] = transfers_cleaned_TER.loc[:,'from_stop_id'].astype(np.int64)
transfers_cleaned_TER.loc[:,'to_stop_id'] = transfers_cleaned_TER.loc[:,'to_stop_id'].astype(np.int64)
transfers_cleaned_TER

' To clean the transfers_TER df.'

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_route_id,to_route_id
0,80142893,80142893,2,300,NaN,NaN
1,80142901,80142901,2,300,NaN,NaN
2,80142919,80142919,2,300,NaN,NaN
3,80142927,80142927,2,300,NaN,NaN
5,80143099,80143099,2,300,NaN,NaN
...,...,...,...,...,...,...
5007,88810000,88810000,2,300,NaN,NaN
5008,88850010,88850010,2,300,NaN,NaN
5009,88850685,88850685,2,300,NaN,NaN
5011,88857040,88857040,2,300,NaN,NaN


In [71]:
''' To clean the routes_TER df.'''
# To keep the routes of trains and to delete the other routes
routes_filtered_TER = routes_TER[routes_TER['route_type'] == 2]
routes_filtered_TER

' To clean the routes_TER df.'

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,OCESN-1,OCESN,JJ01,Rennes - Brest,NaN,2,NaN,NaN,NaN
1,OCESN-10,OCESN,JJ24,Dol - Saint Brieuc,NaN,2,NaN,NaN,NaN
2,OCESN-1021888,OCESN,H09,PARIS-DIJON - MULHOUSE - TER GRAND EST,NaN,2,NaN,NaN,NaN
12,OCESN-11,OCESN,JJ25,Guingamp - Paimpol,NaN,2,NaN,NaN,NaN
13,OCESN-1109941,OCESN,L10,Decize - Nevers - Moulins,NaN,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
520,OCESN-873849,OCESN,L 3A,Auxerre - Dijon,NaN,2,NaN,NaN,NaN
522,OCESN-87746875-87746479,OCEdefault,004,Vallorcine - Saint-Gervais-les-Bains Le Fayet,NaN,2,NaN,NaN,NaN
535,OCESN-909879,OCESN,L 3-4,Is-sur-Tille - Dijon - Les-Laumes,NaN,2,NaN,NaN,NaN
540,OCESN-FR:Line::3DCDF960-BA93-4443-899F-283A2A1...,OCESN,LEX04,ANNEMASSE - COPPET,NaN,2,NaN,NaN,NaN


In [72]:
''' To clean the trips_TER df.'''
# To add starting characters 'TER-' to the service_id column to avoid confusion when the same service_id numbers are used 
# for different dates in datasets of different railway systems
trips_cleaned_TER_column = 'TER-' + trips_TER['service_id'].astype(str)
trips_cleaned_TER = trips_TER.copy()
trips_cleaned_TER['service_id'] = trips_cleaned_TER_column
trips_cleaned_TER

' To clean the trips_TER df.'

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,OCESN-1998130,TER-1,OCESN13101F274333:2021-03-09T18:51:45Z,13101,1.0,1,NaN
1,OCESN-1998130,TER-1,OCESN13102F275321:2021-03-09T18:51:45Z,13102,0.0,2,NaN
2,OCESN-1998130,TER-2,OCESN13103F270470:2021-03-07T18:51:13Z,13103,1.0,3,NaN
3,OCESN-1998130,TER-1,OCESN13103F275317:2021-03-09T18:51:45Z,13103,1.0,4,NaN
4,OCESN-1998130,TER-3,OCESN13103F275331:2021-03-09T18:51:45Z,13103,1.0,5,NaN
...,...,...,...,...,...,...,...
20670,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,TER-2785,OCESN96782F261466:2021-03-09T18:51:45Z,96782,NaN,20671,NaN
20671,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,TER-2785,OCESN96783F261183:2021-03-01T18:51:13Z,96783,NaN,20672,NaN
20672,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,TER-2785,OCESN96784F261182:2021-03-08T18:51:30Z,96784,NaN,20673,NaN
20673,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,TER-2796,OCESN96785F256122:2021-03-07T18:51:13Z,96785,NaN,20674,NaN


In [73]:
''' To clean the calendar_TER df.'''
# To add starting characters 'TER-' to the service_id column to avoid confusion when the same service_id numbers are used 
# for different dates in datasets of different railway systems
calendar_cleaned_TER_column = 'TER-' + calendar_TER['service_id'].astype(str)
calendar_cleaned_TER = calendar_TER.copy()
calendar_cleaned_TER['service_id'] = calendar_cleaned_TER_column
calendar_cleaned_TER

' To clean the calendar_TER df.'

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,TER-1,0,0,0,0,0,0,0,20210309,20210608
1,TER-2,0,0,0,0,0,0,0,20210309,20210608
2,TER-3,0,0,0,0,0,0,0,20210309,20210608
3,TER-4,0,0,0,0,0,0,0,20210309,20210608
4,TER-5,0,0,0,0,0,0,0,20210309,20210608
...,...,...,...,...,...,...,...,...,...,...
2792,TER-2793,0,0,0,0,0,0,0,20210309,20210608
2793,TER-2794,0,0,0,0,0,0,0,20210309,20210608
2794,TER-2795,0,0,0,0,0,0,0,20210309,20210608
2795,TER-2796,0,0,0,0,0,0,0,20210309,20210608


In [74]:
''' To clean the calendar_dates_TER df.'''
# To add starting characters 'TER-' to the service_id column to avoid confusion when the same service_id numbers are used 
# for different dates in datasets of different railway systems
calendar_dates_cleaned_TER_column = 'TER-' + calendar_dates_TER['service_id'].astype(str)
calendar_dates_cleaned_TER = calendar_dates_TER.copy()
calendar_dates_cleaned_TER['service_id'] = calendar_dates_cleaned_TER_column
calendar_dates_cleaned_TER

' To clean the calendar_dates_TER df.'

,service_id,date,exception_type
0,TER-1,20210309,1
1,TER-1,20210310,1
2,TER-1,20210311,1
3,TER-1,20210312,1
4,TER-1,20210315,1
...,...,...,...
82031,TER-0,20210604,1
82032,TER-0,20210605,1
82033,TER-0,20210606,1
82034,TER-0,20210607,1


# TEST

In [75]:
testTER1 = pd.merge(trips_cleaned_TER[['route_id', 'trip_id']], stop_times_cleaned_TER[['trip_id', 'stop_id', 'stop_sequence']], on=['trip_id'])
testTER2 = pd.merge(testTER1, stops_cleaned_TGV[['stop_id', 'stop_name']], on=['stop_id'])
testTER3 = testTER2.groupby(['route_id','trip_id', 'stop_sequence'])[['stop_name']].last()
testTER3

stop_name
route_id                                           trip_id                                 stop_sequence                                  
OCESN-1                                            OCESN855806F275451:2021-03-09T18:51:45Z 0                                         Brest
                                                                                           1                                    Landerneau
                                                                                           3                                       Morlaix
                                                                                           4                               Plouaret Trégor
                                                                                           5                                      Guingamp
...                                                                                                                                    ...
OCESN-FR:Line::E937D19D-BA51-4EC8-9C8E-6B20FD5D... OCESN23486F273242:2021-03-09T18:51:45Z  9                                     Annemasse
                                                   OCESN23488F275001:2021-03-09T18:51:45Z  0              Saint-Gervais-les-Bains Le Fayet
                                                                                           1                Sallanches - Combloux - Megève
                                                                                           3                                        Cluses
                                                                                           9                                     Annemasse

[28767 rows x 1 columns]

## Exploratory data analysis with the French TER railway datasets

In [76]:
'''To calculate the number of unique route_ids in the French TER railway system '''
set_routes_TER = {r for r in routes_filtered_TER['route_id']}
print("There are",len(set_routes_TER), "unique routes in the TER railway dataset.")

'To calculate the number of unique route_ids in the French TER railway system '

There are 286 unique routes in the TER railway dataset.


In [77]:
'''To calculate the number of stations in the stops_cleaned_TER dataset'''
set_stations_TER = {s for s in stops_cleaned_TER['stop_id']}
print("There are",len(set_stations_TER), "unique stations in the TER railway dataset.")

'To calculate the number of stations in the stops_cleaned_TER dataset'

There are 2381 unique stations in the TER railway dataset.


# French Transilien railway system

## Import of the French Transilien railway datasets

In [78]:
'''To import the agency dataset that contains limited information about the French Transilien railway agency.'''
agency_Transilien = pd.read_csv(datalink + "agency_Transilien.csv", sep=",")
#agency_Transilien

'To import the agency dataset that contains limited information about the French Transilien railway agency.'

In [79]:
'''To import the stops dataset that contains information about the ids, the names and the geographical coordinates of the French Transilien railway stations.'''
stops_Transilien = pd.read_csv(datalink + "stops_Transilien.csv", sep=",")
stops_Transilien

'To import the stops dataset that contains information about the ids, the names and the geographical coordinates of the French Transilien railway stations.'

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,StopArea:DUA8727605,Gare d'Ermont - Eaubonne,NaN,48.980146,2.271422,NaN,NaN,1,NaN
1,StopArea:DUA8727658,ERMONT HALTE,NaN,48.990314,2.263270,NaN,NaN,1,NaN
2,StopArea:DUA8727659,Gare de Gros Noyer Saint-Prix,NaN,48.996532,2.259410,NaN,NaN,1,NaN
3,StopArea:DUA8727660,Gare de Saint-Leu-la-Forêt,NaN,49.015699,2.242256,NaN,NaN,1,NaN
4,StopArea:DUA8727662,VAUCELLES,NaN,49.021767,2.229353,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...
1037,StopPoint:DUA8743091,CHAMPGOULIN,NaN,48.791684,3.256354,NaN,NaN,0,StopArea:DUA8743091
1038,StopPoint:DUA8743090,EUSTACHE LENOIR,NaN,48.798279,3.267056,NaN,NaN,0,StopArea:DUA8743090
1039,StopPoint:DUA8743088,JOUY SUR MORIN MONUMENT,NaN,48.790683,3.274062,NaN,NaN,0,StopArea:DUA8743088
1040,StopPoint:DUA8711639,LA FERTE GAUCHER,NaN,48.782711,3.297558,NaN,NaN,0,StopArea:DUA8711639


In [80]:
'''To import the transfers dataset that gives the minimum transfer time to switch routes at each French Transilien railway station.'''
transfers_Transilien = pd.read_csv(datalink + "transfers_Transilien.csv", sep=",")
transfers_Transilien

'To import the transfers dataset that gives the minimum transfer time to switch routes at each French Transilien railway station.'

,from_stop_id,to_stop_id,transfer_type,min_transfer_time
0,StopPoint:DUA8775859,StopPoint:DUA8728189,2,912
1,StopPoint:DUA8754700,StopPoint:DUA8754702,2,60
2,StopPoint:DUA8754702,StopPoint:DUA8754700,2,924
3,StopPoint:DUA8759118,StopPoint:DUA8711300,2,279
4,StopPoint:DUA8759118,StopPoint:DUA8759492,2,946
...,...,...,...,...
95,StopPoint:DUA8727116,StopPoint:DUA8769732,2,966
96,StopPoint:DUA8711371,StopPoint:DUA8775830,2,207
97,StopPoint:DUA8775830,StopPoint:DUA8711371,2,460
98,StopPoint:DUA8727605,StopPoint:DUA8753413,2,249


In [81]:
'''To import the routes dataset that provides the id, the name and the type of vehicle used for the French Transilien railway routes.'''
routes_Transilien = pd.read_csv(datalink + "routes_Transilien.csv", sep=",")
routes_Transilien

'To import the routes dataset that provides the id, the name and the type of vehicle used for the French Transilien railway routes.'

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,DUA800853021,DUA853,H,Gare du Nord - Pontoise / Luzarches / Beauvais...,NaN,2,NaN,844C54,FFFFFF
1,DUA800853521,DUA853,H,Gare du Nord - Pontoise / Luzarches / Beauvais...,NaN,3,NaN,7B4339,FFFFFF
2,DUA800854542,DUA854,L,Gare St-Lazare - Versailles R. Droite / Cergy ...,NaN,3,NaN,7584BC,FFFFFF
3,DUA800854541,DUA854,J,Gare St-Lazare - Ermont Eaubonne / Vernon / Gi...,NaN,3,NaN,CDCD00,FFFFFF
4,DUA800853522,DUA853,K,Gare du Nord - Crépy en Valois,NaN,3,NaN,C7B300,FFFFFF
5,DUA800803571,DUA803,C,Pontoise / Versailles R. Gauche / St-Quentin e...,NaN,3,NaN,FCD946,FFFFFF
6,DUA810801541,DUA801,A,Cergy le Haut - Sartrouville (Bus de substitut...,NaN,3,NaN,D1302F,FFFFFF
7,DUA800805591,DUA805,E,Haussmann Saint-Lazare - Chelles Gournay / Tou...,NaN,3,NaN,BD76A1,FFFFFF
8,DUA800804581,DUA804,D,Creil - Corbeil Essonnes / Melun / Malesherbes,NaN,3,NaN,5E9620,FFFFFF
9,DUA800850509,DUA850,T4,Aulnay sous Bois - Bondy,NaN,3,NaN,FFBF00,FFFFFF


In [82]:
'''To import the trips dataset that gives for all routes an overview of the trips and the headsigns of these trips belonging to the French Transilien railway route.
The service_id is an indication of all the dates this trip is valid (consultable in the calendar_dates dataset).'''
trips_Transilien = pd.read_csv(datalink + "trips_Transilien.csv", sep=",")
trips_Transilien

'To import the trips dataset that gives for all routes an overview of the trips and the headsigns of these trips belonging to the French Transilien railway route.\nThe service_id is an indication of all the dates this trip is valid (consultable in the calendar_dates dataset).'

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id
0,DUA800853021,9307,DUASN124601F03001-1_361260,POTA,0,NaN
1,DUA800853021,9307,DUASN124603F02001-1_361261,POTA,0,NaN
2,DUA800853021,9307,DUASN124605F03001-1_361262,POTA,0,NaN
3,DUA800853021,9307,DUASN124607F01001-1_361263,POTA,0,NaN
4,DUA800853021,9307,DUASN124613F03001-1_361264,POTA,0,NaN
...,...,...,...,...,...,...
34298,DUA800850511,8612,DUASN412671R01001-1_77502,412671,1,NaN
34299,DUA800850511,9699,DUASN850192R01001-1_77503,850192,1,NaN
34300,DUA800850511,9699,DUASN850193R01001-1_77504,850193,1,NaN
34301,DUA800850511,8798,DUASN887131R01001-1_77505,887131,1,NaN


In [83]:
'''To import the stop_times dataset that gives for all trips an overview of the ids of the stations served and the sequence in which these stations are served. 
In addition, for all the trips the arrival and departure times at the stations served are given.'''
stop_times_Transilien = pd.read_csv(datalink + "stop_times_Transilien.csv", sep=",")
stop_times_Transilien

'To import the stop_times dataset that gives for all trips an overview of the ids of the stations served and the sequence in which these stations are served. \nIn addition, for all the trips the arrival and departure times at the stations served are given.'

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type
0,DUASN124601F03001-1_361260,05:45:00,05:45:00,StopPoint:DUA8727605,0,NaN,0,1
1,DUASN124601F03001-1_361260,05:47:00,05:48:00,StopPoint:DUA8727658,1,NaN,0,0
2,DUASN124601F03001-1_361260,05:49:00,05:50:00,StopPoint:DUA8727659,2,NaN,0,0
3,DUASN124601F03001-1_361260,05:52:00,05:53:00,StopPoint:DUA8727660,3,NaN,0,0
4,DUASN124601F03001-1_361260,05:55:00,05:55:00,StopPoint:DUA8727662,4,NaN,0,0
...,...,...,...,...,...,...,...,...
346694,DUASN887133R01001-1_77506,25:51:00,25:51:00,StopPoint:DUA8711369,11,NaN,0,0
346695,DUASN887133R01001-1_77506,26:01:00,26:01:00,StopPoint:DUA8711321,12,NaN,0,0
346696,DUASN887133R01001-1_77506,26:23:00,26:23:00,StopPoint:DUA8711320,13,NaN,0,0
346697,DUASN887133R01001-1_77506,26:36:00,26:36:00,StopPoint:DUA8765479,14,NaN,0,0


In [84]:
'''To import the calendar dataset that gives the first and last date of all French Transilien railway data observations.'''
calendar_Transilien = pd.read_csv(datalink + "calendar_Transilien.csv", sep=",")
#calendar_Transilien

'To import the calendar dataset that gives the first and last date of all French Transilien railway data observations.'

In [85]:
'''To import the calendar_dates dataset that gives for each service_id of the French Transilien railway system all the exact 
dates when that service_id is valid.'''
calendar_dates_Transilien = pd.read_csv(datalink + "calendar_dates_Transilien.csv", sep=",")
#calendar_dates_Transilien

'To import the calendar_dates dataset that gives for each service_id of the French Transilien railway system all the exact \ndates when that service_id is valid.'

## Cleaning of the French Transilien railway datasets

In [86]:
''' To clean the stops_Transilien df.'''
# To keep only the stop_ids that begin with 'StopPoint:DUA'
stops_filtered_Transilien = stops_Transilien[stops_Transilien['stop_id'].str.startswith('StopPoint:DUA')]

# To make the stop_ids numerical such that a station is recognised in the same way if it appears in several French railway systems
stops_cleaned_Transilien_column = stops_filtered_Transilien['stop_id'].str.split('StopPoint:DUA').str[1]
stops_cleaned_Transilien = stops_filtered_Transilien.copy()
stops_cleaned_Transilien['stop_id'] = stops_cleaned_Transilien_column
stops_cleaned_Transilien.loc[:,'stop_id'] = stops_cleaned_Transilien.loc[:,'stop_id'].astype(np.int64)
stops_cleaned_Transilien

' To clean the stops_Transilien df.'

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
507,8727605,ERMONT EAUBONNE,NaN,48.980658,2.271039,NaN,NaN,0,StopArea:DUA8727605
508,8727658,ERMONT HALTE,NaN,48.990314,2.263270,NaN,NaN,0,StopArea:DUA8727658
509,8727659,GROS NOYER SAINT-PRIX,NaN,48.996107,2.259629,NaN,NaN,0,StopArea:DUA8727659
510,8727660,SAINT-LEU LA FORET,NaN,49.015838,2.242563,NaN,NaN,0,StopArea:DUA8727660
511,8727662,VAUCELLES,NaN,49.021767,2.229353,NaN,NaN,0,StopArea:DUA8727662
...,...,...,...,...,...,...,...,...,...
1037,8743091,CHAMPGOULIN,NaN,48.791684,3.256354,NaN,NaN,0,StopArea:DUA8743091
1038,8743090,EUSTACHE LENOIR,NaN,48.798279,3.267056,NaN,NaN,0,StopArea:DUA8743090
1039,8743088,JOUY SUR MORIN MONUMENT,NaN,48.790683,3.274062,NaN,NaN,0,StopArea:DUA8743088
1040,8711639,LA FERTE GAUCHER,NaN,48.782711,3.297558,NaN,NaN,0,StopArea:DUA8711639


In [87]:
''' To clean the stop_times_Transilien df.'''
# To make the stop_ids numerical such that a station is recognised in the same way if it appears in several French railway systems
stop_times_cleaned_Transilien_column = stop_times_Transilien['stop_id'].str.split('StopPoint:DUA').str[1]
stop_times_cleaned_Transilien = stop_times_Transilien.copy()
stop_times_cleaned_Transilien['stop_id'] = stop_times_cleaned_Transilien_column
stop_times_cleaned_Transilien.loc[:,'stop_id'] = stop_times_cleaned_Transilien.loc[:,'stop_id'].astype(np.int64)
stop_times_cleaned_Transilien

' To clean the stop_times_Transilien df.'

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type
0,DUASN124601F03001-1_361260,05:45:00,05:45:00,8727605,0,NaN,0,1
1,DUASN124601F03001-1_361260,05:47:00,05:48:00,8727658,1,NaN,0,0
2,DUASN124601F03001-1_361260,05:49:00,05:50:00,8727659,2,NaN,0,0
3,DUASN124601F03001-1_361260,05:52:00,05:53:00,8727660,3,NaN,0,0
4,DUASN124601F03001-1_361260,05:55:00,05:55:00,8727662,4,NaN,0,0
...,...,...,...,...,...,...,...,...
346694,DUASN887133R01001-1_77506,25:51:00,25:51:00,8711369,11,NaN,0,0
346695,DUASN887133R01001-1_77506,26:01:00,26:01:00,8711321,12,NaN,0,0
346696,DUASN887133R01001-1_77506,26:23:00,26:23:00,8711320,13,NaN,0,0
346697,DUASN887133R01001-1_77506,26:36:00,26:36:00,8765479,14,NaN,0,0


In [88]:
''' To clean the transfers_Transilien df.'''
# To make the stop_ids numerical such that a station is recognised in the same way if it appears in several French railway systems
transfers_cleaned_Transilien_column1 = transfers_Transilien['from_stop_id'].str.split('StopPoint:DUA').str[1]
transfers_cleaned_Transilien_column2 = transfers_Transilien['to_stop_id'].str.split('StopPoint:DUA').str[1]
transfers_cleaned_Transilien = transfers_Transilien.copy()
transfers_cleaned_Transilien['from_stop_id'] = transfers_cleaned_Transilien_column1
transfers_cleaned_Transilien['to_stop_id'] = transfers_cleaned_Transilien_column2
transfers_cleaned_Transilien.loc[:,'from_stop_id'] = transfers_cleaned_Transilien.loc[:,'from_stop_id'].astype(np.int64)
transfers_cleaned_Transilien.loc[:,'to_stop_id'] = transfers_cleaned_Transilien.loc[:,'to_stop_id'].astype(np.int64)
transfers_cleaned_Transilien

' To clean the transfers_Transilien df.'

,from_stop_id,to_stop_id,transfer_type,min_transfer_time
0,8775859,8728189,2,912
1,8754700,8754702,2,60
2,8754702,8754700,2,924
3,8759118,8711300,2,279
4,8759118,8759492,2,946
...,...,...,...,...
95,8727116,8769732,2,966
96,8711371,8775830,2,207
97,8775830,8711371,2,460
98,8727605,8753413,2,249


In [89]:
''' To clean the routes_Transilien df.'''
# To keep the routes of trains and to delete the other routes
routes_filtered_Transilien = routes_Transilien[routes_Transilien['route_type'] == 2]
routes_filtered_Transilien

' To clean the routes_Transilien df.'

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,DUA800853021,DUA853,H,Gare du Nord - Pontoise / Luzarches / Beauvais...,NaN,2,NaN,844C54,FFFFFF
11,DUA800850011,DUA850,P,Gare de l'Est - La Ferté Milon / Château-Thier...,NaN,2,NaN,EAAB00,000000
12,DUA800852051,DUA852,N,Gare Montparnasse / Gare Montparnasse Vaugirar...,NaN,2,NaN,00B092,ffffff
13,DUA800855048,DUA855,U,La Verrière - La Défense Grande Arche via Chav...,NaN,2,NaN,DE4F75,ffffff
14,DUA800851081,DUA851,R,Gare de Lyon / Gare de Bercy - Montereau / Mon...,NaN,2,NaN,E59FDB,FFFFFF
15,DUA800854042,DUA854,L,Gare St-Lazare - Versailles R. Droite / Cergy ...,NaN,2,NaN,7577C0,ffffff
16,DUA800805091,DUA805,E,Haussmann Saint-Lazare - Chelles Gournay / Tou...,NaN,2,NaN,DE81D3,ffffff
17,DUA800804081,DUA804,D,Creil - Corbeil Essonnes / Melun / Malesherbes,NaN,2,NaN,3F9C35,ffffff
18,DUA800853022,DUA853,K,Gare du Nord - Crépy en Valois,NaN,2,NaN,AE9A00,000000
19,DUA800854041,DUA854,J,Gare St-Lazare - Ermont Eaubonne / Vernon / Gi...,NaN,2,NaN,B6BF00,ffffff


In [90]:
''' To clean the trips_Transilien df.'''
# To add starting characters 'Transilien-' to the service_id column to avoid confusion when the same service_id numbers are used 
# for different dates in datasets of different railway systems
trips_cleaned_Transilien_column = 'Transilien-' + trips_Transilien['service_id'].astype(str)
trips_cleaned_Transilien = trips_Transilien.copy()
trips_cleaned_Transilien['service_id'] = trips_cleaned_Transilien_column
trips_cleaned_Transilien

' To clean the trips_Transilien df.'

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id
0,DUA800853021,Transilien-9307,DUASN124601F03001-1_361260,POTA,0,NaN
1,DUA800853021,Transilien-9307,DUASN124603F02001-1_361261,POTA,0,NaN
2,DUA800853021,Transilien-9307,DUASN124605F03001-1_361262,POTA,0,NaN
3,DUA800853021,Transilien-9307,DUASN124607F01001-1_361263,POTA,0,NaN
4,DUA800853021,Transilien-9307,DUASN124613F03001-1_361264,POTA,0,NaN
...,...,...,...,...,...,...
34298,DUA800850511,Transilien-8612,DUASN412671R01001-1_77502,412671,1,NaN
34299,DUA800850511,Transilien-9699,DUASN850192R01001-1_77503,850192,1,NaN
34300,DUA800850511,Transilien-9699,DUASN850193R01001-1_77504,850193,1,NaN
34301,DUA800850511,Transilien-8798,DUASN887131R01001-1_77505,887131,1,NaN


In [91]:
''' To clean the calendar_Transilien df.'''
# To add starting characters 'Transilien-' to the service_id column to avoid confusion when the same service_id numbers are used 
# for different dates in datasets of different railway systems
calendar_cleaned_Transilien_column = 'Transilien-' + calendar_Transilien['service_id'].astype(str)
calendar_cleaned_Transilien = calendar_Transilien.copy()
calendar_cleaned_Transilien['service_id'] = calendar_cleaned_Transilien_column
calendar_cleaned_Transilien

' To clean the calendar_Transilien df.'

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,Transilien-8258,0,1,1,0,0,0,0,20210601,20210609
1,Transilien-8260,1,1,1,1,0,0,0,20210607,20210610
2,Transilien-8261,0,1,1,1,1,0,0,20210601,20210611
3,Transilien-8262,0,1,1,1,1,1,0,20210601,20210611
4,Transilien-8263,1,1,1,1,1,0,0,20210607,20210611
...,...,...,...,...,...,...,...,...,...,...
1047,Transilien-9970,1,1,1,1,1,1,1,20210304,20210612
1048,Transilien-9971,1,1,1,1,1,1,1,20210304,20210612
1049,Transilien-9972,1,1,1,1,1,1,1,20210304,20210612
1050,Transilien-9974,1,1,1,1,1,1,1,20210304,20210611


In [92]:
''' To clean the calendar_dates_Transilien df.'''
# To add starting characters 'Transilien-' to the service_id column to avoid confusion when the same service_id numbers are used 
# for different dates in datasets of different railway systems
calendar_dates_cleaned_Transilien_column = 'Transilien-' + calendar_dates_Transilien['service_id'].astype(str)
calendar_dates_cleaned_Transilien = calendar_dates_Transilien.copy()
calendar_dates_cleaned_Transilien['service_id'] = calendar_dates_cleaned_Transilien_column
calendar_dates_cleaned_Transilien

' To clean the calendar_dates_Transilien df.'

,service_id,date,exception_type
0,Transilien-8264,20210612,1
1,Transilien-8267,20210501,1
2,Transilien-8268,20210502,1
3,Transilien-8269,20210503,1
4,Transilien-8273,20210508,1
...,...,...,...
11110,Transilien-9971,20210508,2
11111,Transilien-9971,20210509,2
11112,Transilien-9971,20210529,2
11113,Transilien-9971,20210530,2


# TEST

In [93]:
testTransilien1 = pd.merge(trips_cleaned_Transilien[['route_id', 'trip_id']], stop_times_cleaned_Transilien[['trip_id', 'stop_id', 'stop_sequence']], on=['trip_id'])
testTransilien2 = pd.merge(testTransilien1, stops_cleaned_TGV[['stop_id', 'stop_name']], on=['stop_id'])
testTransilien3 = testTransilien2.groupby(['route_id','trip_id', 'stop_sequence'])[['stop_name']].last()
testTransilien3

,,,stop_name
route_id,trip_id,stop_sequence,


In [94]:
testTransilien1 = pd.merge(trips_cleaned_TER[['route_id', 'trip_id']], stop_times_cleaned_TER[['trip_id', 'stop_id', 'stop_sequence']], on=['trip_id'])
testTransilien2 = pd.merge(testTransilien1, stops_cleaned_TGV[['stop_id', 'stop_name']], on=['stop_id'])
testTransilien3 = testTransilien2.groupby(['route_id','trip_id', 'stop_sequence'])[['stop_name']].last()
testTransilien3

stop_name
route_id                                           trip_id                                 stop_sequence                                  
OCESN-1                                            OCESN855806F275451:2021-03-09T18:51:45Z 0                                         Brest
                                                                                           1                                    Landerneau
                                                                                           3                                       Morlaix
                                                                                           4                               Plouaret Trégor
                                                                                           5                                      Guingamp
...                                                                                                                                    ...
OCESN-FR:Line::E937D19D-BA51-4EC8-9C8E-6B20FD5D... OCESN23486F273242:2021-03-09T18:51:45Z  9                                     Annemasse
                                                   OCESN23488F275001:2021-03-09T18:51:45Z  0              Saint-Gervais-les-Bains Le Fayet
                                                                                           1                Sallanches - Combloux - Megève
                                                                                           3                                        Cluses
                                                                                           9                                     Annemasse

[28767 rows x 1 columns]

## Exploratory data analysis with the French Transilien railway datasets

In [95]:
'''To calculate the number of unique route_ids in the French Transilien railway system '''
set_routes_Transilien = {r for r in routes_filtered_Transilien['route_id']}
print("There are",len(set_routes_Transilien), "unique routes in the Transilien railway dataset.")

'To calculate the number of unique route_ids in the French Transilien railway system '

There are 13 unique routes in the Transilien railway dataset.


In [96]:
'''To calculate the number of stations in the stops_cleaned_Transilien dataset'''
set_stations_Transilien = {s for s in stops_cleaned_Transilien['stop_id']}
print("There are",len(set_stations_Transilien), "unique stations in the Transilien railway dataset.")

'To calculate the number of stations in the stops_cleaned_Transilien dataset'

There are 535 unique stations in the Transilien railway dataset.


## Check if overlap exists between GTFS formats of different French railway systems

### Check if overlap exists between GTFS formats of the French TER and the French Transilien railway systems

In [97]:
''' To check if overlap exists between GTFS formats of the French TER and the French Transilien railway systems'''
# Check for the route_ids of TER and the Transilien railway system
mergedf1 = pd.merge(routes_filtered_Transilien, routes_filtered_TER, on=['route_id'], how='inner')
print("The TER and the Transilien datasets have no common route_ids.")
mergedf1

# Check for the route_long_name of TER and the Transilien railway system
mergedf2 = pd.merge(routes_filtered_Transilien, routes_filtered_TER, on=['route_long_name'], how='inner')
print("The TER and the Transilien datasets have no common route_long_names.")
mergedf2

# Check for the trip_id of TER and the Transilien railway system
mergedf3 = pd.merge(stop_times_cleaned_Transilien, stop_times_cleaned_TER, on=['trip_id'], how='inner')
print("The TER and the Transilien datasets have no common trip_ids.")
mergedf3

# Check for the stop_id of TER and the Transilien railway system
mergedf4 = pd.merge(stops_cleaned_Transilien, stops_cleaned_TER, on=['stop_id'], how='inner')
print("The TER and the Transilien datasets have no common stop_ids.")
mergedf4

# Check for the stop_names of TER and the Transilien railway system
mergedf5 = pd.merge(stops_cleaned_Transilien, stops_cleaned_TER, on=['stop_name'], how='inner')
print("The TER and the Transilien datasets have no common stop_names.")
mergedf5

# Check for the service_ids of TER and the Transilien railway system
mergedf6 = pd.merge(trips_cleaned_Transilien, trips_cleaned_TER, on=['service_id'], how='inner')
print("The TER and the Transilien datasets have no common service_ids.")
mergedf6

' To check if overlap exists between GTFS formats of the French TER and the French Transilien railway systems'

The TER and the Transilien datasets have no common route_ids.


,route_id,agency_id_x,route_short_name_x,route_long_name_x,route_desc_x,route_type_x,route_url_x,route_color_x,route_text_color_x,agency_id_y,route_short_name_y,route_long_name_y,route_desc_y,route_type_y,route_url_y,route_color_y,route_text_color_y


The TER and the Transilien datasets have no common route_long_names.


,route_id_x,agency_id_x,route_short_name_x,route_long_name,route_desc_x,route_type_x,route_url_x,route_color_x,route_text_color_x,route_id_y,agency_id_y,route_short_name_y,route_desc_y,route_type_y,route_url_y,route_color_y,route_text_color_y


...

,stop_id,stop_name_x,stop_desc_x,stop_lat_x,stop_lon_x,zone_id_x,stop_url_x,location_type_x,parent_station_x,stop_name_y,stop_desc_y,stop_lat_y,stop_lon_y,zone_id_y,stop_url_y,location_type_y,parent_station_y


The TER and the Transilien datasets have no common stop_names.


,stop_id_x,stop_name,stop_desc_x,stop_lat_x,stop_lon_x,zone_id_x,stop_url_x,location_type_x,parent_station_x,stop_id_y,stop_desc_y,stop_lat_y,stop_lon_y,zone_id_y,stop_url_y,location_type_y,parent_station_y


The TER and the Transilien datasets have no common service_ids.


,route_id_x,service_id,trip_id_x,trip_headsign_x,direction_id_x,block_id_x,route_id_y,trip_id_y,trip_headsign_y,direction_id_y,block_id_y,shape_id


### Check if overlap exists between GTFS formats of the French TGV and the French Transilien railway systems

In [98]:
''' To check if overlap exists between GTFS formats of the French TGV and the French Transilien railway systems'''
# Check for the route_ids of TGV and the Transilien railway system
mergedf1 = pd.merge(routes_filtered_Transilien, routes_TGV, on=['route_id'], how='inner')
print("The TGV and the Transilien datasets have no common route_ids.")
mergedf1

# Check for the route_long_name of TGV and the Transilien railway system
mergedf2 = pd.merge(routes_filtered_Transilien, routes_TGV, on=['route_id'], how='inner')
print("The TGV and the Transilien datasets have no common route_long_names.")
mergedf2

# Check for the trip_id of TGV and the Transilien railway system
mergedf3 = pd.merge(stop_times_cleaned_Transilien, stop_times_cleaned_TGV, on=['trip_id'], how='inner')
print("The TGV and the Transilien datasets have no common trip_ids.")
mergedf3

# Check for the stop_id of TGV and the Transilien railway system
mergedf4 = pd.merge(stops_cleaned_Transilien, stops_cleaned_TGV, on=['stop_id'], how='inner')
print("The TGV and the Transilien datasets have no common stop_ids.")
mergedf4

# Check for the stop_names of TGV and the Transilien railway system
mergedf5 = pd.merge(stops_cleaned_Transilien, stops_cleaned_TGV, on=['stop_name'], how='inner')
print("The TGV and the Transilien datasets have no common stop_names.")
mergedf5

# Check for the service_ids of TGV and the Transilien railway system
mergedf6 = pd.merge(trips_cleaned_Transilien, trips_cleaned_TGV, on=['service_id'], how='inner')
print("The TGV and the Transilien datasets have no common service_ids.")
mergedf6

' To check if overlap exists between GTFS formats of the French TGV and the French Transilien railway systems'

The TGV and the Transilien datasets have no common route_ids.


,route_id,agency_id_x,route_short_name_x,route_long_name_x,route_desc_x,route_type_x,route_url_x,route_color_x,route_text_color_x,agency_id_y,route_short_name_y,route_long_name_y,route_desc_y,route_type_y,route_url_y,route_color_y,route_text_color_y


The TGV and the Transilien datasets have no common route_long_names.


,route_id,agency_id_x,route_short_name_x,route_long_name_x,route_desc_x,route_type_x,route_url_x,route_color_x,route_text_color_x,agency_id_y,route_short_name_y,route_long_name_y,route_desc_y,route_type_y,route_url_y,route_color_y,route_text_color_y


...

,stop_id,stop_name_x,stop_desc_x,stop_lat_x,stop_lon_x,zone_id_x,stop_url_x,location_type_x,parent_station_x,stop_name_y,stop_desc_y,stop_lat_y,stop_lon_y,zone_id_y,stop_url_y,location_type_y,parent_station_y


The TGV and the Transilien datasets have no common stop_names.


,stop_id_x,stop_name,stop_desc_x,stop_lat_x,stop_lon_x,zone_id_x,stop_url_x,location_type_x,parent_station_x,stop_id_y,stop_desc_y,stop_lat_y,stop_lon_y,zone_id_y,stop_url_y,location_type_y,parent_station_y


The TGV and the Transilien datasets have no common service_ids.


,route_id_x,service_id,trip_id_x,trip_headsign_x,direction_id_x,block_id_x,route_id_y,trip_id_y,trip_headsign_y,direction_id_y,block_id_y,shape_id


### Check if overlap exists between GTFS formats of the French TGV and the French TER railway systems

In [99]:
''' To check if overlap exists between GTFS formats of the French TGV and the French Transilien railway systems'''
# Check for the route_ids of TGV and the TER railway system
mergedf1 = pd.merge(routes_TGV, routes_filtered_TER, on=['route_id'], how='inner')
print("The TGV and the TER datasets have no common route_ids.")
mergedf1

# Check for the route_long_name of TGV and the TER railway system
mergedf2 = pd.merge(routes_TGV, routes_filtered_TER, on=['route_long_name'], how='inner')
print("The TGV and the TER datasets have no common route_long_names.")
mergedf2

# Check for the trip_id of TGV and the TER railway system
mergedf3 = pd.merge(stop_times_cleaned_TGV, stop_times_cleaned_TER, on=['trip_id'], how='inner')
print("The TGV and the TER datasets have no common trip_ids.")
mergedf3

# Check for the stop_id of TGV and the TER railway system
mergedf4 = pd.merge(stops_cleaned_TGV, stops_cleaned_TER, on=['stop_id'], how='inner')
print("The TGV and the TER datasets have common stop_ids.")
mergedf4

# Check for the stop_names of TGV and the TER railway system
mergedf5 = pd.merge(stops_cleaned_TGV, stops_cleaned_TER, on=['stop_name'], how='inner')
print("The TGV and the TER datasets have common stop_names.")
mergedf5

# Check for the service_ids of TGV and the TER railway system
mergedf6 = pd.merge(trips_cleaned_TGV, trips_cleaned_TER, on=['service_id'], how='inner')
print("The TGV and the TER datasets no longer have equal service_ids.")
mergedf6

' To check if overlap exists between GTFS formats of the French TGV and the French Transilien railway systems'

The TGV and the TER datasets have no common route_ids.


,route_id,agency_id_x,route_short_name_x,route_long_name_x,route_desc_x,route_type_x,route_url_x,route_color_x,route_text_color_x,agency_id_y,route_short_name_y,route_long_name_y,route_desc_y,route_type_y,route_url_y,route_color_y,route_text_color_y


The TGV and the TER datasets have no common route_long_names.


,route_id_x,agency_id_x,route_short_name_x,route_long_name,route_desc_x,route_type_x,route_url_x,route_color_x,route_text_color_x,route_id_y,agency_id_y,route_short_name_y,route_desc_y,route_type_y,route_url_y,route_color_y,route_text_color_y


...

,stop_id,stop_name_x,stop_desc_x,stop_lat_x,stop_lon_x,zone_id_x,stop_url_x,location_type_x,parent_station_x,stop_name_y,stop_desc_y,stop_lat_y,stop_lon_y,zone_id_y,stop_url_y,location_type_y,parent_station_y
0,80143099,Offenburg,NaN,48.476500,7.946800,NaN,NaN,0,StopArea:OCE80143099,Offenburg,NaN,48.476500,7.946800,NaN,NaN,0,StopArea:OCE80143099
1,82001000,Luxembourg,NaN,49.599722,6.134722,NaN,NaN,0,StopArea:OCE82001000,Luxembourg,NaN,49.599722,6.134722,NaN,NaN,0,StopArea:OCE82001000
2,87113001,Paris Est,NaN,48.876742,2.358424,NaN,NaN,0,StopArea:OCE87113001,Paris Est,NaN,48.876742,2.358424,NaN,NaN,0,StopArea:OCE87113001
3,87141002,Nancy,NaN,48.689857,6.174579,NaN,NaN,0,StopArea:OCE87141002,Nancy,NaN,48.689857,6.174579,NaN,NaN,0,StopArea:OCE87141002
4,87141150,Lunéville,NaN,48.587990,6.496997,NaN,NaN,0,StopArea:OCE87141150,Lunéville,NaN,48.587990,6.496997,NaN,NaN,0,StopArea:OCE87141150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,87775007,Nîmes,NaN,43.832313,4.365872,NaN,NaN,0,StopArea:OCE87775007,Nîmes,NaN,43.832313,4.365872,NaN,NaN,0,StopArea:OCE87775007
160,87781005,Béziers,NaN,43.336295,3.218895,NaN,NaN,0,StopArea:OCE87781005,Béziers,NaN,43.336295,3.218895,NaN,NaN,0,StopArea:OCE87781005
161,87781104,Narbonne,NaN,43.190618,3.005667,NaN,NaN,0,StopArea:OCE87781104,Narbonne,NaN,43.190618,3.005667,NaN,NaN,0,StopArea:OCE87781104
162,87781278,Agde,NaN,43.317280,3.466203,NaN,NaN,0,StopArea:OCE87781278,Agde,NaN,43.317280,3.466203,NaN,NaN,0,StopArea:OCE87781278


The TGV and the TER datasets have common stop_names.


,stop_id_x,stop_name,stop_desc_x,stop_lat_x,stop_lon_x,zone_id_x,stop_url_x,location_type_x,parent_station_x,stop_id_y,stop_desc_y,stop_lat_y,stop_lon_y,zone_id_y,stop_url_y,location_type_y,parent_station_y
0,80143099,Offenburg,NaN,48.476500,7.946800,NaN,NaN,0,StopArea:OCE80143099,80143099,NaN,48.476500,7.946800,NaN,NaN,0,StopArea:OCE80143099
1,82001000,Luxembourg,NaN,49.599722,6.134722,NaN,NaN,0,StopArea:OCE82001000,82001000,NaN,49.599722,6.134722,NaN,NaN,0,StopArea:OCE82001000
2,87113001,Paris Est,NaN,48.876742,2.358424,NaN,NaN,0,StopArea:OCE87113001,87113001,NaN,48.876742,2.358424,NaN,NaN,0,StopArea:OCE87113001
3,87141002,Nancy,NaN,48.689857,6.174579,NaN,NaN,0,StopArea:OCE87141002,87141002,NaN,48.689857,6.174579,NaN,NaN,0,StopArea:OCE87141002
4,87141150,Lunéville,NaN,48.587990,6.496997,NaN,NaN,0,StopArea:OCE87141150,87141150,NaN,48.587990,6.496997,NaN,NaN,0,StopArea:OCE87141150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,87775007,Nîmes,NaN,43.832313,4.365872,NaN,NaN,0,StopArea:OCE87775007,87775007,NaN,43.832313,4.365872,NaN,NaN,0,StopArea:OCE87775007
160,87781005,Béziers,NaN,43.336295,3.218895,NaN,NaN,0,StopArea:OCE87781005,87781005,NaN,43.336295,3.218895,NaN,NaN,0,StopArea:OCE87781005
161,87781104,Narbonne,NaN,43.190618,3.005667,NaN,NaN,0,StopArea:OCE87781104,87781104,NaN,43.190618,3.005667,NaN,NaN,0,StopArea:OCE87781104
162,87781278,Agde,NaN,43.317280,3.466203,NaN,NaN,0,StopArea:OCE87781278,87781278,NaN,43.317280,3.466203,NaN,NaN,0,StopArea:OCE87781278


The TGV and the TER datasets no longer have equal service_ids.


,route_id_x,service_id,trip_id_x,trip_headsign_x,direction_id_x,block_id_x,shape_id_x,route_id_y,trip_id_y,trip_headsign_y,direction_id_y,block_id_y,shape_id_y


### Check if overlap exists between GTFS formats of the French TGV and the French Intercités railway systems

In [100]:
''' To check if overlap exists between GTFS formats of the French TGV and the French Intercités railway systems'''
# Check for the route_ids of TGV and the Intercités railway system
mergedf1 = pd.merge(routes_TGV, routes_Intercites, on=['route_id'], how='inner')
print("The TGV and the Intercités datasets have no common route_ids.")
mergedf1

# Check for the route_long_name of TGV and the Intercités railway system
mergedf2 = pd.merge(routes_TGV, routes_Intercites, on=['route_long_name'], how='inner')
print("The TGV and the Intercités datasets have no common route_long_names.")
mergedf2

# Check for the trip_id of TGV and the Intercités railway system
mergedf3 = pd.merge(stop_times_cleaned_TGV, stop_times_cleaned_Intercites, on=['trip_id'], how='inner')
print("The TGV and the Intercités datasets have no common trip_ids.")
mergedf3

# Check for the stop_id of TGV and the Intercités railway system
mergedf4 = pd.merge(stops_cleaned_TGV, stops_cleaned_Intercites, on=['stop_id'], how='inner')
print("The TGV and the Intercités datasets have common stop_ids.")
mergedf4

# Check for the stop_names of TGV and the Intercités railway system
mergedf5 = pd.merge(stops_cleaned_TGV, stops_cleaned_Intercites, on=['stop_name'], how='inner')
print("The TGV and the Intercités datasets common stop_names.")
mergedf5

# Check for the service_ids of TGV and the Intercités railway system
mergedf6 = pd.merge(trips_cleaned_TGV, trips_cleaned_Intercites, on=['service_id'], how='inner')
print("The TGV and the Intercités datasets have no common service_ids.")
mergedf6

' To check if overlap exists between GTFS formats of the French TGV and the French Intercités railway systems'

The TGV and the Intercités datasets have no common route_ids.


,route_id,agency_id_x,route_short_name_x,route_long_name_x,route_desc_x,route_type_x,route_url_x,route_color_x,route_text_color_x,agency_id_y,route_short_name_y,route_long_name_y,route_desc_y,route_type_y,route_url_y,route_color_y,route_text_color_y


The TGV and the Intercités datasets have no common route_long_names.


,route_id_x,agency_id_x,route_short_name_x,route_long_name,route_desc_x,route_type_x,route_url_x,route_color_x,route_text_color_x,route_id_y,agency_id_y,route_short_name_y,route_desc_y,route_type_y,route_url_y,route_color_y,route_text_color_y


...

,stop_id,stop_name_x,stop_desc_x,stop_lat_x,stop_lon_x,zone_id_x,stop_url_x,location_type_x,parent_station_x,stop_name_y,stop_desc_y,stop_lat_y,stop_lon_y,zone_id_y,stop_url_y,location_type_y,parent_station_y
0,87481002,Nantes,NaN,47.217491,-1.542077,NaN,NaN,0,StopArea:OCE87481002,Nantes,NaN,47.217491,-1.542077,NaN,NaN,0,StopArea:OCE87481002
1,87484006,Angers Saint-Laud,NaN,47.464647,-0.556820,NaN,NaN,0,StopArea:OCE87484006,Angers Saint-Laud,NaN,47.464647,-0.556820,NaN,NaN,0,StopArea:OCE87484006
2,87485003,La Rochelle,NaN,46.152705,-1.145853,NaN,NaN,0,StopArea:OCE87485003,La Rochelle,NaN,46.152705,-1.145853,NaN,NaN,0,StopArea:OCE87485003
3,87486019,La Roche-sur-Yon,NaN,46.672075,-1.435660,NaN,NaN,0,StopArea:OCE87486019,La Roche-sur-Yon,NaN,46.672075,-1.435660,NaN,NaN,0,StopArea:OCE87486019
4,87487603,Saumur,NaN,47.268994,-0.071403,NaN,NaN,0,StopArea:OCE87487603,Saumur,NaN,47.268994,-0.071403,NaN,NaN,0,StopArea:OCE87487603
5,87571240,Saint-Pierre-des-Corps,NaN,47.386140,0.723539,NaN,NaN,0,StopArea:OCE87571240,Saint-Pierre-des-Corps,NaN,47.386140,0.723539,NaN,NaN,0,StopArea:OCE87571240
6,87581009,Bordeaux Saint-Jean,NaN,44.825873,-0.556697,NaN,NaN,0,StopArea:OCE87581009,Bordeaux Saint-Jean,NaN,44.825873,-0.556697,NaN,NaN,0,StopArea:OCE87581009
7,87586008,Agen,NaN,44.207967,0.620867,NaN,NaN,0,StopArea:OCE87586008,Agen,NaN,44.207967,0.620867,NaN,NaN,0,StopArea:OCE87586008
8,87611004,Toulouse Matabiau,NaN,43.611206,1.453616,NaN,NaN,0,StopArea:OCE87611004,Toulouse Matabiau,NaN,43.611206,1.453616,NaN,NaN,0,StopArea:OCE87611004
9,87611244,Montauban Ville Bourbon,NaN,44.014440,1.341499,NaN,NaN,0,StopArea:OCE87611244,Montauban Ville Bourbon,NaN,44.014440,1.341499,NaN,NaN,0,StopArea:OCE87611244


The TGV and the Intercités datasets common stop_names.


,stop_id_x,stop_name,stop_desc_x,stop_lat_x,stop_lon_x,zone_id_x,stop_url_x,location_type_x,parent_station_x,stop_id_y,stop_desc_y,stop_lat_y,stop_lon_y,zone_id_y,stop_url_y,location_type_y,parent_station_y
0,87481002,Nantes,NaN,47.217491,-1.542077,NaN,NaN,0,StopArea:OCE87481002,87481002,NaN,47.217491,-1.542077,NaN,NaN,0,StopArea:OCE87481002
1,87484006,Angers Saint-Laud,NaN,47.464647,-0.556820,NaN,NaN,0,StopArea:OCE87484006,87484006,NaN,47.464647,-0.556820,NaN,NaN,0,StopArea:OCE87484006
2,87485003,La Rochelle,NaN,46.152705,-1.145853,NaN,NaN,0,StopArea:OCE87485003,87485003,NaN,46.152705,-1.145853,NaN,NaN,0,StopArea:OCE87485003
3,87486019,La Roche-sur-Yon,NaN,46.672075,-1.435660,NaN,NaN,0,StopArea:OCE87486019,87486019,NaN,46.672075,-1.435660,NaN,NaN,0,StopArea:OCE87486019
4,87487603,Saumur,NaN,47.268994,-0.071403,NaN,NaN,0,StopArea:OCE87487603,87487603,NaN,47.268994,-0.071403,NaN,NaN,0,StopArea:OCE87487603
5,87571240,Saint-Pierre-des-Corps,NaN,47.386140,0.723539,NaN,NaN,0,StopArea:OCE87571240,87571240,NaN,47.386140,0.723539,NaN,NaN,0,StopArea:OCE87571240
6,87581009,Bordeaux Saint-Jean,NaN,44.825873,-0.556697,NaN,NaN,0,StopArea:OCE87581009,87581009,NaN,44.825873,-0.556697,NaN,NaN,0,StopArea:OCE87581009
7,87586008,Agen,NaN,44.207967,0.620867,NaN,NaN,0,StopArea:OCE87586008,87586008,NaN,44.207967,0.620867,NaN,NaN,0,StopArea:OCE87586008
8,87611004,Toulouse Matabiau,NaN,43.611206,1.453616,NaN,NaN,0,StopArea:OCE87611004,87611004,NaN,43.611206,1.453616,NaN,NaN,0,StopArea:OCE87611004
9,87611244,Montauban Ville Bourbon,NaN,44.014440,1.341499,NaN,NaN,0,StopArea:OCE87611244,87611244,NaN,44.014440,1.341499,NaN,NaN,0,StopArea:OCE87611244


The TGV and the Intercités datasets have no common service_ids.


,route_id_x,service_id,trip_id_x,trip_headsign_x,direction_id_x,block_id_x,shape_id_x,route_id_y,trip_id_y,trip_headsign_y,direction_id_y,block_id_y,shape_id_y


### Check if overlap exists between GTFS formats of the French TER and the French Intercités railway systems

In [101]:
''' To check if overlap exists between GTFS formats of the French TGV and the French Transilien railway systems'''
# Check for the route_ids of Intercités and the TER railway system
mergedf1 = pd.merge(routes_Intercites, routes_filtered_TER, on=['route_id'], how='inner')
print("The Intercités and the TER datasets have no common route_ids.")
mergedf1

# Check for the route_long_name of Intercités and the TER railway system
mergedf2 = pd.merge(routes_Intercites, routes_filtered_TER, on=['route_long_name'], how='inner')
print("The Intercités and the TER datasets have no common route_long_names.")
mergedf2

# Check for the trip_id of Intercités and the TER railway system
mergedf3 = pd.merge(stop_times_cleaned_Intercites, stop_times_cleaned_TER, on=['trip_id'], how='inner')
print("The Intercités and the TER datasets have common trip_ids.")
mergedf3

# Check for the stop_id of Intercités and the TER railway system
mergedf4 = pd.merge(stops_cleaned_Intercites, stops_cleaned_TER, on=['stop_id'], how='inner')
print("The Intercités and the TER datasets have common stop_ids.")
mergedf4

# Check for the stop_names of Intercités and the TER railway system
mergedf5 = pd.merge(stops_cleaned_Intercites, stops_cleaned_TER, on=['stop_name'], how='inner')
print("The Intercités and the TER datasets have common stop_names.")
mergedf5

# Check for the service_ids of Intercités and the TER railway system
mergedf6 = pd.merge(trips_cleaned_Intercites, trips_cleaned_TER, on=['service_id'], how='inner')
print("The Intercités and the TER datasets have no common service_ids.")
mergedf6

' To check if overlap exists between GTFS formats of the French TGV and the French Transilien railway systems'

The Intercités and the TER datasets have no common route_ids.


,route_id,agency_id_x,route_short_name_x,route_long_name_x,route_desc_x,route_type_x,route_url_x,route_color_x,route_text_color_x,agency_id_y,route_short_name_y,route_long_name_y,route_desc_y,route_type_y,route_url_y,route_color_y,route_text_color_y


The Intercités and the TER datasets have no common route_long_names.


,route_id_x,agency_id_x,route_short_name_x,route_long_name,route_desc_x,route_type_x,route_url_x,route_color_x,route_text_color_x,route_id_y,agency_id_y,route_short_name_y,route_desc_y,route_type_y,route_url_y,route_color_y,route_text_color_y


...

,stop_id,stop_name_x,stop_desc_x,stop_lat_x,stop_lon_x,zone_id_x,stop_url_x,location_type_x,parent_station_x,stop_name_y,stop_desc_y,stop_lat_y,stop_lon_y,zone_id_y,stop_url_y,location_type_y,parent_station_y
0,87481002,Nantes,NaN,47.217491,-1.542077,NaN,NaN,0,StopArea:OCE87481002,Nantes,NaN,47.217491,-1.542077,NaN,NaN,0,StopArea:OCE87481002
1,87484006,Angers Saint-Laud,NaN,47.464647,-0.556820,NaN,NaN,0,StopArea:OCE87484006,Angers Saint-Laud,NaN,47.464647,-0.556820,NaN,NaN,0,StopArea:OCE87484006
2,87485003,La Rochelle,NaN,46.152705,-1.145853,NaN,NaN,0,StopArea:OCE87485003,La Rochelle,NaN,46.152705,-1.145853,NaN,NaN,0,StopArea:OCE87485003
3,87485144,Rochefort,NaN,45.947189,-0.963556,NaN,NaN,0,StopArea:OCE87485144,Rochefort,NaN,45.947189,-0.963556,NaN,NaN,0,StopArea:OCE87485144
4,87486019,La Roche-sur-Yon,NaN,46.672075,-1.435660,NaN,NaN,0,StopArea:OCE87486019,La Roche-sur-Yon,NaN,46.672075,-1.435660,NaN,NaN,0,StopArea:OCE87486019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,87784231,Argelès-sur-Mer,NaN,42.543908,3.021919,NaN,NaN,0,StopArea:OCE87784231,Argelès-sur-Mer,NaN,42.543908,3.021919,NaN,NaN,0,StopArea:OCE87784231
127,87784256,Collioure,NaN,42.526928,3.078202,NaN,NaN,0,StopArea:OCE87784256,Collioure,NaN,42.526928,3.078202,NaN,NaN,0,StopArea:OCE87784256
128,87784264,Port-Vendres,NaN,42.513608,3.102576,NaN,NaN,0,StopArea:OCE87784264,Port-Vendres,NaN,42.513608,3.102576,NaN,NaN,0,StopArea:OCE87784264
129,87784298,Banyuls-sur-Mer,NaN,42.482958,3.125050,NaN,NaN,0,StopArea:OCE87784298,Banyuls-sur-Mer,NaN,42.482958,3.125050,NaN,NaN,0,StopArea:OCE87784298


The Intercités and the TER datasets have common stop_names.


,stop_id_x,stop_name,stop_desc_x,stop_lat_x,stop_lon_x,zone_id_x,stop_url_x,location_type_x,parent_station_x,stop_id_y,stop_desc_y,stop_lat_y,stop_lon_y,zone_id_y,stop_url_y,location_type_y,parent_station_y
0,87481002,Nantes,NaN,47.217491,-1.542077,NaN,NaN,0,StopArea:OCE87481002,87481002,NaN,47.217491,-1.542077,NaN,NaN,0,StopArea:OCE87481002
1,87484006,Angers Saint-Laud,NaN,47.464647,-0.556820,NaN,NaN,0,StopArea:OCE87484006,87484006,NaN,47.464647,-0.556820,NaN,NaN,0,StopArea:OCE87484006
2,87485003,La Rochelle,NaN,46.152705,-1.145853,NaN,NaN,0,StopArea:OCE87485003,87485003,NaN,46.152705,-1.145853,NaN,NaN,0,StopArea:OCE87485003
3,87485144,Rochefort,NaN,45.947189,-0.963556,NaN,NaN,0,StopArea:OCE87485144,87485144,NaN,45.947189,-0.963556,NaN,NaN,0,StopArea:OCE87485144
4,87486019,La Roche-sur-Yon,NaN,46.672075,-1.435660,NaN,NaN,0,StopArea:OCE87486019,87486019,NaN,46.672075,-1.435660,NaN,NaN,0,StopArea:OCE87486019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,87784231,Argelès-sur-Mer,NaN,42.543908,3.021919,NaN,NaN,0,StopArea:OCE87784231,87784231,NaN,42.543908,3.021919,NaN,NaN,0,StopArea:OCE87784231
127,87784256,Collioure,NaN,42.526928,3.078202,NaN,NaN,0,StopArea:OCE87784256,87784256,NaN,42.526928,3.078202,NaN,NaN,0,StopArea:OCE87784256
128,87784264,Port-Vendres,NaN,42.513608,3.102576,NaN,NaN,0,StopArea:OCE87784264,87784264,NaN,42.513608,3.102576,NaN,NaN,0,StopArea:OCE87784264
129,87784298,Banyuls-sur-Mer,NaN,42.482958,3.125050,NaN,NaN,0,StopArea:OCE87784298,87784298,NaN,42.482958,3.125050,NaN,NaN,0,StopArea:OCE87784298


The Intercités and the TER datasets have no common service_ids.


,route_id_x,service_id,trip_id_x,trip_headsign_x,direction_id_x,block_id_x,shape_id_x,route_id_y,trip_id_y,trip_headsign_y,direction_id_y,block_id_y,shape_id_y


### Check if overlap exists between GTFS formats of the French Transilien and the French Intercités railway systems

In [102]:
''' To check if overlap exists between GTFS formats of the French TER and the French Transilien railway systems'''
# Check for the route_ids of Intercités and the Transilien railway system
mergedf1 = pd.merge(routes_filtered_Transilien, routes_Intercites, on=['route_id'], how='inner')
print("The Intercités and the Transilien datasets have no common route_ids.")
mergedf1

# Check for the route_long_name of Intercités and the Transilien railway system
mergedf2 = pd.merge(routes_filtered_Transilien, routes_Intercites, on=['route_long_name'], how='inner')
print("The Intercités and the Transilien datasets have no common route_long_names.")
mergedf2

# Check for the trip_id of Intercités and the Transilien railway system
mergedf3 = pd.merge(stop_times_cleaned_Transilien, stop_times_cleaned_Intercites, on=['trip_id'], how='inner')
print("The Intercités and the Transilien datasets have no common trip_ids.")
mergedf3

# Check for the stop_id of Intercités and the Transilien railway system
mergedf4 = pd.merge(stops_cleaned_Transilien, stops_cleaned_Intercites, on=['stop_id'], how='inner')
print("The Intercités and the Transilien datasets have no common stop_ids.")
mergedf4

# Check for the stop_names of Intercités and the Transilien railway system
mergedf5 = pd.merge(stops_cleaned_Transilien, stops_cleaned_Intercites, on=['stop_name'], how='inner')
print("The Intercités and the Transilien datasets have no common stop_names.")
mergedf5

# Check for the service_ids of Intercités and the Transilien railway system
mergedf6 = pd.merge(trips_cleaned_Transilien, trips_cleaned_Intercites, on=['service_id'], how='inner')
print("The Intercités and the Transilien datasets have no common service_ids.")
mergedf6

' To check if overlap exists between GTFS formats of the French TER and the French Transilien railway systems'

The Intercités and the Transilien datasets have no common route_ids.


,route_id,agency_id_x,route_short_name_x,route_long_name_x,route_desc_x,route_type_x,route_url_x,route_color_x,route_text_color_x,agency_id_y,route_short_name_y,route_long_name_y,route_desc_y,route_type_y,route_url_y,route_color_y,route_text_color_y


The Intercités and the Transilien datasets have no common route_long_names.


,route_id_x,agency_id_x,route_short_name_x,route_long_name,route_desc_x,route_type_x,route_url_x,route_color_x,route_text_color_x,route_id_y,agency_id_y,route_short_name_y,route_desc_y,route_type_y,route_url_y,route_color_y,route_text_color_y


...

,stop_id,stop_name_x,stop_desc_x,stop_lat_x,stop_lon_x,zone_id_x,stop_url_x,location_type_x,parent_station_x,stop_name_y,stop_desc_y,stop_lat_y,stop_lon_y,zone_id_y,stop_url_y,location_type_y,parent_station_y


The Intercités and the Transilien datasets have no common stop_names.


,stop_id_x,stop_name,stop_desc_x,stop_lat_x,stop_lon_x,zone_id_x,stop_url_x,location_type_x,parent_station_x,stop_id_y,stop_desc_y,stop_lat_y,stop_lon_y,zone_id_y,stop_url_y,location_type_y,parent_station_y


The Intercités and the Transilien datasets have no common service_ids.


,route_id_x,service_id,trip_id_x,trip_headsign_x,direction_id_x,block_id_x,route_id_y,trip_id_y,trip_headsign_y,direction_id_y,block_id_y,shape_id


### To check the overlap between the stop_times_cleaned_TER and the stop_times_cleaned_Intercites dataset

In [103]:
stop_times_cleaned_Intercites

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,OCESN13101F274333:2021-03-09T18:51:45Z,05:46:00,05:46:00,87384008,0,NaN,0,1,NaN
1,OCESN13101F274333:2021-03-09T18:51:45Z,06:34:00,06:36:00,87415604,1,NaN,0,0,NaN
2,OCESN13101F274333:2021-03-09T18:51:45Z,06:43:00,06:44:00,87415620,2,NaN,0,0,NaN
3,OCESN13101F274333:2021-03-09T18:51:45Z,06:58:00,06:59:00,87415877,3,NaN,0,0,NaN
4,OCESN13101F274333:2021-03-09T18:51:45Z,07:07:00,07:08:00,87411207,4,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...
177818,OCESN96789F261181:2021-03-09T18:51:45Z,06:13:00,06:14:00,85010033,3,NaN,0,0,NaN
177819,OCESN96789F261181:2021-03-09T18:51:45Z,06:15:00,06:16:00,85010009,4,NaN,0,0,NaN
177820,OCESN96789F261181:2021-03-09T18:51:45Z,06:17:00,06:18:00,85010066,5,NaN,0,0,NaN
177821,OCESN96789F261181:2021-03-09T18:51:45Z,06:21:00,06:22:00,85010074,6,NaN,0,0,NaN


In [104]:
stop_times_cleaned_TER

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,OCESN13101F274333:2021-03-09T18:51:45Z,05:46:00,05:46:00,87384008,0,NaN,0,1,NaN
1,OCESN13101F274333:2021-03-09T18:51:45Z,06:34:00,06:36:00,87415604,1,NaN,0,0,NaN
2,OCESN13101F274333:2021-03-09T18:51:45Z,06:43:00,06:44:00,87415620,2,NaN,0,0,NaN
3,OCESN13101F274333:2021-03-09T18:51:45Z,06:58:00,06:59:00,87415877,3,NaN,0,0,NaN
4,OCESN13101F274333:2021-03-09T18:51:45Z,07:07:00,07:08:00,87411207,4,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...
177818,OCESN96789F261181:2021-03-09T18:51:45Z,06:13:00,06:14:00,85010033,3,NaN,0,0,NaN
177819,OCESN96789F261181:2021-03-09T18:51:45Z,06:15:00,06:16:00,85010009,4,NaN,0,0,NaN
177820,OCESN96789F261181:2021-03-09T18:51:45Z,06:17:00,06:18:00,85010066,5,NaN,0,0,NaN
177821,OCESN96789F261181:2021-03-09T18:51:45Z,06:21:00,06:22:00,85010074,6,NaN,0,0,NaN


In [105]:
check_overlap_stop_times = pd.merge(stop_times_cleaned_Intercites, stop_times_cleaned_TER, on=['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence'], how='inner')
check_overlap_stop_times = check_overlap_stop_times[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence']]
check_overlap_stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,OCESN13101F274333:2021-03-09T18:51:45Z,05:46:00,05:46:00,87384008,0
1,OCESN13101F274333:2021-03-09T18:51:45Z,06:34:00,06:36:00,87415604,1
2,OCESN13101F274333:2021-03-09T18:51:45Z,06:43:00,06:44:00,87415620,2
3,OCESN13101F274333:2021-03-09T18:51:45Z,06:58:00,06:59:00,87415877,3
4,OCESN13101F274333:2021-03-09T18:51:45Z,07:07:00,07:08:00,87411207,4
...,...,...,...,...,...
137570,OCESN96789F261181:2021-03-09T18:51:45Z,06:13:00,06:14:00,85010033,3
137571,OCESN96789F261181:2021-03-09T18:51:45Z,06:15:00,06:16:00,85010009,4
137572,OCESN96789F261181:2021-03-09T18:51:45Z,06:17:00,06:18:00,85010066,5
137573,OCESN96789F261181:2021-03-09T18:51:45Z,06:21:00,06:22:00,85010074,6


Conclusion of this check: all rows of the stop_times_cleaned_TER dataset are included in the stop_times_cleaned_Intercites dataset

### To check if the routes_Intercites and the routes_filtered_TER contain some identical routes

In [106]:
routes_cleaned_Intercites_column = routes_Intercites['route_id'].str.split('OCESN-').str[1]
routes_cleaned_TER_column = routes_filtered_TER['route_id'].str.split('OCESN-').str[1]

overlap_routes_Intercites_TER = set(routes_cleaned_Intercites_column) & set(routes_cleaned_TER_column)
overlap_routes_Intercites_TER

set()

Conclusion of this check: there are no common routes in the Intercites and the TER datasets

### To check if the stop_ids that occur in the datasets of several French railway systems have an equal minimum transfer time in the transfer datasets of these different French railway systems

In [107]:
''' To check if the stations of the French TGV and the TER railway systems have an equal minimum transfer time'''
overlap_stations_1 = pd.merge(transfers_cleaned_TGV, transfers_cleaned_TER, on=['from_stop_id', 'to_stop_id'], how='inner')
len(overlap_stations_1)
overlap_stations_2 = pd.merge(transfers_cleaned_TGV, transfers_cleaned_TER, on=['from_stop_id', 'to_stop_id', 'min_transfer_time'], how='inner')
len(overlap_stations_2)

' To check if the stations of the French TGV and the TER railway systems have an equal minimum transfer time'

1476

1476

In [108]:
''' To check if the stations of the French TGV and the Intercités railway systems have an equal minimum transfer time'''
overlap_stations_3 = pd.merge(transfers_cleaned_TGV, transfers_cleaned_Intercites, on=['from_stop_id', 'to_stop_id'], how='inner')
len(overlap_stations_3)
overlap_stations_4 = pd.merge(transfers_cleaned_TGV, transfers_cleaned_Intercites, on=['from_stop_id', 'to_stop_id', 'min_transfer_time'], how='inner')
len(overlap_stations_4)

' To check if the stations of the French TGV and the Intercités railway systems have an equal minimum transfer time'

570

570

In [109]:
''' To check if the stations of the French TGV and the Intercités railway systems have an equal minimum transfer time'''
overlap_stations_5 = pd.merge(transfers_cleaned_Intercites, transfers_cleaned_TER, on=['from_stop_id', 'to_stop_id'], how='inner')
len(overlap_stations_5)
overlap_stations_6 = pd.merge(transfers_cleaned_Intercites, transfers_cleaned_TER, on=['from_stop_id', 'to_stop_id', 'min_transfer_time'], how='inner')
len(overlap_stations_6)

' To check if the stations of the French TGV and the Intercités railway systems have an equal minimum transfer time'

156

156

Conclusion of this check: the stations that appear in multiple French railway systems have an equal minimum transfer time in those railway systems

# **Research question 1: How similar are railway systems in terms of their global network metrics?**

## For this a comparison of different railway systems in a L-space network representation is performed.

##### Remember that in the L-space representation the nodes denote the stations and there exist an edge between two stations when these stations are consecutively serviced by at least one route​.

# **Comparison of the Belgian and the French railway system in L-space representation**


## **Preparation for the L-space representation of the Belgian railway system**



In [110]:
'''To merge a selection of the stops_cleaned dataset with a selection of the stop_times dataset'''
stops_cleaned_stop_times_merge = pd.merge(stop_times[['trip_id','arrival_time', 'departure_time','stop_id','stop_sequence']], stops_cleaned[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']], on='stop_id')
stops_cleaned_stop_times_merge

'To merge a selection of the stops_cleaned dataset with a selection of the stop_times dataset'

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon
0,88____:049::8892338:8892205:6:1925:20210314,17:35:00,17:35:00,8892338,1,La Panne,51.07740,2.601966
1,88____:049::8892338:8892205:6:720:20210314,05:30:00,05:30:00,8892338,1,La Panne,51.07740,2.601966
2,88____:049::8892338:8892205:6:820:20210314,06:30:00,06:30:00,8892338,1,La Panne,51.07740,2.601966
3,88____:049::8892338:8892205:6:920:20210314,07:30:00,07:30:00,8892338,1,La Panne,51.07740,2.601966
4,88____:049::8892338:8892205:6:1020:20210314,08:30:00,08:30:00,8892338,1,La Panne,51.07740,2.601966
...,...,...,...,...,...,...,...,...
440853,87____:005::8727100:8814308:2:1434:20210424,12:31:00,12:31:00,8727100,1,Paris Nord (FR),48.86667,2.333333
440854,87____:005::8727100:8814308:2:1432:20211211,12:31:00,12:31:00,8727100,1,Paris Nord (FR),48.86667,2.333333
440855,88____:005::8814001:8727100:3:1756:20211210,17:56:00,17:56:00,8727100,3,Paris Nord (FR),48.86667,2.333333
440856,87____:005::8727100:8814308:2:2111:20210430,18:49:00,18:49:00,8727100,1,Paris Nord (FR),48.86667,2.333333


In [111]:
'''To merge a selection of the stops_cleaned_stop_times_merge dataset with the filtered_trips dataset.'''
stops_cleaned_stop_times_trips_merge = pd.merge(filtered_trips, stops_cleaned_stop_times_merge, on='trip_id', how = 'left')
stops_cleaned_stop_times_trips_merge

'To merge a selection of the stops_cleaned_stop_times_merge dataset with the filtered_trips dataset.'

,route_id,service_id,trip_id,trip_headsign,route_long_name,arrival_time,departure_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon
0,117,14,88____:007::8885704:8885001:4:523:20210418,Tournai,Tournai -- Mouscron,05:23:00,05:23:00,8885001,4,Tournai,50.61313,3.396940
1,117,14,88____:007::8885704:8885001:4:523:20210418,Tournai,Tournai -- Mouscron,05:19:00,05:19:00,8885068,3,Froyennes,50.62989,3.354835
2,117,14,88____:007::8885704:8885001:4:523:20210418,Tournai,Tournai -- Mouscron,05:12:00,05:12:00,8885753,2,Herseaux,50.71390,3.245961
3,117,14,88____:007::8885704:8885001:4:523:20210418,Tournai,Tournai -- Mouscron,05:07:00,05:07:00,8885704,1,Mouscron,50.74100,3.228449
4,117,14,88____:007::8885704:8885001:4:623:20210418,Tournai,Tournai -- Mouscron,06:23:00,06:23:00,8885001,4,Tournai,50.61313,3.396940
...,...,...,...,...,...,...,...,...,...,...,...,...
409730,739,21,88____:007::8821105:8812005:22:1723:20210418,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord,17:01:00,17:01:00,8811270,19,Veltem,50.90052,4.633520
409731,739,21,88____:007::8821105:8812005:22:1723:20210418,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord,16:59:00,16:59:00,8811288,18,Herent,50.90353,4.672190
409732,739,21,88____:007::8821105:8812005:22:1723:20210418,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord,17:10:00,17:12:00,8819406,23,Brussels Airport-Zaventem,50.89646,4.482072
409733,739,21,88____:007::8821105:8812005:22:1723:20210418,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord,16:11:00,16:11:00,8821063,5,Anvers-Luchtbal,51.24413,4.425033


In [112]:
'''To create a route_sequence dataset that gives for each trip_id that belongs to a route the sequence of stations served'''
route_sequence = stops_cleaned_stop_times_trips_merge.groupby(['route_id','route_long_name','trip_headsign','trip_id','stop_sequence'], as_index=False)[['stop_name', 'stop_lat', 'stop_lon']].last()
route_sequence

'To create a route_sequence dataset that gives for each trip_id that belongs to a route the sequence of stations served'

,route_id,route_long_name,trip_headsign,trip_id,stop_sequence,stop_name,stop_lat,stop_lon
0,117,Tournai -- Mouscron,Mouscron,88____:007::8885001:8885704:4:1052:20210418,1,Tournai,50.61313,3.396940
1,117,Tournai -- Mouscron,Mouscron,88____:007::8885001:8885704:4:1052:20210418,2,Froyennes,50.62989,3.354835
2,117,Tournai -- Mouscron,Mouscron,88____:007::8885001:8885704:4:1052:20210418,3,Herseaux,50.71390,3.245961
3,117,Tournai -- Mouscron,Mouscron,88____:007::8885001:8885704:4:1052:20210418,4,Mouscron,50.74100,3.228449
4,117,Tournai -- Mouscron,Mouscron,88____:007::8885001:8885704:4:1052:20210530,1,Tournai,50.61313,3.396940
...,...,...,...,...,...,...,...,...
409730,739,Den Haag HS (NL) -- Bruxelles-Nord,Den Haag HS (NL),88____:007::8812005:8400131:23:1618:20210418,19,Anvers-Berchem,51.19923,4.432219
409731,739,Den Haag HS (NL) -- Bruxelles-Nord,Den Haag HS (NL),88____:007::8812005:8400131:23:1618:20210418,20,Anvers-Central,51.21720,4.421098
409732,739,Den Haag HS (NL) -- Bruxelles-Nord,Den Haag HS (NL),88____:007::8812005:8400131:23:1618:20210418,21,Anvers-Luchtbal,51.24413,4.425033
409733,739,Den Haag HS (NL) -- Bruxelles-Nord,Den Haag HS (NL),88____:007::8812005:8400131:23:1618:20210418,22,Noorderkempen (Brecht),51.35683,4.632200


### code voor trips_hash => too long so it is read from a csv

'''To calculate the hash value for the stop sequence of each trip_id'''

###To copy the filtered_trips dataset
trips_hash = filtered_trips.copy()

#####To create a column called hash that contains NaN values
trips_hash['hash'] = np.nan

######For each trip_id in filtered_trips, the stop_sequence that gets calculated is the subset of the stop_time dataset for that trip_id. 
####The tuple that results from the stop_id column of this subset dataset contains all the stop_ids that get served by this trip_id. 
#####The hash value of the tuple of the stop_id column is calculated and is placed in the hash column of the trip_id in the filtered_trips dataset

trips_hash['hash_inverse'] = np.nan
for trip in filtered_trips['trip_id'].unique():
    stop_sequence = stop_times[stop_times['trip_id'] == trip].sort_values(by = 'stop_sequence')
    trips_hash.loc[trips_hash['trip_id'] == trip, 'hash'] = hash(tuple(stop_sequence['stop_id']))
    trips_hash.loc[trips_hash['trip_id'] == trip, 'hash_inverse'] = hash(tuple(list(stop_sequence['stop_id'])[::-1]))
trips_hash
trips_hash.to_csv(r'/Users/pol/Desktop/CSV_export/trips_hash.csv', index = False, header=True, encoding='utf-8-sig')

In [113]:
trips_hash = pd.read_csv(datalink + "trips_hash.csv", sep=",")

In [114]:
trips_hash

,route_id,service_id,trip_id,trip_headsign,route_long_name,hash,hash_inverse
0,1,1,88____:007::8841673:8841004:6:559:20201212,Namur,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18
1,1,48,88____:007::8841673:8841004:6:528:20200524,Liège-Guillemins,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18
2,1,48,88____:007::8841004:8841673:6:857:20200524,Liers,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18
3,1,48,88____:007::8841673:8841004:6:728:20200524,Liège-Guillemins,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18
4,1,48,88____:007::8841004:8841673:6:1057:20200524,Liers,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18
...,...,...,...,...,...,...,...
22113,679,716,84____:007::8400131:8400280:3:1720:20200705,Den Haag HS (NL),Den Haag HS (NL) -- Bruxelles-Nord,-4.809365e+18,5.255191e+18
22114,679,716,84____:007::8400280:8821105:4:1600:20200705,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord,-3.326243e+18,8.239505e+18
22115,679,716,88____:007::8821105:8812005:22:1723:20200705,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord,8.867455e+18,-2.368845e+18
22116,679,716,84____:007::8400280:8821105:4:2000:20200705,Bruxelles-Nord,Den Haag HS (NL) -- Bruxelles-Nord,-3.326243e+18,8.239505e+18


In [115]:
# To groupby the trip_id and to order the stop_sequence in an ascending order
# Otherwise, different hash values could correspond to a same stop_sequence (since the stop_sequences of some
# routes are initially in descending order while other stop_sequences are in ascending order)

trip_stop_sequence_sorted = stops_cleaned_stop_times_merge.groupby(['trip_id'], as_index=False).apply(lambda x: x.sort_values('stop_sequence'))
trip_stop_sequence_sorted

trip_id arrival_time  \
0     87690   80____:046::8015345:8849023:2:1011:20210328     10:04:00   
      87323   80____:046::8015345:8849023:2:1011:20210328     10:11:00   
1     87688   80____:046::8015345:8849023:2:1011:20210402     10:04:00   
      87319   80____:046::8015345:8849023:2:1011:20210402     10:11:00   
2     87695   80____:046::8015345:8849023:2:1011:20210405     10:04:00   
...                                                   ...          ...   
30104 209512  88____:L73::8843208:8841400:13:909:20210402     08:54:00   
      209106  88____:L73::8843208:8841400:13:909:20210402     08:57:00   
      208700  88____:L73::8843208:8841400:13:909:20210402     09:01:00   
      208294  88____:L73::8843208:8841400:13:909:20210402     09:05:00   
      207888  88____:L73::8843208:8841400:13:909:20210402     09:09:00   

             departure_time  stop_id  stop_sequence              stop_name  \
0     87690        10:04:00  8015345              1        Aachen Hbf (DE)   
      87323        10:11:00  8849023              2  Hergenrath-FrontiÃ¨re   
1     87688        10:04:00  8015345              1        Aachen Hbf (DE)   
      87319        10:11:00  8849023              2  Hergenrath-FrontiÃ¨re   
2     87695        10:04:00  8015345              1        Aachen Hbf (DE)   
...                     ...      ...            ...                    ...   
30104 209512       08:54:00  8841467              9  Fexhe-Le-Haut-Clocher   
      209106       08:57:00  8841459             10                Momalle   
      208700       09:01:00  8841442             11              Remicourt   
      208294       09:05:00  8841434             12                BlÃ©ret   
      207888       09:09:00  8841400             13                Waremme   

              stop_lat  stop_lon  
0     87690   50.77083  6.105277  
      87323   50.71896  6.041269  
1     87688   50.77083  6.105277  
      87319   50.71896  6.041269  
2     87695   50.77083  6.105277  
...                ...       ...  
30104 209512  50.66405  5.398450  
      209106  50.66991  5.367600  
      208700  50.67861  5.321410  
      208294  50.68507  5.286398  
      207888  50.69455  5.249480  

[440858 rows x 8 columns]

In [116]:
# To put the stop_names per trip_id in a list
trip_stop_sequence = trip_stop_sequence_sorted.groupby('trip_id')['stop_name'].apply(lambda group_series: group_series.tolist()).reset_index()
trip_stop_sequence.rename(columns={'stop_name':'stop_sequence'}, inplace=True)
trip_stop_sequence

,trip_id,stop_sequence
0,80____:046::8015345:8849023:2:1011:20210328,"[Aachen Hbf (DE), Hergenrath-FrontiÃ¨re]"
1,80____:046::8015345:8849023:2:1011:20210402,"[Aachen Hbf (DE), Hergenrath-FrontiÃ¨re]"
2,80____:046::8015345:8849023:2:1011:20210405,"[Aachen Hbf (DE), Hergenrath-FrontiÃ¨re]"
3,80____:046::8015345:8849023:2:1011:20210411,"[Aachen Hbf (DE), Hergenrath-FrontiÃ¨re]"
4,80____:046::8015345:8849023:2:1011:20210416,"[Aachen Hbf (DE), Hergenrath-FrontiÃ¨re]"
...,...,...
30100,88____:L73::8843208:8841004:5:834:20211210,"[FlÃ©malle-Haute, Seraing, OugrÃ©e, Y.RENORY, ..."
30101,88____:L73::8843208:8841400:13:1709:20210402,"[FlÃ©malle-Haute, Seraing, OugrÃ©e, Y.RENORY, ..."
30102,88____:L73::8843208:8841400:13:1809:20210402,"[FlÃ©malle-Haute, Seraing, OugrÃ©e, Y.RENORY, ..."
30103,88____:L73::8843208:8841400:13:809:20210402,"[FlÃ©malle-Haute, Seraing, OugrÃ©e, Y.RENORY, ..."


In [117]:
# To add the stop_sequence of stations to the trips_hash dataset in the trips_hash_stop_sequence dataset by joining on trip_id
trips_hash_stop_sequence = pd.merge(trips_hash, trip_stop_sequence, on='trip_id', how='left')

# To put the columns in a more logical order
trips_hash_stop_sequence = trips_hash_stop_sequence[['route_id', 'route_long_name','service_id','trip_headsign','trip_id','hash', 'hash_inverse','stop_sequence']]
trips_hash_stop_sequence

,route_id,route_long_name,service_id,trip_headsign,trip_id,hash,hash_inverse,stop_sequence
0,1,Liège-Guillemins -- Liers,1,Namur,88____:007::8841673:8841004:6:559:20201212,-4.288363e+18,-6.465769e+18,NaN
1,1,Liège-Guillemins -- Liers,48,Liège-Guillemins,88____:007::8841673:8841004:6:528:20200524,-4.288363e+18,-6.465769e+18,NaN
2,1,Liège-Guillemins -- Liers,48,Liers,88____:007::8841004:8841673:6:857:20200524,-6.465769e+18,-4.288363e+18,NaN
3,1,Liège-Guillemins -- Liers,48,Liège-Guillemins,88____:007::8841673:8841004:6:728:20200524,-4.288363e+18,-6.465769e+18,NaN
4,1,Liège-Guillemins -- Liers,48,Liers,88____:007::8841004:8841673:6:1057:20200524,-6.465769e+18,-4.288363e+18,NaN
...,...,...,...,...,...,...,...,...
22113,679,Den Haag HS (NL) -- Bruxelles-Nord,716,Den Haag HS (NL),84____:007::8400131:8400280:3:1720:20200705,-4.809365e+18,5.255191e+18,NaN
22114,679,Den Haag HS (NL) -- Bruxelles-Nord,716,Bruxelles-Nord,84____:007::8400280:8821105:4:1600:20200705,-3.326243e+18,8.239505e+18,NaN
22115,679,Den Haag HS (NL) -- Bruxelles-Nord,716,Bruxelles-Nord,88____:007::8821105:8812005:22:1723:20200705,8.867455e+18,-2.368845e+18,NaN
22116,679,Den Haag HS (NL) -- Bruxelles-Nord,716,Bruxelles-Nord,84____:007::8400280:8821105:4:2000:20200705,-3.326243e+18,8.239505e+18,NaN


In [118]:
# To count the number of dates for each service_id
service_id_df = calendar_dates.groupby(['service_id'])[['service_id']].count().rename(columns={'service_id':'count_service_id'}).reset_index()
service_id_df

,service_id,count_service_id
0,0,283
1,1,1
2,2,5
3,3,6
4,4,44
...,...,...
4182,6111,243
4183,6112,203
4184,6113,217
4185,6114,26


In [119]:
#regroup the days per service id in a set
service_id_dates = calendar_dates.groupby('service_id')['date'].apply(lambda group_series: set(group_series.tolist())).reset_index()
service_id_dates.rename(columns={'date':'dates'}, inplace=True)
service_id_dates = service_id_dates.merge(service_id_df, on='service_id', how='left')
service_id_dates

,service_id,dates,count_service_id
0,0,"{20211201, 20211202, 20211203, 20211204, 20211...",283
1,1,{20210314},1
2,2,"{20210315, 20210316, 20210317, 20210318, 20210...",5
3,3,"{20210306, 20210307, 20210313, 20210314, 20210...",6
4,4,"{20210304, 20210305, 20210306, 20210308, 20210...",44
...,...,...,...
4182,6111,"{20211201, 20211202, 20211203, 20211204, 20211...",243
4183,6112,"{20211201, 20211202, 20211203, 20211204, 20211...",203
4184,6113,"{20211201, 20211202, 20211203, 20211205, 20211...",217
4185,6114,"{20210304, 20210305, 20210307, 20210308, 20210...",26


In [120]:
# To put the different trip_ids in a list after joining on (route_id, route_long_name, hash and service_id)
route_hash_freq = trips_hash_stop_sequence.groupby(['route_id','route_long_name','hash', 'hash_inverse', 'service_id'])['trip_id'].apply(lambda group_series: group_series.tolist()).reset_index()
route_hash_freq

,route_id,route_long_name,hash,hash_inverse,service_id,trip_id
0,1,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18,48,"[88____:007::8841004:8841673:6:857:20200524, 8..."
1,1,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18,146,[88____:007::8841004:8841673:6:57:20200525]
2,1,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18,1,[88____:007::8841673:8841004:6:559:20201212]
3,1,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18,48,"[88____:007::8841673:8841004:6:528:20200524, 8..."
4,2,Liège-Guillemins -- Namur,-6.464971e+18,-3.538426e+18,1,[88____:046::8841004:8863008:20:722:20201212]
...,...,...,...,...,...,...
6325,678,Den Haag HS (NL) -- Bruxelles-Midi,8.032946e+18,5.656301e+18,1719,[84____:007::8400530:8400280:2:1320:20200507]
6326,679,Den Haag HS (NL) -- Bruxelles-Nord,-4.809365e+18,5.255191e+18,716,"[84____:007::8400131:8400280:3:920:20200705, 8..."
6327,679,Den Haag HS (NL) -- Bruxelles-Nord,-3.326243e+18,8.239505e+18,716,"[84____:007::8400280:8821105:4:1200:20200705, ..."
6328,679,Den Haag HS (NL) -- Bruxelles-Nord,2.556917e+18,4.170221e+18,716,"[88____:007::8812005:8400131:23:818:20200705, ..."


In [121]:
# To add the sequence of stops to the route_hash_freq dataset
route_hash_freq = pd.merge(route_hash_freq, trips_hash_stop_sequence[['route_id','hash', 'hash_inverse', 'service_id','stop_sequence']], on=['route_id', 'hash', 'hash_inverse', 'service_id'], how='left')
route_hash_freq = route_hash_freq.drop_duplicates( subset = ['route_id', 'hash', 'service_id'], keep = 'first')

route_hash_freq

,route_id,route_long_name,hash,hash_inverse,service_id,trip_id,stop_sequence
0,1,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18,48,"[88____:007::8841004:8841673:6:857:20200524, 8...",NaN
8,1,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18,146,[88____:007::8841004:8841673:6:57:20200525],NaN
9,1,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18,1,[88____:007::8841673:8841004:6:559:20201212],NaN
10,1,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18,48,"[88____:007::8841673:8841004:6:528:20200524, 8...",NaN
19,2,Liège-Guillemins -- Namur,-6.464971e+18,-3.538426e+18,1,[88____:046::8841004:8863008:20:722:20201212],NaN
...,...,...,...,...,...,...,...
22105,678,Den Haag HS (NL) -- Bruxelles-Midi,8.032946e+18,5.656301e+18,1719,[84____:007::8400530:8400280:2:1320:20200507],NaN
22106,679,Den Haag HS (NL) -- Bruxelles-Nord,-4.809365e+18,5.255191e+18,716,"[84____:007::8400131:8400280:3:920:20200705, 8...",NaN
22109,679,Den Haag HS (NL) -- Bruxelles-Nord,-3.326243e+18,8.239505e+18,716,"[84____:007::8400280:8821105:4:1200:20200705, ...",NaN
22112,679,Den Haag HS (NL) -- Bruxelles-Nord,2.556917e+18,4.170221e+18,716,"[88____:007::8812005:8400131:23:818:20200705, ...",NaN


In [122]:
# To calculate the number of trip ids in the list of trip_ids and to add it as a new column
number_trip_ids = []
for list_trip_ids in route_hash_freq['trip_id']:
    count = len(list_trip_ids)
    number_trip_ids.append(count)
route_hash_freq['number_trip_ids'] = number_trip_ids

route_hash_freq

,route_id,route_long_name,hash,hash_inverse,service_id,trip_id,stop_sequence,number_trip_ids
0,1,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18,48,"[88____:007::8841004:8841673:6:857:20200524, 8...",NaN,8
8,1,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18,146,[88____:007::8841004:8841673:6:57:20200525],NaN,1
9,1,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18,1,[88____:007::8841673:8841004:6:559:20201212],NaN,1
10,1,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18,48,"[88____:007::8841673:8841004:6:528:20200524, 8...",NaN,9
19,2,Liège-Guillemins -- Namur,-6.464971e+18,-3.538426e+18,1,[88____:046::8841004:8863008:20:722:20201212],NaN,1
...,...,...,...,...,...,...,...,...
22105,678,Den Haag HS (NL) -- Bruxelles-Midi,8.032946e+18,5.656301e+18,1719,[84____:007::8400530:8400280:2:1320:20200507],NaN,1
22106,679,Den Haag HS (NL) -- Bruxelles-Nord,-4.809365e+18,5.255191e+18,716,"[84____:007::8400131:8400280:3:920:20200705, 8...",NaN,3
22109,679,Den Haag HS (NL) -- Bruxelles-Nord,-3.326243e+18,8.239505e+18,716,"[84____:007::8400280:8821105:4:1200:20200705, ...",NaN,3
22112,679,Den Haag HS (NL) -- Bruxelles-Nord,2.556917e+18,4.170221e+18,716,"[88____:007::8812005:8400131:23:818:20200705, ...",NaN,3


In [123]:
# To merge the route_hash_freq df with the service_id_dates to get the sets of corresponding dates
route_hash_service_freq = pd.merge(route_hash_freq, service_id_dates, on='service_id', how='left')
route_hash_service_freq

,route_id,route_long_name,hash,hash_inverse,service_id,trip_id,stop_sequence,number_trip_ids,dates,count_service_id
0,1,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18,48,"[88____:007::8841004:8841673:6:857:20200524, 8...",NaN,8,"{20210408, 20210409, 20210407}",3.0
1,1,Liège-Guillemins -- Liers,-6.465769e+18,-4.288363e+18,146,[88____:007::8841004:8841673:6:57:20200525],NaN,1,"{20210501, 20210502, 20210410, 20210411, 20210...",14.0
2,1,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18,1,[88____:007::8841673:8841004:6:559:20201212],NaN,1,{20210314},1.0
3,1,Liège-Guillemins -- Liers,-4.288363e+18,-6.465769e+18,48,"[88____:007::8841673:8841004:6:528:20200524, 8...",NaN,9,"{20210408, 20210409, 20210407}",3.0
4,2,Liège-Guillemins -- Namur,-6.464971e+18,-3.538426e+18,1,[88____:046::8841004:8863008:20:722:20201212],NaN,1,{20210314},1.0
...,...,...,...,...,...,...,...,...,...,...
6325,678,Den Haag HS (NL) -- Bruxelles-Midi,8.032946e+18,5.656301e+18,1719,[84____:007::8400530:8400280:2:1320:20200507],NaN,1,"{20211201, 20211202, 20211203, 20211206, 20211...",146.0
6326,679,Den Haag HS (NL) -- Bruxelles-Nord,-4.809365e+18,5.255191e+18,716,"[84____:007::8400131:8400280:3:920:20200705, 8...",NaN,3,NaN,NaN
6327,679,Den Haag HS (NL) -- Bruxelles-Nord,-3.326243e+18,8.239505e+18,716,"[84____:007::8400280:8821105:4:1200:20200705, ...",NaN,3,NaN,NaN
6328,679,Den Haag HS (NL) -- Bruxelles-Nord,2.556917e+18,4.170221e+18,716,"[88____:007::8812005:8400131:23:818:20200705, ...",NaN,3,NaN,NaN


In [124]:
'''Get the distinct stop sequences for all routes to create the possible track combinations later on'''
distinct_stop_sequences = route_hash_service_freq.drop_duplicates(subset = ["route_id", 'hash'])[['route_id','hash','route_long_name','stop_sequence']]
distinct_stop_sequences

'Get the distinct stop sequences for all routes to create the possible track combinations later on'

,route_id,hash,route_long_name,stop_sequence
0,1,-6.465769e+18,Liège-Guillemins -- Liers,NaN
2,1,-4.288363e+18,Liège-Guillemins -- Liers,NaN
4,2,-6.464971e+18,Liège-Guillemins -- Namur,NaN
9,2,-5.388870e+18,Liège-Guillemins -- Namur,NaN
11,2,-4.718380e+18,Liège-Guillemins -- Namur,NaN
...,...,...,...,...
6320,678,8.032946e+18,Den Haag HS (NL) -- Bruxelles-Midi,NaN
6326,679,-4.809365e+18,Den Haag HS (NL) -- Bruxelles-Nord,NaN
6327,679,-3.326243e+18,Den Haag HS (NL) -- Bruxelles-Nord,NaN
6328,679,2.556917e+18,Den Haag HS (NL) -- Bruxelles-Nord,NaN


In [125]:
'''creates all the sequences of routes possible to reconstruct the real route'''
import copy
#create an empty df for the process of route creation
route_creation  = pd.DataFrame()
for route_id in distinct_stop_sequences['route_id'].unique():
    #create a copy of the df with only the route considered in the loop iteration
    routes_with_route_id = distinct_stop_sequences[distinct_stop_sequences['route_id'] == route_id].copy()
    #put the two parts of the route name into a list ex: 'Namur -- Liège' to ['Namur', 'Liège']
    route_long_name = routes_with_route_id.iloc[0]['route_long_name'].split(' -- ')
    #create a df where only the routes that have an end stop as their first element of the sequence
    route_creation_route_id = routes_with_route_id[routes_with_route_id['stop_sequence'].apply(lambda x: any(item for item in route_long_name if item == x[0]))][['route_id','hash', 'stop_sequence']].copy()
    #make the hash column as a column of lists
    route_creation_route_id['hash'] = route_creation_route_id['hash'].apply(lambda x: [x])
    route_creation_route_id = route_creation_route_id.reset_index(drop=True)
    #to stop the while loop when all the routes are complete in the df for the route_id of the loop iteration
    complete_routes = 0
    while complete_routes < len(route_creation_route_id.index):
        #use a deepcopy to not impact the iterrows of the main loop
        route_creation_deep_copy = copy.deepcopy(route_creation_route_id)
        for index_original, route_part in route_creation_deep_copy.iterrows():
            #if the trip is not complete (contains begin and end), add some other parts
            if not(set(route_long_name).issubset(route_part['stop_sequence'])):
                #create a dataframe of the possible extentions for each route_part
                #here checks if the extention is the next part of the route and also that no other station are repeated in the sequence if this extention is added(otherwise it might cause an infinite loop)
                possible_extentions = routes_with_route_id[routes_with_route_id['stop_sequence'].apply(lambda x: any(item for item in [route_part['stop_sequence'][-1]] if (item == x[0]) and not(set(x[1:]) & set(route_part['stop_sequence']))))].copy()
                #checks whether it is possible to extend the route_part
                if not (possible_extentions.empty):
                    #if so, extend it with every single possibilities
                    for index_extention, possible_extention in possible_extentions.iterrows():
                        #must create a deepcopy, otherwise the orignal hash list will change as well (mutable)
                        updated_hash = copy.deepcopy(route_part['hash'])
                        updated_hash.append(possible_extention['hash'])
                        updated_route_sequence = route_part['stop_sequence'] + possible_extention['stop_sequence'][1:]
                        route_creation_route_id.loc[max(route_creation_route_id.index)+1] = [route_id, updated_hash, updated_route_sequence] 
                #then delete the route with the index (see loop here above)
                route_creation_route_id = route_creation_route_id.drop(index = index_original)
            #when the route is complete adds one for the while loop to know when to stop   
            else:
                complete_routes += 1
    #adds all the possible routes created with the trips of the route_id of the main loop
    route_creation = route_creation.append(route_creation_route_id, ignore_index = True)

'creates all the sequences of routes possible to reconstruct the real route'

TypeError: 'float' object is not subscriptable

In [ ]:
'''calculates the frequency of the constructed routes just made in the route_creation dataframe'''
#put the default value of the frequency to 0
route_creation['frequency'] = 0
for index_sequence, sequence in route_creation[['route_id','hash']].iterrows():
    #initialize the varibles
    sequence_frequency = 0
    set_common_service_id = set()
    #take all the trips with that route_id
    selected_route_hash_service_freq = route_hash_service_freq[route_hash_service_freq['route_id'] == sequence['route_id']]
    for trip_hash in sequence['hash']:
        corresponding_service_id = set()
        #loops on the trips with that hash value and put their service_id in the set corresponding_service_id
        for service_id in selected_route_hash_service_freq[selected_route_hash_service_freq['hash'] == trip_hash]['service_id']:
            corresponding_service_id.add(service_id)
        #here, we check which service_id are common for each hash value of the new route, so we know when that route actually operates
        #if the set is empty, initiate it
        if not set_common_service_id:
            set_common_service_id = corresponding_service_id
        #if the set is not empty take the intersection, to only have the service_ids that are common
        else:
            set_common_service_id = set_common_service_id.intersection(corresponding_service_id)
    if set_common_service_id:
        #only select the trips with the hash value contained in the sequence and with the same route_id
        containing_hash = selected_route_hash_service_freq[selected_route_hash_service_freq['hash'].apply(lambda x: any(item for item in sequence['hash'] if x == item))]
        #loop over each service_id that were common during the trip
        for service_id in set_common_service_id:
            #take all the trips with the selected hashes that contain the service_id of the loop
            containing_service_id = containing_hash[containing_hash['service_id'].apply(lambda x: any(item for item in [service_id] if item == x))]
            #takes the out of all the daily frequency the minimum of all the trips (otherwise they won't be able to run completely, since some part do not run anymore)
            containing_service_id_grouped = containing_service_id.groupby(['hash'], as_index = False)['number_trip_ids'].sum()
            #adds the minimum number of trips per day multiplied by the number of days in the service_id
            sequence_frequency += containing_service_id_grouped['number_trip_ids'].min() * service_id_df[service_id_df['service_id'] == service_id].iloc[0]['count_service_id']
        #adds the frequency in of the new route sequence
        route_creation.loc[index_sequence, 'frequency'] = sequence_frequency

In [ ]:
'''Makes the hash and hash_inverse of route_creation'''
#To copy the filtered_trips dataset
route_creation_hash = route_creation.copy()

#To create a column called hash that contains NaN values
route_creation_hash['hash'] = np.nan

#For each trip_id in filtered_trips, the stop_sequence that gets calculated is the subset of the stop_time dataset for that trip_id. 
#The tuple that results from the stop_id column of this subset dataset contains all the stop_ids that get served by this trip_id. 
#The hash value of the tuple of the stop_id column is calculated and is placed in the hash column of the trip_id in the filtered_trips dataset

route_creation_hash['hash_inverse'] = np.nan
for index, route_sequence in route_creation_hash.iterrows():
    route_creation_hash.loc[index, 'hash'] = hash(tuple(route_sequence['stop_sequence']))
    route_creation_hash.loc[index, 'hash_inverse'] = hash(tuple(list(route_sequence['stop_sequence'])[::-1]))
route_creation_hash

In [ ]:
'''Regroup the routes that are the same (even though they are in the opposite direction)'''
route_creation_max_hash = route_creation_hash.copy()
route_creation_max_hash['max_hash'] = route_creation_max_hash[['hash', 'hash_inverse']].max(axis=1)
#create a df that sums the frequence of the trips going from opposite directions
route_creation_max_hash_freq = route_creation_max_hash.groupby(['route_id','max_hash'], as_index = False)[['frequency']].sum()
#renames the max_hash column into hash so it the dataframe can be merged with route_hash_without_freq
route_creation_max_hash_freq = route_creation_max_hash_freq.rename(columns = {'max_hash':'hash'})
#drops the column freq_sequence_route because the one that is of interest is in route_creation_max_hash_freq
route_hash_without_freq = route_creation_hash.copy().drop(['frequency'], axis = 1)
route_hash_without_freq = route_hash_without_freq.drop_duplicates(subset=['route_id', 'hash'])
route_hash_freq_combined_first_merge = pd.merge(route_creation_max_hash_freq, route_hash_without_freq, on=['route_id', 'hash'], how='left')
route_hash_freq_combined_first_merge = route_hash_freq_combined_first_merge.drop(['hash_inverse'], axis = 1)
#selects the part of the dataset that doesn't have NaN (because their hash_value that was max was the one in hash_inverse and it didn't exist in the other df), so we can concatenate it with the part that had NaN later
route_hash_freq_first_part = route_hash_freq_combined_first_merge[pd.notnull(route_hash_freq_combined_first_merge['stop_sequence'])]
#selects one part the part of the dataset that does have NaN, so we can concatenate it with the part that has no NaN.
#but first, we will need to fill those NaN values (done in the code lines behind this one)
route_hash_freq_second_part = route_hash_freq_combined_first_merge[pd.isnull(route_hash_freq_combined_first_merge['stop_sequence'])][['route_id', 'hash', 'frequency']]
#renames the hash column into hash_inverse so it the dataframe can be merged with route_hash_without_freq (because it didn't work with 'hash')
route_hash_freq_second_part = route_hash_freq_second_part.rename(columns = {'hash':'hash_inverse'})
route_hash_freq_second_part = pd.merge(route_hash_freq_second_part, route_hash_without_freq, on=['route_id', 'hash_inverse'], how='left')
#the hash that is of interest in the final df will be hash and not hash_inverse
route_hash_freq_second_part  = route_hash_freq_second_part.drop(['hash_inverse'], axis = 1)
route_hash_freq_combined_not_sorted = pd.concat([route_hash_freq_first_part, route_hash_freq_second_part])
route_hash_freq_combined = route_hash_freq_combined_not_sorted.sort_values(by = ['route_id'])
route_hash_freq_combined = route_hash_freq_combined.reset_index(drop = True)

In [ ]:
'''Deletes the routes that do not represent 10% of the total route frequency and creates new route, if some of them are different'''
#calculates the total frequency per route_id
frequency_each_route = route_hash_freq_combined.groupby(['route_id'], as_index = False)['frequency'].sum()
frequency_treshold = frequency_each_route.copy()
#calculates the treshold
frequency_treshold['frequency'] = frequency_treshold['frequency']/10
frequency_treshold.rename(columns = {'frequency':'frequency_treshold'}, inplace = True)
route_hash_freq_treshold = route_hash_freq_combined.merge(frequency_treshold, on='route_id', how = 'left')
#find the sequences that are not more than 10% of the route frequency and delete them
index_names = route_hash_freq_treshold[route_hash_freq_treshold['frequency'] < route_hash_freq_treshold['frequency_treshold']].index
route_hash_freq_treshold.drop(index_names, inplace = True)
#selects the sequences that are not the most frequent per route_id
sequences_max_freq = route_hash_freq_treshold.groupby(['route_id'],as_index = False)['frequency'].max()
sequences_max_freq.rename(columns = {'frequency':'max_frequency'}, inplace = True)
sequences_max_freq_merged = route_hash_freq_treshold.merge(sequences_max_freq, on='route_id', how='left')
sequences_non_max_freq_index = sequences_max_freq_merged[sequences_max_freq_merged['frequency'] != sequences_max_freq_merged['max_frequency']].index
#those selected sequences get a new route_id that starts from routes['route_id'].max() + 1 and increments by one for each new route
route_id_creation =  route_hash_freq_combined['route_id'].max() + 1
new_route_id_column = list(range(route_id_creation, route_id_creation + len(sequences_non_max_freq_index)))    
sequences_max_freq_merged.loc[sequences_non_max_freq_index, 'route_id'] = new_route_id_column
sequences_max_freq_merged = sequences_max_freq_merged.sort_values(by=['route_id'],ignore_index=True)
final_routes = sequences_max_freq_merged.drop(columns=['hash', 'frequency', 'frequency_treshold', 'max_frequency'])
final_routes

In [ ]:
'''Makes a set that can be used for building the edges of the graph using Networkx package
This means:
...
The df_for_edges dataset contains for each route_id one trip_id. However, there are still trip_id's with the same sequence of stations in df_for_edges
'''
#takes the list stop sequence and make it a new column for each stop
stop_sequence_values = final_routes.apply(lambda x: pd.Series(x['stop_sequence']),axis=1).stack().reset_index(level=1, drop=True)
stop_sequence_values.name = 'stop_sequence'
final_routes_stops = final_routes.drop('stop_sequence', axis=1).join(stop_sequence_values)
final_routes_stops = final_routes_stops.reset_index(drop=True)
#Creates a shifted instance of the df to use it for the final result
final_routes_stops_shifted = final_routes_stops.shift()
#Check if which of the rows are followed by a row with the same trip_id
final_routes_stops_shifted['match'] = final_routes_stops_shifted['route_id'].eq(final_routes_stops['route_id'])
#Drop the rows for which this condition is not satisfied
final_routes_stops_shifted.drop(final_routes_stops_shifted[final_routes_stops_shifted['match'] == False].index, inplace = True)
final_routes_stops_shifted.rename(columns=
  {"stop_sequence": "stop_name_1",
  "stop_name": "stop_name_1"}, inplace=True)
#Joins the df with its shifted version sothat each sequence of two stations is represented in the table as a row
df_for_edges = final_routes_stops_shifted.join(final_routes_stops[['stop_sequence']], lsuffix='_caller', rsuffix='_other', how='left')
df_for_edges.rename(columns=
  {"stop_sequence": "stop_name_2",
  "stop_name": "stop_name_2"}, inplace=True)

df_for_edges['route_id'] = df_for_edges['route_id'].astype(np.int64)
df_for_edges = df_for_edges.drop_duplicates()
df_for_edges = df_for_edges[['route_id','stop_name_1', 'stop_name_2']]
df_for_edges = df_for_edges.reset_index(drop=True)
df_for_edges

## **Construction of the Belgian railway system in the L-space representation and the calculation of the global characteristics of this Belgian railway system in the L-space representation**



In [ ]:
G = nx.Graph()
G = nx.from_pandas_edgelist(df_for_edges, 'stop_name_1', 'stop_name_2')

* Weights toevoegen

In [ ]:
#Assign node weights to graph
for i in range(len(stops)):
    for n, d in G.nodes(data=True):
        if stops.stop_name[i] == n:
            G.nodes[n]['id']=stops.stop_id[i];
            G.nodes[n]['latitude']=stops.stop_lat[i];
            G.nodes[n]['longitude']=stops.stop_lon[i];
            
G.nodes(data=True)

In [ ]:
#Assign the route_id as a list (because there can be more than one per edge) on each edge
routes_on_edge = {}
for node1,node2, nodeData in G.edges(data = True):
    if (node1, node2) not in routes_on_edge:
            routes_on_edge[(node1, node2)] = set()
    route_id_node1_node2= df_for_edges[(df_for_edges['stop_name_1'] == node1 ) & (df_for_edges['stop_name_2'] == node2 )]['route_id']
    route_id_node2_node1= df_for_edges[(df_for_edges['stop_name_1'] == node2 ) & (df_for_edges['stop_name_2'] == node1 )]['route_id']
    for route_id in pd.concat([route_id_node1_node2, route_id_node2_node1]):
        routes_on_edge[(node1, node2)].add(route_id)
    nodeData['route_id'] = routes_on_edge[(node1, node2)] 

G.edges(data=True)

* Characteristics and global network metrics:




In [ ]:
#aantal nodes
len(G.nodes())

In [ ]:
#degrees van de nodes
G.degree()

In [ ]:
# average degree (indicating the number of directions a commuter can take from that given station)
sum = 0
for n in G.nodes():
    sum = sum + G.degree[n]
print(sum/len(G.nodes()))

In [ ]:
#The existing number of edges
len(G.edges())

In [ ]:
#gamma_index
gamma_index = len(G.edges()) / (3*(len(G.nodes()) - 2))
gamma_index

In [ ]:
#Mean local clustering coefficient
#indicating for a random station the estimation of the division of the number of neighbouring stations of a station that are connected divided by the possible number of connections between the neighbours of that station
print ("mean local clustering:", nx.average_clustering (G))

In [ ]:
#The degree distribution

#Sort degree values
degree_sequence=sorted([d for n,d in G.degree()], reverse=True) 
#Count number of frequences of degree values
degreeCount=collections.Counter(degree_sequence)
#Define variables deg and cnt and probability
deg, cnt = zip(*degreeCount.items())
probability = []
for v in cnt:
    pv= v/len(G.nodes())
    probability.append(pv)

#plot degree probability histogram
fig, ax = plt.subplots(figsize=(5, 5))
plt.bar(deg, probability, width=0.95, color='lightgrey')

#plot degree distribution
plt.plot(deg, probability, color="b")

#graph settings
plt.title("Degree distribution")
plt.ylabel("P (k)")
plt.xlabel("k")
ax.set_xticks([d for d in deg])
ax.set_xticklabels(deg)

#count
fig, ax = plt.subplots(figsize=(5, 5))
plt.bar(deg, cnt, width=0.95, color='lightgrey')

plt.plot(deg, cnt, color="b")

#graph settings
plt.title("Degree count")
plt.ylabel("Count")
plt.xlabel("k")
ax.set_xticks([d for d in deg])
ax.set_xticklabels(deg)

plt.show()

In [ ]:
#Detection of the scale-free property (A PTN with the scale-free property consists of a large number of vertices that exhibit low degree and a few number of vertices that exhibit very high degree)

The scale-free property is present in the Belgian railway network because the majority of nodes exhibit a low degree value equal to 2 and almost none of the nodes exhibit a degree value equal to 5 or higher

In [ ]:
nx.draw_networkx(G, with_labels = False, node_size = 5)

In [ ]:
#Average path length (In the L-space it represents the average of all the shortest paths to be transversed between all public transport stop pairs which can be reached by means of one or multiple routes)
#print(nx.average_shortest_path_length(G))

In [ ]:
#Assortativity of the network 
#In an assortative network, high-degree vertices tend to be connected and low-degree vertices tend to stick together as well. 
#In a disassortative network conversely, high-degree and low-degree vertices tend to be connected to each other instead of being connected to vertices of the same kind).

print(nx.degree_pearson_correlation_coefficient(G))

## **Preparation for the L-space representation of the French railway system**

### To construct the datasets that combine the information of the different French railway systems

In [126]:
''' To construct the agency_France df'''
agency_frames = [agency_TGV, agency_Intercites, agency_TER, agency_Transilien]
agency_France = pd.concat(agency_frames)
agency_France = agency_France.drop_duplicates()
agency_France

' To construct the agency_France df'

,agency_id,agency_name,agency_url,agency_timezone,agency_lang
0,OCESN,SNCF,http://www.sncf.com,Europe/Paris,fr
0,OCEdefault,SNCF,http://www.sncf.com,Europe/Paris,fr
0,DUA853,Paris Nord,http://www.transilien.com,Europe/Paris,fr
1,DUA854,Paris St Lazare,http://www.transilien.com,Europe/Paris,fr
2,DUA803,RER C,http://www.transilien.com,Europe/Paris,fr
3,DUA801,RER A,http://www.transilien.com,Europe/Paris,fr
4,DUA805,RER E,http://www.transilien.com,Europe/Paris,fr
5,DUA804,RER D,http://www.transilien.com,Europe/Paris,fr
6,DUA850,Paris Est,http://www.transilien.com,Europe/Paris,fr
7,DUA855,La Verrière - La Défense,http://www.transilien.com,Europe/Paris,fr


In [127]:
''' To construct the stops_France df.'''
stops_frames = [stops_cleaned_TGV, stops_cleaned_Intercites, stops_cleaned_TER, stops_cleaned_Transilien]
stops_France = pd.concat(stops_frames)
stops_France = stops_France.drop_duplicates()
stops_France

' To construct the stops_France df.'

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
1,80143099,Offenburg,NaN,48.476500,7.946800,NaN,NaN,0,StopArea:OCE80143099
3,80143198,Lahr (Schwarzw),NaN,48.340600,7.835300,NaN,NaN,0,StopArea:OCE80143198
5,80143313,Emmendingen,NaN,48.119300,7.847900,NaN,NaN,0,StopArea:OCE80143313
7,80143503,Freiburg (Breisgau) Hbf,NaN,47.997900,7.841600,NaN,NaN,0,StopArea:OCE80143503
9,82001000,Luxembourg,NaN,49.599722,6.134722,NaN,NaN,0,StopArea:OCE82001000
...,...,...,...,...,...,...,...,...,...
1037,8743091,CHAMPGOULIN,NaN,48.791684,3.256354,NaN,NaN,0,StopArea:DUA8743091
1038,8743090,EUSTACHE LENOIR,NaN,48.798279,3.267056,NaN,NaN,0,StopArea:DUA8743090
1039,8743088,JOUY SUR MORIN MONUMENT,NaN,48.790683,3.274062,NaN,NaN,0,StopArea:DUA8743088
1040,8711639,LA FERTE GAUCHER,NaN,48.782711,3.297558,NaN,NaN,0,StopArea:DUA8711639


In [128]:
''' To construct the transfers_France df.'''
transfers_frames = [transfers_cleaned_TGV, transfers_cleaned_Intercites, transfers_cleaned_TER, transfers_cleaned_Transilien]
transfers_France = pd.concat(transfers_frames)
transfers_France = transfers_France.drop_duplicates()
transfers_France

' To construct the transfers_France df.'

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_route_id,to_route_id
0,80143099,80143099,2,300,NaN,NaN
1,80143198,80143198,2,300,NaN,NaN
2,80143313,80143313,2,300,NaN,NaN
3,80143503,80143503,2,300,NaN,NaN
4,82001000,82001000,2,300,NaN,NaN
...,...,...,...,...,...,...
95,8727116,8769732,2,966,NaN,NaN
96,8711371,8775830,2,207,NaN,NaN
97,8775830,8711371,2,460,NaN,NaN
98,8727605,8753413,2,249,NaN,NaN


In [129]:
''' To construct the routes_France df.'''
routes_frames = [routes_TGV, routes_Intercites, routes_filtered_TER, routes_filtered_Transilien]
routes_France = pd.concat(routes_frames)
routes_France

' To construct the routes_France df.'

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,OCESN-1041887,OCESN,633,SUD-EST,NaN,2,NaN,NaN,NaN
1,OCESN-1105915,OCESN,333,EST,NaN,2,NaN,NaN,NaN
2,OCESN-1105916,OCESN,433A,ATL-NORD,NaN,2,NaN,NaN,NaN
3,OCESN-1109949,OCESN,533-00,NORD,NaN,2,NaN,NaN,NaN
4,OCESN-1209971,OCESN,803A,Paris - Metz/Luxembourg Allemagne,NaN,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
18,DUA800853022,DUA853,K,Gare du Nord - Crépy en Valois,NaN,2,NaN,AE9A00,000000
19,DUA800854041,DUA854,J,Gare St-Lazare - Ermont Eaubonne / Vernon / Gi...,NaN,2,NaN,B6BF00,ffffff
22,DUA810802061,DUA802,B,Aéroport CDG 2 / Mitry Claye - Robinson / St-R...,NaN,2,NaN,4B92DB,ffffff
23,DUA810801041,DUA801,A,Cergy Le Haut / Poissy / St-Germain-en-Laye - ...,NaN,2,NaN,F7403A,ffffff


In [130]:
''' To construct the trips_France df.'''
trips_frames = [trips_cleaned_TGV, trips_cleaned_Intercites, trips_cleaned_TER, trips_cleaned_Transilien]
trips_France = pd.concat(trips_frames)
trips_France 

' To construct the trips_France df.'

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,OCESN-1209972,TGV-1,OCESN2052F198219:2021-02-19T18:51:30Z,2052,NaN,1.0,NaN
1,OCESN-1209972,TGV-2,OCESN2052F224808:2021-02-20T18:50:55Z,2052,NaN,2.0,NaN
2,OCESN-1209972,TGV-3,OCESN2060F160675:2021-02-19T18:51:30Z,2060,0.0,3.0,NaN
3,OCESN-1209972,TGV-4,OCESN2060F192057:2021-02-21T18:50:52Z,2060,0.0,4.0,NaN
4,OCESN-1209972,TGV-5,OCESN2060F212734:2021-02-21T18:50:52Z,2060,0.0,5.0,NaN
...,...,...,...,...,...,...,...
34298,DUA800850511,Transilien-8612,DUASN412671R01001-1_77502,412671,1.0,NaN,NaN
34299,DUA800850511,Transilien-9699,DUASN850192R01001-1_77503,850192,1.0,NaN,NaN
34300,DUA800850511,Transilien-9699,DUASN850193R01001-1_77504,850193,1.0,NaN,NaN
34301,DUA800850511,Transilien-8798,DUASN887131R01001-1_77505,887131,1.0,NaN,NaN


In [131]:
''' To construct the stop_times_France df.'''
stop_times_frames = [stop_times_cleaned_TGV, stop_times_cleaned_Intercites, stop_times_cleaned_TER, stop_times_cleaned_Transilien]
stop_times_France = pd.concat(stop_times_frames)
stop_times_France = stop_times_France.drop_duplicates()
stop_times_France

' To construct the stop_times_France df.'

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,OCESN2052F198219:2021-02-19T18:51:30Z,17:12:00,17:12:00,87212027,0,NaN,0,1,NaN
1,OCESN2052F198219:2021-02-19T18:51:30Z,19:05:00,19:05:00,87113001,1,NaN,1,0,NaN
2,OCESN2052F224808:2021-02-20T18:50:55Z,17:08:00,17:08:00,87212027,0,NaN,0,1,NaN
3,OCESN2052F224808:2021-02-20T18:50:55Z,19:05:00,19:05:00,87113001,1,NaN,1,0,NaN
4,OCESN2060F160675:2021-02-19T18:51:30Z,18:55:00,18:55:00,87212027,0,NaN,0,1,NaN
...,...,...,...,...,...,...,...,...,...
346694,DUASN887133R01001-1_77506,25:51:00,25:51:00,8711369,11,NaN,0,0,NaN
346695,DUASN887133R01001-1_77506,26:01:00,26:01:00,8711321,12,NaN,0,0,NaN
346696,DUASN887133R01001-1_77506,26:23:00,26:23:00,8711320,13,NaN,0,0,NaN
346697,DUASN887133R01001-1_77506,26:36:00,26:36:00,8765479,14,NaN,0,0,NaN


In [132]:
''' To construct the calendar_France df.'''
calendar_frames = [calendar_cleaned_TGV, calendar_cleaned_Intercites, calendar_cleaned_TER, calendar_cleaned_Transilien]
calendar_France = pd.concat(calendar_frames)
#calendar_France 

' To construct the calendar_France df.'

In [133]:
''' To construct the calendar_dates_France df.'''
calendar_dates_frames = [calendar_dates_cleaned_TGV, calendar_dates_cleaned_Intercites, calendar_dates_cleaned_TER, calendar_dates_cleaned_Transilien]
calendar_dates_France = pd.concat(calendar_dates_frames)
#calendar_dates_France 

' To construct the calendar_dates_France df.'

### Exploratory data analysis with the French railway data

In [134]:
'''To calculate the number of unique route_ids in the French railway system '''
set_routes_France = {r for r in routes_France['route_id']}
print("There are",len(set_routes_France), "unique routes in the French railway system.")

'To calculate the number of unique route_ids in the French railway system '

There are 339 unique routes in the French railway system.


In [135]:
'''To calculate the number of stations in the French railway system '''
set_stations_France = {s for s in stops_France['stop_id']}
print("There are",len(set_stations_France), "unique stations in the French railway system.")

'To calculate the number of stations in the French railway system '

There are 2942 unique stations in the French railway system.


## **Preparation for the L-space representation of the French railway system**

In [136]:
'''To merge a selection of the trips dataset and a selection of the routes dataset on route_id'''
trips_routes_France = pd.merge(trips_France[['route_id','service_id','trip_id', 'trip_headsign']], routes_France[['route_id', 'route_short_name', 'route_long_name']], on='route_id')
trips_routes_France

'To merge a selection of the trips dataset and a selection of the routes dataset on route_id'

,route_id,service_id,trip_id,trip_headsign,route_short_name,route_long_name
0,OCESN-1209972,TGV-1,OCESN2052F198219:2021-02-19T18:51:30Z,2052,807A,Paris - Strasbourg/Alsace Allemagne
1,OCESN-1209972,TGV-2,OCESN2052F224808:2021-02-20T18:50:55Z,2052,807A,Paris - Strasbourg/Alsace Allemagne
2,OCESN-1209972,TGV-3,OCESN2060F160675:2021-02-19T18:51:30Z,2060,807A,Paris - Strasbourg/Alsace Allemagne
3,OCESN-1209972,TGV-4,OCESN2060F192057:2021-02-21T18:50:52Z,2060,807A,Paris - Strasbourg/Alsace Allemagne
4,OCESN-1209972,TGV-5,OCESN2060F212734:2021-02-21T18:50:52Z,2060,807A,Paris - Strasbourg/Alsace Allemagne
...,...,...,...,...,...,...
34951,DUA800803071,Transilien-8668,DUASN148676F01001-1_109435,VICK,C,Pontoise / Versailles R. Gauche / St-Quentin e...
34952,DUA800803071,Transilien-9366,DUASN148678F02001-1_109436,VICK,C,Pontoise / Versailles R. Gauche / St-Quentin e...
34953,DUA800803071,Transilien-9744,DUASN148678F02002-1_109437,VICK,C,Pontoise / Versailles R. Gauche / St-Quentin e...
34954,DUA800803071,Transilien-9366,DUASN148686F02001-1_109438,VICK,C,Pontoise / Versailles R. Gauche / St-Quentin e...


In [137]:
'''To merge a selection of the stop_times_France dataset with a selection of the stop_France dataset'''
stop_times_stops_France = pd.merge(stop_times_France[['trip_id','arrival_time', 'departure_time','stop_id','stop_sequence']], stops_France[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']], on='stop_id')
stop_times_stops_France

'To merge a selection of the stop_times_France dataset with a selection of the stop_France dataset'

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon
0,OCESN2052F198219:2021-02-19T18:51:30Z,17:12:00,17:12:00,87212027,0,Strasbourg,48.584532,7.734793
1,OCESN2052F224808:2021-02-20T18:50:55Z,17:08:00,17:08:00,87212027,0,Strasbourg,48.584532,7.734793
2,OCESN2060F160675:2021-02-19T18:51:30Z,18:55:00,18:55:00,87212027,0,Strasbourg,48.584532,7.734793
3,OCESN2060F192057:2021-02-21T18:50:52Z,18:55:00,18:55:00,87212027,0,Strasbourg,48.584532,7.734793
4,OCESN2060F212734:2021-02-21T18:50:52Z,18:55:00,18:55:00,87212027,0,Strasbourg,48.584532,7.734793
...,...,...,...,...,...,...,...,...
519205,DUASN592807R01001-1_77482,06:45:00,06:45:00,8743081,0,LA FERTE GAUCHER CENTRE,48.782311,3.306949
519206,DUASN592808R01001-1_77483,12:45:00,12:45:00,8743081,0,LA FERTE GAUCHER CENTRE,48.782311,3.306949
519207,DUASN592811R01001-1_77484,16:45:00,16:45:00,8743081,0,LA FERTE GAUCHER CENTRE,48.782311,3.306949
519208,DUASN592814R01001-1_77485,18:45:00,18:45:00,8743081,0,LA FERTE GAUCHER CENTRE,48.782311,3.306949


In [138]:
'''To merge a selection of the stop_times_stops_France dataset with the trips_routes_France dataset.'''
trips_routes_stop_times_stops_France = pd.merge(trips_routes_France, stop_times_stops_France, on='trip_id')
trips_routes_stop_times_stops_France

'To merge a selection of the stop_times_stops_France dataset with the trips_routes_France dataset.'

,route_id,service_id,trip_id,trip_headsign,route_short_name,route_long_name,arrival_time,departure_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon
0,OCESN-1209972,TGV-1,OCESN2052F198219:2021-02-19T18:51:30Z,2052,807A,Paris - Strasbourg/Alsace Allemagne,17:12:00,17:12:00,87212027,0,Strasbourg,48.584532,7.734793
1,OCESN-1209972,TGV-1,OCESN2052F198219:2021-02-19T18:51:30Z,2052,807A,Paris - Strasbourg/Alsace Allemagne,19:05:00,19:05:00,87113001,1,Paris Est,48.876742,2.358424
2,OCESN-1209972,TGV-2,OCESN2052F224808:2021-02-20T18:50:55Z,2052,807A,Paris - Strasbourg/Alsace Allemagne,17:08:00,17:08:00,87212027,0,Strasbourg,48.584532,7.734793
3,OCESN-1209972,TGV-2,OCESN2052F224808:2021-02-20T18:50:55Z,2052,807A,Paris - Strasbourg/Alsace Allemagne,19:05:00,19:05:00,87113001,1,Paris Est,48.876742,2.358424
4,OCESN-1209972,TGV-3,OCESN2060F160675:2021-02-19T18:51:30Z,2060,807A,Paris - Strasbourg/Alsace Allemagne,18:55:00,18:55:00,87212027,0,Strasbourg,48.584532,7.734793
...,...,...,...,...,...,...,...,...,...,...,...,...,...
398248,DUA800803071,Transilien-9366,DUASN148694F02001-1_109439,VICK,C,Pontoise / Versailles R. Gauche / St-Quentin e...,23:35:00,23:36:00,8739304,22,PONT DE L ALMA,48.862052,2.301784
398249,DUA800803071,Transilien-9366,DUASN148694F02001-1_109439,VICK,C,Pontoise / Versailles R. Gauche / St-Quentin e...,23:33:00,23:34:00,8739303,21,INVALIDES,48.862900,2.313893
398250,DUA800803071,Transilien-9366,DUASN148694F02001-1_109439,VICK,C,Pontoise / Versailles R. Gauche / St-Quentin e...,23:31:00,23:31:00,8754730,20,MUSEE D ORSAY,48.860682,2.325609
398251,DUA800803071,Transilien-9366,DUASN148694F02001-1_109439,VICK,C,Pontoise / Versailles R. Gauche / St-Quentin e...,23:27:00,23:28:00,8754731,19,ST MICHEL ND RER C,48.853483,2.343860


In [153]:
'''To create a route_sequence dataset that gives for each trip_id that belongs to a route the sequence of stations served'''
route_sequence_France = trips_routes_stop_times_stops_France.groupby(['route_id','route_long_name','trip_headsign','trip_id','stop_sequence'], as_index=False)[['stop_name', 'stop_lat', 'stop_lon']].last()
route_sequence_France

'To create a route_sequence dataset that gives for each trip_id that belongs to a route the sequence of stations served'

,route_id,route_long_name,trip_headsign,trip_id,stop_sequence,stop_name,stop_lat,stop_lon
0,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,145832,DUASN145832F02001-1_108610,0,DOURDAN LA FORET,48.535479,1.995882
1,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,145832,DUASN145832F02001-1_108610,1,DOURDAN,48.533612,2.009701
2,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,145832,DUASN145832F02001-1_108610,2,SERMAISE,48.535514,2.068816
3,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,145832,DUASN145832F02001-1_108610,3,SAINT-CHERON,48.550965,2.125552
4,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,145832,DUASN145832F02001-1_108610,4,BREUILLET VILLAGE,48.564900,2.171463
...,...,...,...,...,...,...,...,...
398248,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,96789,OCESN96789F261181:2021-03-09T18:51:45Z,5,Meyrin,46.222350,6.076900
398249,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,96789,OCESN96789F261181:2021-03-09T18:51:45Z,6,Vernier,46.220710,6.093892
398250,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,96789,OCESN96789F261181:2021-03-09T18:51:45Z,7,Geneve,46.210220,6.142440
398251,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,461100,OCESN461100R236909:2021-03-09T18:51:45Z,0,Geneve,46.210220,6.142440


'''To calculate the hash and the hash_inverse values for the stop sequence of each trip_id'''

#To copy the trips_routes_France df
trips_hash_France = trips_routes_France.copy()

#To create a column called hash that contains NaN values
trips_hash_France['hash'] = np.nan

#To create a column called hash_inverse that contains NaN values
trips_hash_France['hash_inverse'] = np.nan

#For each trip_id in trips_routes_France, the stop_sequence that gets calculated is the subset of the stop_times dataset for that trip_id. 
#The tuple that results from the stop_id column of this subset dataset contains all the stop_ids that get served by this trip_id. 

#The tuple that results from the stop_id column of this subset dataset contains all the stop_ids that get served by this trip_id. 
#The hash value of the tuple of the stop_id column is calculated and is placed in the hash column of the trip_id in the trips_routes dataset
#The inverse_hash value of the tuple of the stop_id column is calculated as 

for trip_France in trips_routes_France['trip_id'].unique():
    stop_sequence_France = stop_times_France[stop_times_France['trip_id'] == trip_France].sort_values(by = 'stop_sequence')
    trips_hash_France.loc[trips_hash_France['trip_id'] == trip_France, 'hash'] = hash(tuple(stop_sequence_France['stop_id']))
    trips_hash_France.loc[trips_hash_France['trip_id'] == trip_France, 'hash_inverse'] = hash(tuple(list(stop_sequence_France['stop_id'])[::-1]))

In [156]:
trips_hash_France = pd.read_csv(datalink + "trips_hash_France.csv", sep=",")

In [157]:
trips_hash_France

,route_id,service_id,trip_id,trip_headsign,route_short_name,route_long_name,hash,hash_inverse
0,OCESN-1209972,TGV-1,OCESN2052F198219:2021-02-19T18:51:30Z,2052,807A,Paris - Strasbourg/Alsace Allemagne,2.945047e+18,-2.491326e+18
1,OCESN-1209972,TGV-2,OCESN2052F224808:2021-02-20T18:50:55Z,2052,807A,Paris - Strasbourg/Alsace Allemagne,2.945047e+18,-2.491326e+18
2,OCESN-1209972,TGV-3,OCESN2060F160675:2021-02-19T18:51:30Z,2060,807A,Paris - Strasbourg/Alsace Allemagne,2.945047e+18,-2.491326e+18
3,OCESN-1209972,TGV-4,OCESN2060F192057:2021-02-21T18:50:52Z,2060,807A,Paris - Strasbourg/Alsace Allemagne,2.945047e+18,-2.491326e+18
4,OCESN-1209972,TGV-5,OCESN2060F212734:2021-02-21T18:50:52Z,2060,807A,Paris - Strasbourg/Alsace Allemagne,2.945047e+18,-2.491326e+18
...,...,...,...,...,...,...,...,...
34951,DUA800803071,Transilien-8668,DUASN148676F01001-1_109435,VICK,C,Pontoise / Versailles R. Gauche / St-Quentin e...,8.393012e+18,-4.398464e+17
34952,DUA800803071,Transilien-9366,DUASN148678F02001-1_109436,VICK,C,Pontoise / Versailles R. Gauche / St-Quentin e...,8.393012e+18,-4.398464e+17
34953,DUA800803071,Transilien-9744,DUASN148678F02002-1_109437,VICK,C,Pontoise / Versailles R. Gauche / St-Quentin e...,8.393012e+18,-4.398464e+17
34954,DUA800803071,Transilien-9366,DUASN148686F02001-1_109438,VICK,C,Pontoise / Versailles R. Gauche / St-Quentin e...,8.393012e+18,-4.398464e+17


In [142]:
''' To groupby the trip_id and to order the stop_sequence in an ascending order (the stop_sequences of some
routes are initially in descending order while other stop_sequences are in ascending order) '''

trips_stop_sequence_ascending_France = stop_times_stops_France.groupby(['trip_id'], as_index=False).apply(lambda x: x.sort_values('stop_sequence'))
trips_stop_sequence_ascending_France

' To groupby the trip_id and to order the stop_sequence in an ascending order (the stop_sequences of some\nroutes are initially in descending order while other stop_sequences are in ascending order) '

trip_id arrival_time departure_time  \
0     264619           DUASN003314F01001-1_153140     17:33:00       17:33:00   
      250253           DUASN003314F01001-1_153140     18:26:00       18:26:00   
1     264620           DUASN003340F01001-1_153141     18:40:00       18:40:00   
      250254           DUASN003340F01001-1_153141     19:29:00       19:29:00   
2     264621           DUASN003374F01001-1_153142     12:14:00       12:14:00   
...                                           ...          ...            ...   
57746 17545   OCESN998F96797:2021-02-20T18:50:55Z     15:04:00       15:09:00   
      21764   OCESN998F96797:2021-02-20T18:50:55Z     16:02:00       16:06:00   
      23232   OCESN998F96797:2021-02-20T18:50:55Z     16:42:00       16:46:00   
      23944   OCESN998F96797:2021-02-20T18:50:55Z     17:24:00       17:28:00   
      22430   OCESN998F96797:2021-02-20T18:50:55Z     18:45:00       18:45:00   

               stop_id  stop_sequence           stop_name   stop_lat  stop_lon  
0     264619   8738150              0     MANTES LA JOLIE  48.989687  1.703294  
      250253   8738400              1  PARIS SAINT-LAZARE  48.875358  2.324733  
1     264620   8738150              0     MANTES LA JOLIE  48.989687  1.703294  
      250254   8738400              1  PARIS SAINT-LAZARE  48.875358  2.324733  
2     264621   8738150              0     MANTES LA JOLIE  48.989687  1.703294  
...                ...            ...                 ...        ...       ...  
57746 17545   87393702              6           Massy TGV  48.725758  2.261254  
      21764   87396002              7             Le Mans  47.995615  0.192614  
      23232   87484006              8   Angers Saint-Laud  47.464647 -0.556820  
      23944   87481002              9              Nantes  47.217491 -1.542077  
      22430   87471003             10              Rennes  48.103517 -1.672744  

[519210 rows x 8 columns]

In [143]:
''' To put the stop_names of a stop sequence of a trip_id in a list '''
trips_stop_sequence_France = trips_stop_sequence_ascending_France.groupby('trip_id')['stop_name'].apply(lambda group_series: group_series.tolist()).reset_index()
trips_stop_sequence_France.rename(columns={'stop_name':'stop_sequence'}, inplace=True)
trips_stop_sequence_France

' To put the stop_names of a stop sequence of a trip_id in a list '

,trip_id,stop_sequence
0,DUASN003314F01001-1_153140,"[MANTES LA JOLIE, PARIS SAINT-LAZARE]"
1,DUASN003340F01001-1_153141,"[MANTES LA JOLIE, PARIS SAINT-LAZARE]"
2,DUASN003374F01001-1_153142,"[MANTES LA JOLIE, PARIS SAINT-LAZARE]"
3,DUASN003376F01001-1_153143,"[MANTES LA JOLIE, PARIS SAINT-LAZARE]"
4,DUASN003378F01001-1_153144,"[MANTES LA JOLIE, PARIS SAINT-LAZARE]"
...,...,...
57742,OCESN9898F202613:2021-02-21T18:50:52Z,"[Montpellier Saint-Roch, Nîmes, Valence TGV Rh..."
57743,OCESN9898F223732:2021-02-21T18:50:52Z,"[Montpellier Saint-Roch, Nîmes, Valence TGV Rh..."
57744,OCESN990F96807:2021-02-21T18:50:52Z,"[Bourg-Saint-Maurice, Landry, Aime - La Plagne..."
57745,OCESN996F96801:2021-02-21T18:50:52Z,"[Bourg-Saint-Maurice, Landry, Aime - La Plagne..."


In [144]:
''' To add the list of stop_sequence of stations to the trips_hash_France df by joining on trip_id'''
# To add the stop_sequence of stations to the trips_hash_France df by joining on trip_id
trips_hash_stop_sequence_France = pd.merge(trips_hash_France, trips_stop_sequence_France, on='trip_id', how='left')

# To put the columns in a more logical order
trips_hash_stop_sequence_France = trips_hash_stop_sequence_France[['route_id', 'route_long_name','service_id','trip_headsign','trip_id','hash', 'hash_inverse','stop_sequence']]
trips_hash_stop_sequence_France

' To add the list of stop_sequence of stations to the trips_hash_France df by joining on trip_id'

,route_id,route_long_name,service_id,trip_headsign,trip_id,hash,hash_inverse,stop_sequence
0,OCESN-1209972,Paris - Strasbourg/Alsace Allemagne,TGV-1,2052,OCESN2052F198219:2021-02-19T18:51:30Z,2.945047e+18,-2.491326e+18,"[Strasbourg, Paris Est]"
1,OCESN-1209972,Paris - Strasbourg/Alsace Allemagne,TGV-2,2052,OCESN2052F224808:2021-02-20T18:50:55Z,2.945047e+18,-2.491326e+18,"[Strasbourg, Paris Est]"
2,OCESN-1209972,Paris - Strasbourg/Alsace Allemagne,TGV-3,2060,OCESN2060F160675:2021-02-19T18:51:30Z,2.945047e+18,-2.491326e+18,"[Strasbourg, Paris Est]"
3,OCESN-1209972,Paris - Strasbourg/Alsace Allemagne,TGV-4,2060,OCESN2060F192057:2021-02-21T18:50:52Z,2.945047e+18,-2.491326e+18,"[Strasbourg, Paris Est]"
4,OCESN-1209972,Paris - Strasbourg/Alsace Allemagne,TGV-5,2060,OCESN2060F212734:2021-02-21T18:50:52Z,2.945047e+18,-2.491326e+18,"[Strasbourg, Paris Est]"
...,...,...,...,...,...,...,...,...
34951,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,Transilien-8668,VICK,DUASN148676F01001-1_109435,8.393012e+18,-4.398464e+17,"[VERSAILLES CHANTIERS, PETIT JOUY LES LOGES, J..."
34952,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,Transilien-9366,VICK,DUASN148678F02001-1_109436,8.393012e+18,-4.398464e+17,"[VERSAILLES CHANTIERS, PETIT JOUY LES LOGES, J..."
34953,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,Transilien-9744,VICK,DUASN148678F02002-1_109437,8.393012e+18,-4.398464e+17,"[VERSAILLES CHANTIERS, PETIT JOUY LES LOGES, J..."
34954,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,Transilien-9366,VICK,DUASN148686F02001-1_109438,8.393012e+18,-4.398464e+17,"[VERSAILLES CHANTIERS, PETIT JOUY LES LOGES, J..."


In [145]:
''' To count the number of dates for each service_id '''
service_id_df_France = calendar_dates_France.groupby(['service_id'])[['service_id']].count().rename(columns={'service_id':'count_service_id'}).reset_index()
service_id_df_France

' To count the number of dates for each service_id '

,service_id,count_service_id
0,Intercites-0,92
1,Intercites-1,1
2,Intercites-10,42
3,Intercites-100,8
4,Intercites-101,1
...,...,...
5363,Transilien-9965,16
5364,Transilien-9966,1
5365,Transilien-9970,26
5366,Transilien-9971,4


In [146]:
''' To regroup the days per service_id in a set '''
service_id_dates_France = calendar_dates_France.groupby('service_id')['date'].apply(lambda group_series: set(group_series.tolist())).reset_index()
service_id_dates_France.rename(columns={'date':'dates'}, inplace=True)
service_id_dates_France = service_id_dates_France.merge(service_id_df_France, on='service_id', how='left')
service_id_dates_France

' To regroup the days per service_id in a set '

,service_id,dates,count_service_id
0,Intercites-0,"{20210309, 20210310, 20210311, 20210312, 20210...",92
1,Intercites-1,{20210523},1
2,Intercites-10,"{20210601, 20210602, 20210603, 20210604, 20210...",42
3,Intercites-100,"{20210530, 20210502, 20210314, 20210509, 20210...",8
4,Intercites-101,{20210424},1
...,...,...,...
5363,Transilien-9965,"{20210529, 20210530, 20210410, 20210507, 20210...",16
5364,Transilien-9966,{20210410},1
5365,Transilien-9970,"{20210601, 20210602, 20210603, 20210604, 20210...",26
5366,Transilien-9971,"{20210529, 20210530, 20210508, 20210509}",4


In [147]:
''' To put the different trip_ids in a list after joining on (route_id, route_long_name, hash and service_id) '''
route_hash_freq_France = trips_hash_stop_sequence_France.groupby(['route_id','route_long_name','hash', 'hash_inverse', 'service_id'])['trip_id'].apply(lambda group_series: group_series.tolist()).reset_index()
route_hash_freq_France

' To put the different trip_ids in a list after joining on (route_id, route_long_name, hash and service_id) '

,route_id,route_long_name,hash,hash_inverse,service_id,trip_id
0,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-9.166547e+18,-6.369926e+18,Transilien-9744,"[DUASN147339F01001-1_108213, DUASN147341F01001..."
1,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-8.924379e+18,7.105125e+18,Transilien-9744,[DUASN147184F01001-1_108144]
2,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-8.910339e+18,-7.853936e+18,Transilien-9363,"[DUASN141548F02001-1_108145, DUASN141552F01001..."
3,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-8.555006e+18,7.146731e+18,Transilien-8668,"[DUASN143845F01001-1_107763, DUASN145809F01001..."
4,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-8.555006e+18,7.146731e+18,Transilien-8959,[DUASN145881F02001-1_107770]
...,...,...,...,...,...,...
16024,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,4.408651e+18,-3.193764e+18,TER-2715,[OCESN96772F256123:2021-03-09T18:51:45Z]
16025,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,4.408651e+18,-3.193764e+18,TER-2784,[OCESN96720F276524:2021-03-09T18:51:45Z]
16026,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,4.408651e+18,-3.193764e+18,TER-2785,"[OCESN96770F263119:2021-03-09T18:51:45Z, OCESN..."
16027,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,4.408651e+18,-3.193764e+18,TER-2795,"[OCESN96714F274792:2021-03-09T18:51:45Z, OCESN..."


In [148]:
''' To add the sequence of stops to the route_hash_freq dataset '''
route_hash_freq_France = pd.merge(route_hash_freq_France, trips_hash_stop_sequence_France[['route_id','hash', 'hash_inverse', 'service_id','stop_sequence']], on=['route_id', 'hash', 'hash_inverse', 'service_id'], how='left')
route_hash_freq_France = route_hash_freq_France.drop_duplicates( subset = ['route_id', 'hash', 'service_id'], keep = 'first')

route_hash_freq_France

' To add the sequence of stops to the route_hash_freq dataset '

,route_id,route_long_name,hash,hash_inverse,service_id,trip_id,stop_sequence
0,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-9.166547e+18,-6.369926e+18,Transilien-9744,"[DUASN147339F01001-1_108213, DUASN147341F01001...","[PORTE DE CLICHY, PEREIRE LEVALLOIS, NEUILLY P..."
2,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-8.924379e+18,7.105125e+18,Transilien-9744,[DUASN147184F01001-1_108144],"[PONTOISE, SAINT-OUEN L'AUMONE, SAINT-OUEN L'A..."
3,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-8.910339e+18,-7.853936e+18,Transilien-9363,"[DUASN141548F02001-1_108145, DUASN141552F01001...","[PONTOISE, SAINT-OUEN L'AUMONE, SAINT-OUEN L'A..."
38,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-8.555006e+18,7.146731e+18,Transilien-8668,"[DUASN143845F01001-1_107763, DUASN145809F01001...","[INVALIDES, MUSEE D ORSAY, ST MICHEL ND RER C,..."
42,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-8.555006e+18,7.146731e+18,Transilien-8959,[DUASN145881F02001-1_107770],"[INVALIDES, MUSEE D ORSAY, ST MICHEL ND RER C,..."
...,...,...,...,...,...,...,...
34941,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,4.408651e+18,-3.193764e+18,TER-2715,[OCESN96772F256123:2021-03-09T18:51:45Z],"[Geneve, Vernier, Meyrin, Zimeysa, Satigny, Ru..."
34942,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,4.408651e+18,-3.193764e+18,TER-2784,[OCESN96720F276524:2021-03-09T18:51:45Z],"[Geneve, Vernier, Meyrin, Zimeysa, Satigny, Ru..."
34943,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,4.408651e+18,-3.193764e+18,TER-2785,"[OCESN96770F263119:2021-03-09T18:51:45Z, OCESN...","[Geneve, Vernier, Meyrin, Zimeysa, Satigny, Ru..."
34948,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,4.408651e+18,-3.193764e+18,TER-2795,"[OCESN96714F274792:2021-03-09T18:51:45Z, OCESN...","[Geneve, Vernier, Meyrin, Zimeysa, Satigny, Ru..."


In [149]:
''' To calculate the number of trip_ids in the list of trip_ids and to add it as a new column '''
number_trip_ids_France = []
for list_trip_ids_France in route_hash_freq_France['trip_id']:
    count_France = len(list_trip_ids_France)
    number_trip_ids_France.append(count_France)
route_hash_freq_France['number_trip_ids'] = number_trip_ids_France

route_hash_freq_France

' To calculate the number of trip_ids in the list of trip_ids and to add it as a new column '

,route_id,route_long_name,hash,hash_inverse,service_id,trip_id,stop_sequence,number_trip_ids
0,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-9.166547e+18,-6.369926e+18,Transilien-9744,"[DUASN147339F01001-1_108213, DUASN147341F01001...","[PORTE DE CLICHY, PEREIRE LEVALLOIS, NEUILLY P...",2
2,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-8.924379e+18,7.105125e+18,Transilien-9744,[DUASN147184F01001-1_108144],"[PONTOISE, SAINT-OUEN L'AUMONE, SAINT-OUEN L'A...",1
3,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-8.910339e+18,-7.853936e+18,Transilien-9363,"[DUASN141548F02001-1_108145, DUASN141552F01001...","[PONTOISE, SAINT-OUEN L'AUMONE, SAINT-OUEN L'A...",35
38,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-8.555006e+18,7.146731e+18,Transilien-8668,"[DUASN143845F01001-1_107763, DUASN145809F01001...","[INVALIDES, MUSEE D ORSAY, ST MICHEL ND RER C,...",4
42,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-8.555006e+18,7.146731e+18,Transilien-8959,[DUASN145881F02001-1_107770],"[INVALIDES, MUSEE D ORSAY, ST MICHEL ND RER C,...",1
...,...,...,...,...,...,...,...,...
34941,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,4.408651e+18,-3.193764e+18,TER-2715,[OCESN96772F256123:2021-03-09T18:51:45Z],"[Geneve, Vernier, Meyrin, Zimeysa, Satigny, Ru...",1
34942,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,4.408651e+18,-3.193764e+18,TER-2784,[OCESN96720F276524:2021-03-09T18:51:45Z],"[Geneve, Vernier, Meyrin, Zimeysa, Satigny, Ru...",1
34943,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,4.408651e+18,-3.193764e+18,TER-2785,"[OCESN96770F263119:2021-03-09T18:51:45Z, OCESN...","[Geneve, Vernier, Meyrin, Zimeysa, Satigny, Ru...",5
34948,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,4.408651e+18,-3.193764e+18,TER-2795,"[OCESN96714F274792:2021-03-09T18:51:45Z, OCESN...","[Geneve, Vernier, Meyrin, Zimeysa, Satigny, Ru...",7


In [150]:
''' To merge the route_hash_freq_France df with the service_id_dates to get the sets of corresponding dates '''
route_hash_service_freq_France = pd.merge(route_hash_freq_France, service_id_dates_France, on='service_id', how='left')
route_hash_service_freq_France

' To merge the route_hash_freq_France df with the service_id_dates to get the sets of corresponding dates '

,route_id,route_long_name,hash,hash_inverse,service_id,trip_id,stop_sequence,number_trip_ids,dates,count_service_id
0,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-9.166547e+18,-6.369926e+18,Transilien-9744,"[DUASN147339F01001-1_108213, DUASN147341F01001...","[PORTE DE CLICHY, PEREIRE LEVALLOIS, NEUILLY P...",2,"{20210513, 20210514, 20210524, 20210405}",4.0
1,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-8.924379e+18,7.105125e+18,Transilien-9744,[DUASN147184F01001-1_108144],"[PONTOISE, SAINT-OUEN L'AUMONE, SAINT-OUEN L'A...",1,"{20210513, 20210514, 20210524, 20210405}",4.0
2,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-8.910339e+18,-7.853936e+18,Transilien-9363,"[DUASN141548F02001-1_108145, DUASN141552F01001...","[PONTOISE, SAINT-OUEN L'AUMONE, SAINT-OUEN L'A...",35,"{20210513, 20210514, 20210524, 20210405}",4.0
3,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-8.555006e+18,7.146731e+18,Transilien-8668,"[DUASN143845F01001-1_107763, DUASN145809F01001...","[INVALIDES, MUSEE D ORSAY, ST MICHEL ND RER C,...",4,NaN,NaN
4,DUA800803071,Pontoise / Versailles R. Gauche / St-Quentin e...,-8.555006e+18,7.146731e+18,Transilien-8959,[DUASN145881F02001-1_107770],"[INVALIDES, MUSEE D ORSAY, ST MICHEL ND RER C,...",1,"{20210528, 20210604, 20210319, 20210611, 20210...",5.0
...,...,...,...,...,...,...,...,...,...,...
16024,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,4.408651e+18,-3.193764e+18,TER-2715,[OCESN96772F256123:2021-03-09T18:51:45Z],"[Geneve, Vernier, Meyrin, Zimeysa, Satigny, Ru...",1,"{20210417, 20210410}",2.0
16025,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,4.408651e+18,-3.193764e+18,TER-2784,[OCESN96720F276524:2021-03-09T18:51:45Z],"[Geneve, Vernier, Meyrin, Zimeysa, Satigny, Ru...",1,"{20210309, 20210310, 20210311, 20210312, 20210...",68.0
16026,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,4.408651e+18,-3.193764e+18,TER-2785,"[OCESN96770F263119:2021-03-09T18:51:45Z, OCESN...","[Geneve, Vernier, Meyrin, Zimeysa, Satigny, Ru...",5,"{20210412, 20210413, 20210414, 20210415, 20210...",6.0
16027,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,BELLEGARDE - GENEVE,4.408651e+18,-3.193764e+18,TER-2795,"[OCESN96714F274792:2021-03-09T18:51:45Z, OCESN...","[Geneve, Vernier, Meyrin, Zimeysa, Satigny, Ru...",7,"{20210309, 20210310, 20210311, 20210312, 20210...",57.0


In [151]:
'''Get the distinct stop sequences for all routes to create the possible track combinations later on'''
distinct_stop_sequences_France = route_hash_service_freq_France.drop_duplicates(subset = ["route_id", 'hash'])[['route_id','hash','route_long_name','stop_sequence']]
distinct_stop_sequences_France

'Get the distinct stop sequences for all routes to create the possible track combinations later on'

,route_id,hash,route_long_name,stop_sequence
0,DUA800803071,-9.166547e+18,Pontoise / Versailles R. Gauche / St-Quentin e...,"[PORTE DE CLICHY, PEREIRE LEVALLOIS, NEUILLY P..."
1,DUA800803071,-8.924379e+18,Pontoise / Versailles R. Gauche / St-Quentin e...,"[PONTOISE, SAINT-OUEN L'AUMONE, SAINT-OUEN L'A..."
2,DUA800803071,-8.910339e+18,Pontoise / Versailles R. Gauche / St-Quentin e...,"[PONTOISE, SAINT-OUEN L'AUMONE, SAINT-OUEN L'A..."
3,DUA800803071,-8.555006e+18,Pontoise / Versailles R. Gauche / St-Quentin e...,"[INVALIDES, MUSEE D ORSAY, ST MICHEL ND RER C,..."
6,DUA800803071,-8.434706e+18,Pontoise / Versailles R. Gauche / St-Quentin e...,"[JUVISY, BIBLIOTHEQUE F. MITTERRAND, GARE D'AU..."
...,...,...,...,...
15999,OCESN-FR:Line::3DCDF960-BA93-4443-899F-283A2A1...,-3.294683e+18,ANNEMASSE - COPPET,"[Geneve, Lancy-Pont-Rouge, Lancy-Bachet, Genèv..."
16009,OCESN-FR:Line::3DCDF960-BA93-4443-899F-283A2A1...,3.881016e+18,ANNEMASSE - COPPET,"[Annemasse, Chêne-Bourg, Genève-Eaux-Vives, Ge..."
16018,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,-3.193764e+18,BELLEGARDE - GENEVE,"[Bellegarde, Pougny - Chancy, Russin, Satigny,..."
16024,OCESN-FR:Line::A06C2948-C00E-4D5B-AD30-86FFEC3...,4.408651e+18,BELLEGARDE - GENEVE,"[Geneve, Vernier, Meyrin, Zimeysa, Satigny, Ru..."


In [199]:
routes_sequences_France = distinct_stop_sequences_France[distinct_stop_sequences_France['route_id'] == 'DUA800803071']
routes_sequences_France.to_csv(r'/Users/pol/Desktop/CSV_export/export.csv', index = False, header=True, encoding='utf-8-sig')

In [200]:
routes_sequences_France

,route_id,hash,route_long_name,stop_sequence
0,DUA800803071,-9.166547e+18,Pontoise / Versailles R. Gauche / St-Quentin e...,"[PORTE DE CLICHY, PEREIRE LEVALLOIS, NEUILLY P..."
1,DUA800803071,-8.924379e+18,Pontoise / Versailles R. Gauche / St-Quentin e...,"[PONTOISE, SAINT-OUEN L'AUMONE, SAINT-OUEN L'A..."
2,DUA800803071,-8.910339e+18,Pontoise / Versailles R. Gauche / St-Quentin e...,"[PONTOISE, SAINT-OUEN L'AUMONE, SAINT-OUEN L'A..."
3,DUA800803071,-8.555006e+18,Pontoise / Versailles R. Gauche / St-Quentin e...,"[INVALIDES, MUSEE D ORSAY, ST MICHEL ND RER C,..."
6,DUA800803071,-8.434706e+18,Pontoise / Versailles R. Gauche / St-Quentin e...,"[JUVISY, BIBLIOTHEQUE F. MITTERRAND, GARE D'AU..."
...,...,...,...,...
251,DUA800803071,8.837220e+18,Pontoise / Versailles R. Gauche / St-Quentin e...,"[MUSEE D ORSAY, ST MICHEL ND RER C, GARE D'AUS..."
258,DUA800803071,8.940848e+18,Pontoise / Versailles R. Gauche / St-Quentin e...,"[GARE D'AUSTERLITZ, BIBLIOTHEQUE F. MITTERRAND..."
263,DUA800803071,9.016457e+18,Pontoise / Versailles R. Gauche / St-Quentin e...,"[VERSAILLES CHANTIERS, PETIT JOUY LES LOGES, J..."
264,DUA800803071,9.081443e+18,Pontoise / Versailles R. Gauche / St-Quentin e...,"[PONTOISE, SAINT-OUEN L'AUMONE, SAINT-OUEN L'A..."


In [223]:
'''Finds the routes that can be either extended from behind or from after'''
index_of_extendable = {}
index_of_begin_sequences = {}
for route_id in routes_sequences_France['route_id'].unique():
    begin_end_stations = set()
    for index_trip, trip in routes_sequences_France[routes_sequences_France['route_id'] == route_id].iterrows():
        possible_extentions1 = routes_sequences_France[routes_sequences_France['stop_sequence'].apply(lambda x: any(item for item in [trip['stop_sequence'][-1]] if (item == x[0]) and not(set(x[1:]) & set(trip['stop_sequence']))))].copy()
        possible_extentions2 = routes_sequences_France[routes_sequences_France['stop_sequence'].apply(lambda x: any(item for item in [trip['stop_sequence'][0]] if (item == x[-1]) and not(set(x[:-1]) & set(trip['stop_sequence']))))].copy()        
        possible_extentions = possible_extentions1.append(possible_extentions2, ignore_index = True)
        if not possible_extentions.empty:
            if route_id not in index_of_extendable:
                index_of_extendable[route_id] = []
            index_of_extendable[route_id].append(index_trip)
            if not possible_extentions1.empty:
                if route_id not in index_of_begin_sequences:
                    index_of_begin_sequences[route_id] = []
                index_of_begin_sequences[route_id].append(index_trip)
print(index_of_extendable)
print(index_of_begin_sequences)


'Finds the routes that can be either extended from behind or from after'

{'DUA800803071': [1, 2, 3, 11, 12, 17, 18, 26, 27, 29, 37, 54, 59, 64, 68, 73, 75, 80, 88, 90, 96, 100, 147, 148, 150, 183, 185, 194, 198, 201, 202, 208, 222, 228, 229, 258, 266]}
{'DUA800803071': [1, 2, 11, 26, 27, 29, 54, 73, 75, 96, 147, 148, 185, 198, 201, 208, 228, 229, 266]}


In [224]:
'''creates all the sequences of routes possible to reconstruct the real route'''
import copy
#create an empty df for the process of route creation
route_creation_France  = pd.DataFrame()
for route_id in routes_sequences_France['route_id'].unique():
    #create a copy of the df with only the route considered in the loop iteration
    routes_with_route_id = routes_sequences_France[routes_sequences_France['route_id'] == route_id].copy()
    #create a df where only the routes that have an end stop as their first element of the sequence
    route_creation_France_route_id = routes_with_route_id.loc[index_of_begin_sequences[route_id]][['route_id', 'hash', 'stop_sequence']]
    #make the hash column as a column of lists
    route_creation_France_route_id['hash'] = route_creation_France_route_id['hash'].apply(lambda x: [x])
    route_creation_France_route_id = route_creation_France_route_id.reset_index(drop=True)
    #to stop the while loop when all the routes are complete in the df for the route_id of the loop iteration
    complete_routes = 0
    while complete_routes < len(route_creation_France_route_id.index):
        #use a deepcopy to not impact the iterrows of the main loop
        route_creation_France_deep_copy = copy.deepcopy(route_creation_France_route_id)
        for index_original, route_part in route_creation_France_deep_copy.iterrows():
            #create a dataframe of the possible extentions for each route_part
            #here checks if the extention is the next part of the route and also that no other station are repeated in the sequence if this extention is added(otherwise it might cause an infinite loop)
            possible_extentions = routes_with_route_id[routes_with_route_id['stop_sequence'].apply(lambda x: any(item for item in [route_part['stop_sequence'][-1]] if (item == x[0]) and not(set(x[1:]) & set(route_part['stop_sequence']))))].copy()
            #checks whether it is possible to extend the route_part
            if not (possible_extentions.empty):
                #if so, extend it with every single possibilities
                for index_extention, possible_extention in possible_extentions.iterrows():
                    #must create a deepcopy, otherwise the orignal hash list will change as well (mutable)
                    updated_hash = copy.deepcopy(route_part['hash'])
                    updated_hash.append(possible_extention['hash'])
                    updated_route_sequence = route_part['stop_sequence'] + possible_extention['stop_sequence'][1:]
                    route_creation_France_route_id.loc[max(route_creation_France_route_id.index)+1] = [route_id, updated_hash, updated_route_sequence] 
            #the route can't be extended anymore
            else:
                complete_routes += 1
            #then delete the route with the index (see loop here above)
            route_creation_France_route_id = route_creation_France_route_id.drop(index = index_original)
    #adds all the possible routes created with the trips of the route_id of the main loop
    route_creation_France = route_creation_France.append(route_creation_France_route_id, ignore_index = True)
route_creation_France

'creates all the sequences of routes possible to reconstruct the real route'

,route_id,hash,stop_sequence
0,DUA800803071,"[-8.924379105402193e+18, -6.561063152648576e+1...","[PONTOISE, SAINT-OUEN L'AUMONE, SAINT-OUEN L'A..."
1,DUA800803071,"[-8.910339425696816e+18, 4.914117829426481e+18...","[PONTOISE, SAINT-OUEN L'AUMONE, SAINT-OUEN L'A..."
2,DUA800803071,"[-6.915284451190455e+18, 4.914117829426481e+18...","[ERMONT EAUBONNE, SAINT-GRATIEN, EPINAY SUR SE..."
3,DUA800803071,"[1.490056396856132e+18, -6.561063152648576e+18...","[BRETIGNY, GARE D'AUSTERLITZ, BIBLIOTHEQUE F. ..."
4,DUA800803071,"[5.724453219679127e+18, 4.914117829426481e+18,...","[MONTIGNY BEAUCHAMP, FRANCONVILLE LE PLESSIS B..."
5,DUA800803071,"[6.456958889587588e+18, 4.914117829426481e+18,...","[VERSAILLES RIVE GAUCHE Château de Versailles,..."
6,DUA800803071,"[9.119263014470295e+18, -6.561063152648576e+18...","[MONTIGNY BEAUCHAMP, FRANCONVILLE LE PLESSIS B..."


In [225]:
route_creation_France.to_csv(r'/Users/pol/Desktop/CSV_export/export.csv', index = False, header=True, encoding='utf-8-sig')

# **Research question 2: Do groups of similar routes in multiple railway systems have a comparable travel time?**

# 2.1 Do groups of similar routes in multiple railway systems have a comparable planned travel time?

## **Preparation for the clustering of the Belgian railway routes based on their metric values**

* Construction of the df route_characteristics

This df contains for each route the aggregated route values

In [ ]:
''' To make a list of all unique route_ids in the df_for_edges dataset'''
list(df_for_edges['route_id'].unique())

In [ ]:
#put all the route_ids in df route_characteristics

''' To create a list of all unique route_ids in the df_for_edges dataset'''
routes_list = list(df_for_edges['route_id'].unique())
routes_list.sort()

''' To create a dictionary where each route of the routes_list represents a key'''
route_characteristics_dic = {'route_id': routes_list}

''' To create a dataframe that contains the values in the route_characteristics_dic as the values in the first column'''
route_characteristics = pd.DataFrame(route_characteristics_dic, columns = ['route_id'])
route_characteristics['route_id'].astype(np.int64)

In [ ]:
#Put the nodes per route in df route_characteristics

''' To construct a new column of the route_characteristics dataset that has for every column value an empty set'''
route_characteristics['nodes'] = [set() for _ in range(len(route_characteristics))]

''' To put the two nodes of an edge in the nodes column of the route_characteristics dataset if the route_id of the corresponding row 
is present in the set of the list values of the corresponding edge that contains all route_ids that contain the edge'''
for node1, node2 in G.edges():
    route_characteristics[['route_id','nodes']].apply(lambda x: x['nodes'].update((node1, node2)) if x['route_id'] in set(G.get_edge_data(node1, node2)['route_id']) else x, axis = 1)
print(route_characteristics)

In [ ]:
for nodes in route_characteristics[route_characteristics['route_id'] == 6]['nodes']:
  print(nodes)

In [ ]:
# The inputs of the get_metric_list function are a dictionary that contains for all node keys the corresponding metric values and a specific route_id of a column of 
# the route_characteristics dataset
# A new empty list metric_list is created for these inputs
# For the nodes that are in the route_characteristics dataset corresponding to this route_id, the corresponding metric values from the dictionary get appended to metric_list

from statistics import mean
def get_metric_list(metric_dic, route_id):
    """this is a sub function that is used in calculate_metric_route_per_node"""
    metric_list = []
    for nodes in route_characteristics[route_characteristics['route_id'] == route_id]['nodes']:
        for node in nodes:
            metric_list.append(metric_dic[node])
    return metric_list

# For the metric, a new column is created in route_characteristics, where every column value consists of an empty list
# The values in the column of the df are determined by giving the metric_dic and the route_id of the row where that value is placed to the get_metric_list function
# The metric_list ultimately contains the metric values for a specific metric for all the stations corresponding to that route

def calculate_metric_route_per_node(metric_dic, metric_name):
    """puts the metrics of each node of a route in a list in the route_characteristics df"""
    route_characteristics[metric_name] = [list for _ in range(len(route_characteristics))]
    route_characteristics[metric_name] = route_characteristics.apply(lambda x: get_metric_list(metric_dic, x['route_id']) , axis = 1)
    
# The function starts from the metric_name, the original_metric_name and the aggregation_method inputs
# A new column named with the metric_name is created: initially it contains all NaN values
# Thereafter the aggregation_method is taken into account. The avg aggregation method returns for all column values the mean of the values in the corresponding row
# of the original_metric_name list. The max aggregation method returns for all column values the max of the values in the corresponding row of the original_metric_name list.

def calculate_metric_aggregation_route(metric_name, original_metric_name, aggregation_method):
    '''calculate the metric aggregation of the routes, only avg and max are supported as string values for aggregation method
    metric_name is the aggregation column name and original_metric_name is the node metric column name'''
    route_characteristics[metric_name] = np.nan
    if aggregation_method == 'avg':
        route_characteristics[metric_name] = route_characteristics.apply(lambda x: mean(x[original_metric_name]) , axis = 1)
    elif aggregation_method == 'max':
        route_characteristics[metric_name] = route_characteristics.apply(lambda x: max(x[original_metric_name]) , axis = 1)

In [ ]:
calculate_metric_route_per_node(dict(G.degree()), 'node_degree')
calculate_metric_aggregation_route('avg_degree', 'node_degree', 'avg')
calculate_metric_aggregation_route('max_degree', 'node_degree', 'max')
calculate_metric_route_per_node(dict(nx.clustering(G)), 'node_clustering')
calculate_metric_aggregation_route('avg_clustering', 'node_clustering', 'avg')
calculate_metric_aggregation_route('max_clustering', 'node_clustering', 'max')
calculate_metric_route_per_node(dict(nx.closeness_centrality(G)), 'node_closeness_centrality')
calculate_metric_aggregation_route('avg_closeness_centrality', 'node_closeness_centrality', 'avg')
calculate_metric_aggregation_route('max_closeness_centrality', 'node_closeness_centrality', 'max')
calculate_metric_route_per_node(dict(nx.betweenness_centrality(G)), 'node_betweenness_centrality')
calculate_metric_aggregation_route('avg_betweenness_centrality', 'node_betweenness_centrality', 'avg')
calculate_metric_aggregation_route('max_betweenness_centrality', 'node_betweenness_centrality', 'max')

In [ ]:
route_characteristics.head(2)

In [ ]:
print(route_characteristics['nodes'][0])

In [ ]:
''' To add the number of stations present in the stop sequence of a route_id'''
number_stations = []
for number_stops in route_characteristics['nodes']:
  number_stations.append(len(number_stops))

route_characteristics['len_stop_sequence'] = number_stations
route_characteristics.head(2)

In [ ]:
stops_cleaned[stops_cleaned['stop_name'] == 'Namur']

In [ ]:
stops_cleaned[stops_cleaned['stop_name'] == 'Bruxelles-Central']

In [ ]:
''' Example: to calculate the distance between Namur and Bruxelles-Central'''
import math

# The radius of the earth
R = 6373.0

# To calculate the coordinates
lat_Namur = math.radians(50.46879)
lon_Namur = math.radians(4.86222)
lat_Brussels = math.radians(50.84565)
lon_Brussels = math.radians(4.356804)

# To calculate the change in coordinates
dlon = lon_Brussels - lon_Namur
dlat = lat_Brussels - lat_Namur

# To use the Haversine formula to get the distance in kilometers between Brussels and Namur
a = math.sin(dlat / 2)**2 + math.cos(lat_Namur) * math.cos(lat_Brussels) * math.sin(dlon / 2)**2
c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

distance = R * c

print(distance)

In [ ]:
''' To add the absolute distances between the starting and the ending station of all routes'''

# Remember that geographical locations that are located further away to the right of the Prime Meridian Greenwich instead of another location that is closer
# (but still to the right) of this Meridian Greenwich, have a greater longitute value.
# Remember that geographical locations that are higher above the Equator than other geographical locations (that are also above the Equator), have a 
# higher latitude value

absolute_distance = []
for route in route_characteristics['route_id']:
  subdf_stop_sequence = df_for_edges[df_for_edges['route_id'] == route]
  number_of_rows = len(subdf_stop_sequence.index)

  # To get the starting station and the ending_station of the stop sequence of a route
  starting_station = subdf_stop_sequence['stop_name_1'].iloc[0]
  ending_station = subdf_stop_sequence['stop_name_2'].iloc[number_of_rows - 1]

  # The radius of the earth
  R = 6373.0

  # To convert the coordinates in radians 
  lat_starting_station = math.radians(stops_cleaned[stops_cleaned['stop_name'] == starting_station]['stop_lat'].iloc[0])
  lon_starting_station = math.radians(stops_cleaned[stops_cleaned['stop_name'] == starting_station]['stop_lon'].iloc[0])
  lat_ending_station = math.radians(stops_cleaned[stops_cleaned['stop_name'] == ending_station]['stop_lat'].iloc[0])
  lon_ending_station = math.radians(stops_cleaned[stops_cleaned['stop_name'] == ending_station]['stop_lon'].iloc[0])

  # To calculate the change in coordinates
  dlon = lon_ending_station - lon_starting_station
  dlat = lat_ending_station - lat_starting_station

  # To use the Haversine formula to get the distance in kilometers between the starting_station and the ending_station
  a = math.sin(dlat / 2)**2 + math.cos(lat_starting_station) * math.cos(lat_ending_station) * math.sin(dlon / 2)**2
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

  # To calculate the distance
  distance = R * c

  # To add the distance in the absolute_distance list
  absolute_distance.append(distance)

# To add the absolute_distance list as a column to the route_characteristics dataset
route_characteristics['absolute_distance'] = absolute_distance
route_characteristics.head(2)

In [ ]:
''' To calculate the relative distance of the stop sequence of the route by only taking the starting and the ending station into account'''
# The relative distance of a route is the distance of the route divided by the distance of the route with the longest distance

relative_distance = []

# To calculate the route which stop sequence has the greatest absolute distance
max_distance = route_characteristics['absolute_distance'].max()

# To calculate the relative distance of each route
for abs_distance in route_characteristics['absolute_distance']:
  rel_distance = abs_distance / max_distance
  relative_distance.append(rel_distance)

# To add the relative_distance list as a column to the route_characteristics dataset
route_characteristics['relative_distance'] = relative_distance
route_characteristics.head(2)

In [ ]:
route_characteristics.dtypes

* Clustering of the routes

In [ ]:
#Construction of the numpy array from the route_characteristics dataset
arr_route_characteristics = route_characteristics.loc[:, ['avg_degree','max_degree','avg_clustering','max_clustering','avg_closeness_centrality','max_closeness_centrality','avg_betweenness_centrality','max_betweenness_centrality', 'len_stop_sequence', 'absolute_distance', 'relative_distance']].values
arr_route_characteristics

In [ ]:
#Normalization of the arr_route_characteristics
scaler = StandardScaler()
norm_arr_route_characteristics = scaler.fit_transform(arr_route_characteristics)
norm_arr_route_characteristics

In [ ]:
#Find the optimal number of clusters by means of the elbow method
wcss=[]

#a loop to fit the k-means algorithm (of the sklearn library) to the data
#The withing cluster sum of squares is calculated and stored in the wcss list
#init parameter indicates the random initialization method
#the max_iter parameter indicates the maximum number of iterations there can be to find the clusters when running the k-means algorithm
#300 is the default value for this
#The n_init parameter indicates the number of times the k-means algorithm will run with a different initial centroid
#The next line indicates that the k-means algorithms fits to the route_characteristics dataset
#The k-means inertia_ attribute indicates the sum of squared distances of samples to their closest cluster center
for i in range(1,11): 
     kmeans = KMeans(n_clusters=i, init ='k-means++', max_iter=300,  n_init=10,random_state=0 )
     kmeans.fit(norm_arr_route_characteristics)
     wcss.append(kmeans.inertia_)

In [ ]:
#Calculate the optimal number of clusters
kl = KneeLocator(range(1, 11), wcss, curve="convex", direction="decreasing")
kl.elbow

In [ ]:
#Plot the elbow graph
plt.plot(range(1,11),wcss)
plt.title('The Elbow Method Graph')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')

kl.plot_knee()

plt.show()

In [ ]:
#Applying k-means algorithm to the route_characteristics dataset.
kmeans = KMeans(n_clusters=kl.elbow, init ='k-means++', max_iter=300, n_init=10,random_state=0 )

#By means of the fit predict method for each observation the cluster it belongs to is returned. 
y_kmeans = kmeans.fit_predict(norm_arr_route_characteristics)
y_kmeans

In [ ]:
#Creation of a dictionary to know which route belongs to which cluster
#Put the routeID column as the index of the dictionary
cl_prediction = pd.Series(index=route_characteristics['route_id']).to_dict()

#select the first key
#list(cl_prediction.keys())[0]

#Put the array values as the values of the dictionary
for i in range(len(y_kmeans)):
  k = list(cl_prediction.keys())[i]
  cl_prediction[k] = y_kmeans[i]

cl_prediction

In [ ]:
#Centers of the clusters
kmeans.cluster_centers_

In [ ]:
#Principal component analysis (PCA) to reduce the number of dimensions
preprocessor = Pipeline(
    [
        ("scaler", MinMaxScaler()),
        ("pca", PCA(n_components=2, random_state=33)),
    ])

clusterer = Pipeline(
    [(
        "kmeans",
        KMeans(
            n_clusters= kl.elbow,
            init="k-means++",
            n_init=50,
            max_iter=500,
            random_state=33,
            ),
        ),
        ])

pipe = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("clusterer", clusterer)
    ]
)

pipe.fit(norm_arr_route_characteristics)
preprocessed_data = pipe["preprocessor"].transform(norm_arr_route_characteristics)
predicted_labels = pipe["clusterer"]["kmeans"].labels_

In [ ]:
#Voor centroid values weer te geven
preprocessor2 = Pipeline(
    [
        ("scaler", MinMaxScaler()),
        ("pca", PCA(n_components=2, random_state=33)),
    ])

clusterer2 = Pipeline(
    [(
        "kmeans",
        KMeans(
            n_clusters= kl.elbow,
            init="k-means++",
            n_init=50,
            max_iter=500,
            random_state=33,
            ),
        ),
        ])

pipe2 = Pipeline(
    [
        ("preprocessor2", preprocessor2),
        ("clusterer2", clusterer2)
    ]
)

pipe2.fit(kmeans.cluster_centers_)
preprocessed_data_2 = pipe2["preprocessor2"].transform(kmeans.cluster_centers_)
predicted_labels_2 = pipe2["clusterer2"]["kmeans"].labels_

In [ ]:
#Plot the centroids
cl_routes_2 = pd.DataFrame(
    pipe2["preprocessor2"].transform(kmeans.cluster_centers_),
    columns=["Principal component 1", "Principal component 2"],
)

#cl_routes_2["predicted_cluster"] = pipe2["clusterer2"]["kmeans"].labels_

plt.style.use("fivethirtyeight")
plt.figure(figsize=(8, 8))

scat = sns.scatterplot(
    "Principal component 1",
    "Principal component 2",
    s=300,
    data=cl_routes_2,
    color=".2", 
)

scat.set_title("Clusters of the Belgian railway centroids")

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

plt.show()

In [ ]:
#Plot the clusters
cl_routes = pd.DataFrame(
    pipe["preprocessor"].transform(norm_arr_route_characteristics),
    columns=["Principal component 1", "Principal component 2"],
)

cl_routes["predicted_cluster"] = pipe["clusterer"]["kmeans"].labels_

cl_routes_2 = pd.DataFrame(
    pipe2["preprocessor2"].transform(kmeans.cluster_centers_),
    columns=["Principal component 1", "Principal component 2"],
)

cl_routes_2["predicted_cluster"] = pipe2["clusterer2"]["kmeans"].labels_

plt.style.use("fivethirtyeight")
plt.figure(figsize=(8, 8))

scat = sns.scatterplot(
    "Principal component 1",
    "Principal component 2",
    s=50,
    data=cl_routes,
    hue="predicted_cluster",
    palette="Set2",
)

scat = sns.scatterplot(
    "Principal component 1",
    "Principal component 2",
    s=300,
    data=cl_routes_2,
    color=".2", 
)

scat.set_title("Clusters of the Belgian railway routes")

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

plt.show()

In [ ]:
#Plot the clusters
cl_routes = pd.DataFrame(
    pipe["preprocessor"].transform(norm_arr_route_characteristics),
    columns=["Principal component 1", "Principal component 2"],
)

cl_routes["predicted_cluster"] = pipe["clusterer"]["kmeans"].labels_

plt.style.use("fivethirtyeight")
plt.figure(figsize=(8, 8))

scat = sns.scatterplot(
    "Principal component 1",
    "Principal component 2",
    s=50,
    data=cl_routes,
    hue="predicted_cluster",
    palette="Set2",
)

scat.set_title("Clusters of the Belgian railway routes")

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

plt.show()

# Testen van de osmnx package

In [ ]:
ox.plot_graph(ox.graph_from_place('Namur, Belgium'))

In [ ]:
B = ox.graph_from_place('Belgium',
                        retain_all=False, truncate_by_edge=True, simplify=True,
                        custom_filter='["railway"~"rail"]')

fig, ax = ox.plot_graph(B, node_size=0, edge_color='w', edge_linewidth=0.2)